In [1]:
import logging
import warnings
import os
import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Tuple, Optional, Dict, List
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks  # FIXED: Added missing import
from scipy.stats import pearsonr
import joblib

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

# Enhanced Configuration with Research-Focused Parameters
BASE_PATH = Path("/kaggle/input/dataset10/Dataset")
USE_WAIST = False
MIN_IMU_SAMPLES = 50
NOISE_STD = 0.01
DEBUG_DIR = Path("debug_output")
KFOLD_SPLITS = 5

# Research-Enhanced Activity Parameters
ACTIVITY_PARAMS = {
    "walking": {
        "TARGET_LEN": 100, "SEQ_LENGTH": 10, "LSTM_UNITS": 64, "EPOCHS": 100,
        "LEARNING_RATE": 0.001, "DROPOUT": 0.3, "BATCH_SIZE": 32,
        "MODEL_TYPE": "standard", "N_FEATURES": 6, "ENSEMBLE_MODELS": 5,
        "FORCE_THRESHOLD": 40, "ACTIVITY_INTENSITY": "low"
    },
    "running": {
        "TARGET_LEN": 50, "SEQ_LENGTH": 10, "LSTM_UNITS": 128, "EPOCHS": 100,
        "LEARNING_RATE": 0.001, "DROPOUT": 0.3, "BATCH_SIZE": 32,
        "MODEL_TYPE": "enhanced", "N_FEATURES": 6, "ENSEMBLE_MODELS": 5,
        "FORCE_THRESHOLD": 100, "ACTIVITY_INTENSITY": "high"
    },
    "jogging": {
        "TARGET_LEN": 75, "SEQ_LENGTH": 12, "LSTM_UNITS": 128, "EPOCHS": 100,
        "LEARNING_RATE": 0.001, "DROPOUT": 0.3, "BATCH_SIZE": 32,
        "MODEL_TYPE": "enhanced", "N_FEATURES": 6, "ENSEMBLE_MODELS": 5,
        "FORCE_THRESHOLD": 60, "ACTIVITY_INTENSITY": "medium"
    },
    "heel": {
        "TARGET_LEN": 30, "SEQ_LENGTH": 8, "LSTM_UNITS": 64, "EPOCHS": 200,
        "LEARNING_RATE": 0.0001, "DROPOUT": 0.5, "BATCH_SIZE": 8,
        "USE_POLYNOMIAL_FEATURES": True, "POLYNOMIAL_DEGREE": 2,
        "USE_FEATURE_SELECTION": True, "SELECT_K_BEST": 10,
        "USE_IMPACT_DETECTION": True, "CORRELATION_THRESHOLD": 0.03,
        "EXTREME_REGULARIZATION": True, "MODEL_TYPE": "robust", 
        "N_FEATURES": 10, "ENSEMBLE_MODELS": 5,
        "FORCE_THRESHOLD": 200, "ACTIVITY_INTENSITY": "impact"
    },
    "drop": {
        "TARGET_LEN": 40, "SEQ_LENGTH": 10, "LSTM_UNITS": 64, "EPOCHS": 200,
        "LEARNING_RATE": 0.0001, "DROPOUT": 0.5, "BATCH_SIZE": 8,
        "USE_POLYNOMIAL_FEATURES": True, "POLYNOMIAL_DEGREE": 2,
        "USE_FEATURE_SELECTION": True, "SELECT_K_BEST": 10,
        "USE_IMPACT_DETECTION": True, "CORRELATION_THRESHOLD": 0.03,
        "EXTREME_REGULARIZATION": True, "MODEL_TYPE": "robust", 
        "N_FEATURES": 10, "ENSEMBLE_MODELS": 5,
        "FORCE_THRESHOLD": 150, "ACTIVITY_INTENSITY": "impact"
    }
}

ACTIVITIES = ["walking", "running", "jogging", "heel", "drop"]

class EnhancedDataProcessor:
    """Enhanced data processor for comprehensive GRF estimation research."""
    
    def __init__(self, base_path: str, use_waist: bool = True, research_mode: bool = True):
        self.base_path = Path(base_path)
        self.use_waist = use_waist
        self.research_mode = research_mode
        self.processed_data = {activity: {
            "X": [], "y": [], "trial_ids": [], "participant_ids": [],
            "metadata": [], "arm_swing_features": []
        } for activity in ACTIVITIES}
        
        # Research analytics storage
        self.data_quality_metrics = {}
        self.correlation_analysis = {}
        self.arm_swing_analysis = {}
        
        # ADDED: Manifest storage for statistical analysis
        self.manifest_data = []
        
        # Create enhanced directory structure
        self._setup_directories()
        
    def _setup_directories(self):
        """Setup comprehensive directory structure for research outputs."""
        dirs = [
            "debug_output", "models", "results", "visualizations", 
            "correlation_analysis", "arm_swing_analysis", "publication_figures",
            "comprehensive_force_analysis"  # ADDED: For manifest output
        ]
        for dir_name in dirs:
            for activity in ACTIVITIES:
                os.makedirs(Path(dir_name) / activity, exist_ok=True)
    
    def extract_participant_info(self, folder_name: str) -> Tuple[Optional[int], Optional[int], Optional[float], Optional[float]]:
        """FIXED: Extract participant information with enhanced error handling."""
        patterns = [
            r'P(\d+)-(\d+)-([\d.]+)-([\d.]+)',  # P1-1984-175.0-70.0 format
            r'P(\d+)_(\d+)_([\d.]+)_([\d.]+)',  # Alternative underscore pattern  
            r'participant_(\d+)_(\d+)_([\d.]+)_([\d.]+)'  # Full word pattern
        ]
        
        for pattern in patterns:
            match = re.match(pattern, folder_name)
            if match:
                pid = int(match.group(1))
                trial_or_year = int(match.group(2))
                height = float(match.group(3))
                weight = float(match.group(4))
                logger.info(f"Parsed info - P{pid:02d}, Year/Trial: {trial_or_year}, Height: {height}cm, Weight: {weight}kg")
                return (pid, trial_or_year, height, weight)
        
        logger.warning(f"Could not parse participant info from: {folder_name}")
        return (None, None, None, None)
    
    def infer_activity(self, filename: str) -> str:
        """Enhanced activity inference with better pattern matching."""
        filename_lower = filename.lower()
        
        # Priority order: most specific first
        activity_patterns = {
            'heel': ['heel', 'heel_drop', 'heeldrop'],
            'drop': ['drop', 'step_drop', 'stepdrop'],
            'running': ['run', 'running'],
            'jogging': ['jog', 'jogging'], 
            'walking': ['walk', 'walking']
        }
        
        for activity, patterns in activity_patterns.items():
            if any(pattern in filename_lower for pattern in patterns):
                return activity
                
        return "unknown"
    
    def calculate_data_quality_metrics(self, imu_data: np.ndarray, force_data: np.ndarray, 
                                     activity: str) -> Dict:
        """Calculate comprehensive data quality metrics."""
        metrics = {}
        
        # Signal quality metrics
        metrics['imu_snr'] = self._calculate_snr(imu_data)
        metrics['force_snr'] = self._calculate_snr(force_data)
        
        # Missing data analysis
        metrics['imu_missing_pct'] = np.isnan(imu_data).sum() / imu_data.size * 100
        metrics['force_missing_pct'] = np.isnan(force_data).sum() / force_data.size * 100
        
        # Signal characteristics
        metrics['imu_dynamic_range'] = np.ptp(imu_data, axis=0)  # Peak-to-peak per channel
        metrics['force_peak'] = np.max(force_data)
        metrics['force_mean'] = np.mean(force_data)
        
        # Temporal characteristics
        metrics['trial_duration'] = len(force_data) / 1000.0  # Assuming 1000Hz force plate
        metrics['imu_sampling_rate'] = len(imu_data) / metrics['trial_duration']
        
        return metrics
    
    def _calculate_snr(self, signal_data: np.ndarray) -> float:
        """Calculate signal-to-noise ratio."""
        if signal_data.ndim == 1:
            signal_power = np.var(signal_data)
            noise_power = np.var(np.diff(signal_data))  # High-freq noise estimation
        else:
            signal_power = np.mean(np.var(signal_data, axis=0))
            noise_power = np.mean(np.var(np.diff(signal_data, axis=0), axis=0))
        
        return 10 * np.log10(signal_power / (noise_power + 1e-10))
    
    def extract_arm_swing_features(self, imu_data: np.ndarray, activity: str) -> Dict:
        """Extract comprehensive arm swing characteristics for wrist sensors."""
        if not self.use_waist:  # Only for wrist-worn sensors
            features = {}
            
            # Basic arm swing metrics
            acc_x, acc_y, acc_z = imu_data[:, 0], imu_data[:, 1], imu_data[:, 2]
            gyro_x, gyro_y, gyro_z = imu_data[:, 3], imu_data[:, 4], imu_data[:, 5]
            
            # Swing amplitude analysis
            features['swing_range_x'] = np.ptp(acc_x)  # Anteroposterior swing
            features['swing_range_y'] = np.ptp(acc_y)  # Mediolateral swing
            features['swing_range_z'] = np.ptp(acc_z)  # Vertical swing
            
            # Swing frequency analysis
            swing_freq_x = self._estimate_swing_frequency(acc_x)
            swing_freq_y = self._estimate_swing_frequency(acc_y)
            features['swing_frequency_x'] = swing_freq_x
            features['swing_frequency_y'] = swing_freq_y
            
            # Coordination metrics
            features['xy_coordination'] = np.corrcoef(acc_x, acc_y)[0, 1]
            features['xz_coordination'] = np.corrcoef(acc_x, acc_z)[0, 1]
            
            # Rhythmic regularity
            features['swing_regularity'] = self._calculate_swing_regularity(acc_x)
            
            # Angular velocity characteristics (wrist rotation)
            features['rotation_intensity'] = np.sqrt(np.mean(gyro_x**2 + gyro_y**2 + gyro_z**2))
            
            # Activity-specific arm swing patterns
            if activity in ['walking', 'jogging', 'running']:
                features['gait_asymmetry'] = self._calculate_gait_asymmetry(acc_x)
                features['cadence_estimate'] = swing_freq_x * 2  # Steps per minute
            
            return features
        else:
            return {}
    
    def _estimate_swing_frequency(self, signal: np.ndarray) -> float:
        """Estimate dominant frequency in arm swing."""
        # Remove DC component
        signal_ac = signal - np.mean(signal)
        
        # FFT analysis
        fft = np.fft.fft(signal_ac)
        freqs = np.fft.fftfreq(len(signal), d=1/100)  # Assuming 100Hz sampling
        
        # Find dominant frequency (exclude DC)
        positive_freqs = freqs[1:len(freqs)//2]
        magnitude = np.abs(fft[1:len(fft)//2])
        
        if len(magnitude) == 0:
            return 0.0
            
        dominant_freq_idx = np.argmax(magnitude)
        return positive_freqs[dominant_freq_idx]
    
    def _calculate_swing_regularity(self, signal: np.ndarray) -> float:
        """Calculate regularity of arm swing patterns."""
        # Autocorrelation analysis
        signal_norm = (signal - np.mean(signal)) / np.std(signal)
        autocorr = np.correlate(signal_norm, signal_norm, mode='full')
        autocorr = autocorr[len(autocorr)//2:]
        
        # Find peak autocorrelation (excluding lag 0)
        if len(autocorr) > 10:
            peak_corr = np.max(autocorr[5:min(50, len(autocorr))])
            return peak_corr / autocorr[0]  # Normalize by zero-lag correlation
        return 0.0
    
    def _calculate_gait_asymmetry(self, signal: np.ndarray) -> float:
        """FIXED: Calculate gait asymmetry from arm swing."""
        try:
            # Find swing peaks (positive and negative)
            peaks_pos, _ = find_peaks(signal, height=np.std(signal))
            peaks_neg, _ = find_peaks(-signal, height=np.std(signal))
            
            if len(peaks_pos) > 0 and len(peaks_neg) > 0:
                avg_pos = np.mean(signal[peaks_pos])
                avg_neg = np.mean(signal[peaks_neg])
                asymmetry = abs(avg_pos - abs(avg_neg)) / (avg_pos + abs(avg_neg))
                return asymmetry
            return 0.0
        except Exception as e:
            logger.warning(f"Error calculating gait asymmetry: {e}")
            return 0.0
    
    def perform_correlation_analysis(self, imu_data: np.ndarray, force_data: np.ndarray, 
                                   activity: str) -> Dict:
        """Comprehensive correlation analysis between IMU and force data."""
        correlations = {}
        
        # Individual channel correlations
        imu_channels = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
        for i, channel in enumerate(imu_channels):
            if i < imu_data.shape[1]:
                corr, p_value = pearsonr(imu_data[:, i], force_data)
                correlations[channel] = {'correlation': corr, 'p_value': p_value}
        
        # Derived feature correlations
        acc_magnitude = np.linalg.norm(imu_data[:, :3], axis=1)
        corr_mag, p_mag = pearsonr(acc_magnitude, force_data)
        correlations['acc_magnitude'] = {'correlation': corr_mag, 'p_value': p_mag}
        
        # Gyro magnitude
        if imu_data.shape[1] >= 6:
            gyro_magnitude = np.linalg.norm(imu_data[:, 3:6], axis=1)
            corr_gyro_mag, p_gyro_mag = pearsonr(gyro_magnitude, force_data)
            correlations['gyro_magnitude'] = {'correlation': corr_gyro_mag, 'p_value': p_gyro_mag}
        
        # Time-lagged correlations (important for arm swing analysis)
        max_lag = 10  # ±10 samples
        lag_correlations = []
        for lag in range(-max_lag, max_lag + 1):
            if lag == 0:
                continue
            
            if lag > 0:
                imu_lagged = acc_magnitude[:-lag]
                force_lagged = force_data[lag:]
            else:
                imu_lagged = acc_magnitude[-lag:]
                force_lagged = force_data[:lag]
            
            if len(imu_lagged) > 10:  # Minimum samples for meaningful correlation
                corr_lag, _ = pearsonr(imu_lagged, force_lagged)
                lag_correlations.append({'lag': lag, 'correlation': corr_lag})
        
        # Find optimal lag
        if lag_correlations:
            best_lag = max(lag_correlations, key=lambda x: abs(x['correlation']))
            correlations['optimal_lag'] = best_lag
        
        return correlations
    
    def preprocess_trial(self, folder_path: Path, fp_file: str, watch_file: str, 
                        activity: str, weight: float, participant_id: int) -> Tuple[Optional[np.ndarray], Optional[np.ndarray], Optional[Dict]]:
        """Enhanced trial preprocessing with comprehensive analysis."""
        target_len = ACTIVITY_PARAMS[activity]["TARGET_LEN"]
        logger.info(f"🔍 Processing {activity} trial {fp_file} (P{participant_id:02d})")
        
        fp_path = folder_path / fp_file
        watch_path = folder_path / watch_file
        
        try:
            # Process Force Plate data
            df_fp = pd.read_csv(fp_path, skiprows=26)
            df_fp.columns = [c.strip() for c in df_fp.columns]
            force_z_col = next(c for c in df_fp.columns if "force" in c.lower() and "z" in c.lower())
            force_z = pd.to_numeric(df_fp[force_z_col], errors='coerce')
            
            # Activity-specific force processing
            force_threshold = ACTIVITY_PARAMS[activity].get("FORCE_THRESHOLD", 40)
            
            if activity in ['heel', 'drop']:
                # Impact activity processing
                valid_force = force_z[force_z > force_threshold]
                if len(valid_force) < 5:
                    raise ValueError(f"No valid impact detected (samples: {len(valid_force)})")
                
                peak_idx = force_z.idxmax()
                start_idx = max(0, peak_idx - target_len // 2)
                end_idx = min(len(force_z), start_idx + target_len)
                impact_window = force_z[start_idx:end_idx]
                y = self.resample_series(impact_window, target_len) / (weight * 9.81)
                
            else:
                # Locomotion activity processing
                stance = self.extract_stance_force(force_z, force_threshold)
                if stance is None or len(stance) < 5:
                    raise ValueError(f"No valid stance phase detected")
                y = self.resample_series(stance, target_len) / (weight * 9.81)
            
            if y is None:
                raise ValueError("Resampling failed for force plate")
            
            # Process IMU data
            df_watch = pd.read_csv(watch_path)
            df_watch.columns = [c.strip() for c in df_watch.columns]
            
            imu_cols = ['userAccelerationX', 'userAccelerationY', 'userAccelerationZ',
                       'rotationRateX', 'rotationRateY', 'rotationRateZ']
            
            if not all(col in df_watch.columns for col in imu_cols):
                raise ValueError(f"Missing IMU columns: {set(imu_cols) - set(df_watch.columns)}")
            
            if len(df_watch) < MIN_IMU_SAMPLES:
                raise ValueError(f"IMU data too short: {len(df_watch)} samples")
            
            # Extract and align IMU segment
            imu_data = df_watch[imu_cols].to_numpy()
            segment_len = target_len
            mid = len(imu_data) // 2
            start = max(0, mid - segment_len // 2)
            end = min(len(imu_data), start + segment_len)
            segment = imu_data[start:end]
            
            if len(segment) < segment_len:
                pad_len = segment_len - len(segment)
                segment = np.pad(segment, ((0, pad_len), (0, 0)), mode='edge')
            
            # Calculate data quality metrics
            quality_metrics = self.calculate_data_quality_metrics(segment, y, activity)
            
            # Perform correlation analysis
            correlation_results = self.perform_correlation_analysis(segment, y, activity)
            
            # Extract arm swing features (for wrist sensors)
            arm_swing_features = self.extract_arm_swing_features(segment, activity)
            
            # Enhanced preprocessing for impact activities
            if activity in ['heel', 'drop']:
                logger.info(f"  🔧 Enhanced preprocessing for {activity}...")
                
                # Impact detection and feature extraction
                impact_features, impact_stats = self.detect_impact_events(segment)
                
                # Combine features
                enhanced_features = np.column_stack([
                    segment,  # Original 6 IMU features
                    impact_features,  # 6 impact features
                    segment[:, 0] * segment[:, 1],  # AccX * AccY interaction
                    segment[:, 0] * segment[:, 2]   # AccX * AccZ interaction
                ])
                
                # Apply polynomial features if enabled
                if ACTIVITY_PARAMS[activity].get("USE_POLYNOMIAL_FEATURES", False):
                    degree = ACTIVITY_PARAMS[activity]["POLYNOMIAL_DEGREE"]
                    poly = PolynomialFeatures(degree=degree, include_bias=False, interaction_only=False)
                    enhanced_features = poly.fit_transform(enhanced_features)
                    logger.info(f"    📊 Polynomial features (degree {degree}): expanded to {enhanced_features.shape[1]}")
                
                # Feature selection if enabled
                if ACTIVITY_PARAMS[activity].get("USE_FEATURE_SELECTION", False):
                    k = ACTIVITY_PARAMS[activity]["SELECT_K_BEST"]
                    selector = SelectKBest(score_func=f_regression, k=k)
                    enhanced_features = selector.fit_transform(enhanced_features, y)
                    logger.info(f"    🎯 Feature selection: {k} features selected")
                
                # Robust scaling
                final_features = RobustScaler().fit_transform(enhanced_features)
                
            else:
                # Standard preprocessing for locomotion activities
                final_features = StandardScaler().fit_transform(segment)
            
            # Calculate peak GRF for manifest
            peak_bw = np.max(y)
            
            # Compile metadata
            metadata = {
                'quality_metrics': quality_metrics,
                'correlation_results': correlation_results,
                'arm_swing_features': arm_swing_features,
                'participant_id': participant_id,
                'activity': activity,
                'trial_file': fp_file,
                'sensor_location': 'waist' if self.use_waist else 'wrist',
                'peak_bw': peak_bw,  # ADDED: For manifest
                'weight_kg': weight   # ADDED: For manifest
            }
            
            # ADDED: Store manifest data
            self.manifest_data.append({
                'participant_id': participant_id,
                'activity': activity,
                'file': fp_file,
                'peak_bw': peak_bw,
                'included': 1,  # Successfully processed
                'weight_kg': weight,
                'processing_method': 'enhanced_biomech',
                'off_plate': False,
                'speed_fail': False,
                'imu_gap': False,
                'sync_warning': False,
                'reason': 'success'
            })
            
            logger.info(f"  ✅ Final {activity} features: {final_features.shape}")
            logger.info(f"    📊 X range: [{final_features.min():.3f}, {final_features.max():.3f}]")
            logger.info(f"    🎯 y range: [{y.min():.3f}, {y.max():.3f}] BW (peak: {peak_bw:.3f})")
            
            return final_features, y, metadata
            
        except Exception as e:
            logger.error(f"❌ Error processing {activity} trial {fp_file}: {e}")
            
            # ADDED: Store failed trial in manifest
            self.manifest_data.append({
                'participant_id': participant_id,
                'activity': activity,
                'file': fp_file,
                'peak_bw': None,
                'included': 0,  # Failed processing
                'weight_kg': weight,
                'processing_method': 'enhanced_biomech',
                'off_plate': False,
                'speed_fail': False,
                'imu_gap': False,
                'sync_warning': True,
                'reason': str(e)
            })
            
            return None, None, None
    
    def extract_stance_force(self, force_z: pd.Series, threshold: float = 40) -> Optional[pd.Series]:
        """Extract stance phase from force data with enhanced detection."""
        stance = force_z[force_z > threshold]
        if len(stance) == 0:
            return None
        start, end = stance.index[0], stance.index[-1]
        return force_z[start:end+1]
    
    def resample_series(self, series: pd.Series, target_len: int = 100) -> Optional[np.ndarray]:
        """Resample series to target length with improved interpolation."""
        if len(series) < 2:
            return None
        return np.interp(np.linspace(0, len(series)-1, target_len), np.arange(len(series)), series)
    
    def detect_impact_events(self, imu_data: np.ndarray, threshold: float = 2.0) -> Tuple[np.ndarray, Dict]:
        """Enhanced impact detection for heel/drop activities."""
        # Calculate acceleration magnitude
        acc_mag = np.sqrt(imu_data[:, 0]**2 + imu_data[:, 1]**2 + imu_data[:, 2]**2)
        
        # Detect impacts above threshold
        impact_indices = np.where(acc_mag > threshold)[0]
        
        # Calculate comprehensive impact statistics
        impact_stats = {
            'n_impacts': len(impact_indices),
            'max_impact': np.max(acc_mag) if len(acc_mag) > 0 else 0,
            'mean_impact': np.mean(acc_mag[impact_indices]) if len(impact_indices) > 0 else 0,
            'impact_rate': len(impact_indices) / len(imu_data) if len(imu_data) > 0 else 0,
            'impact_variance': np.var(acc_mag[impact_indices]) if len(impact_indices) > 0 else 0
        }
        
        # Create enhanced impact features
        impact_features = np.zeros((len(imu_data), 6))
        impact_features[:, 0] = acc_mag  # Magnitude
        impact_features[:, 1] = acc_mag ** 2  # Magnitude squared
        impact_features[:, 2] = acc_mag ** 3  # Magnitude cubed
        
        # Impact proximity feature
        impact_proximity = np.zeros_like(acc_mag)
        for idx in impact_indices:
            start, end = max(0, idx-5), min(len(imu_data), idx+6)
            impact_proximity[start:end] = 1
        impact_features[:, 3] = impact_proximity
        
        # Acceleration derivatives
        impact_features[:, 4] = np.gradient(acc_mag)  # First derivative
        impact_features[:, 5] = np.gradient(np.gradient(acc_mag))  # Second derivative
        
        logger.info(f"    💥 Impact detection: {impact_stats['n_impacts']} events, max: {impact_stats['max_impact']:.3f}g")
        
        return impact_features, impact_stats
    
    def load_all_data(self) -> bool:
        """Load all data with enhanced analytics and quality assessment."""
        logger.info(f"🔄 Loading data from {self.base_path}")
        logger.info(f"📊 Sensor location: {'Waist' if self.use_waist else 'Wrist'}")
        logger.info(f"🔬 Research mode: {'Enabled' if self.research_mode else 'Disabled'}")
        
        for folder in os.listdir(self.base_path):
            folder_path = self.base_path / folder
            if not os.path.isdir(folder_path):
                continue
                
            pid, _, height, weight = self.extract_participant_info(folder)
            if pid is None:
                continue
                
            logger.info(f"\n👤 Processing Participant {pid} (Weight: {weight}kg, Height: {height}cm)")
            
            files = os.listdir(folder_path)
            fp_files = [f for f in files if any(f.startswith(a) for a in ['walk', 'jog', 'run', 'heel', 'drop'])]
            watch_files = [f for f in files if ('Waist' if self.use_waist else 'Wrist') in f]
            
            for fp_file in fp_files:
                activity = self.infer_activity(fp_file)
                if activity not in ACTIVITIES:
                    continue
                    
                trial_id = fp_file.split("_")[0][-4:]
                match = [w for w in watch_files if f"ID_{int(trial_id)}" in w]
                if not match:
                    continue
                    
                X, y, metadata = self.preprocess_trial(folder_path, fp_file, match[0], activity, weight, pid)
                if X is not None and y is not None and metadata is not None:
                    self.processed_data[activity]["X"].append(X)
                    self.processed_data[activity]["y"].append(y)
                    self.processed_data[activity]["trial_ids"].append(fp_file)
                    self.processed_data[activity]["participant_ids"].append(pid)
                    self.processed_data[activity]["metadata"].append(metadata)
                    
                    # Store research analytics
                    if self.research_mode:
                        self._store_analytics(activity, metadata)
        
        # ADDED: Save manifest for statistical analysis
        self._save_manifest()
        
        # Generate comprehensive summary
        self._generate_data_summary()
        
        total_samples = sum(len(self.processed_data[activity]["X"]) for activity in ACTIVITIES)
        logger.info(f"\n✅ Data loading complete! Total trials: {total_samples}")
        
        return total_samples > 0
    
    def _save_manifest(self):
        """ADDED: Save manifest for statistical wrapper integration."""
        if not self.manifest_data:
            logger.warning("No manifest data to save")
            return
        
        manifest_df = pd.DataFrame(self.manifest_data)
        manifest_path = Path("comprehensive_force_analysis") / "manifest.csv"
        manifest_path.parent.mkdir(exist_ok=True)
        
        manifest_df.to_csv(manifest_path, index=False)
        logger.info(f"📊 Manifest saved: {len(self.manifest_data)} trials, {sum(manifest_df['included'])} successful")
        
        # Also save normalized version for compatibility
        normalized_path = Path("comprehensive_force_analysis") / "manifest_normalized.csv"
        manifest_df.to_csv(normalized_path, index=False)
    
    def _store_analytics(self, activity: str, metadata: Dict):
        """Store analytics for research analysis."""
        if activity not in self.data_quality_metrics:
            self.data_quality_metrics[activity] = []
            self.correlation_analysis[activity] = []
            self.arm_swing_analysis[activity] = []
        
        self.data_quality_metrics[activity].append(metadata['quality_metrics'])
        self.correlation_analysis[activity].append(metadata['correlation_results'])
        if metadata['arm_swing_features']:
            self.arm_swing_analysis[activity].append(metadata['arm_swing_features'])
    
    def _generate_data_summary(self):
        """Generate comprehensive data summary for research reporting."""
        logger.info("\n" + "="*80)
        logger.info("📋 COMPREHENSIVE DATA SUMMARY")
        logger.info("="*80)
        
        for activity in ACTIVITIES:
            n_trials = len(self.processed_data[activity]["X"])
            if n_trials == 0:
                continue
                
            logger.info(f"\n🏃 {activity.upper()} Activity:")
            logger.info(f"  📊 Trials: {n_trials}")
            logger.info(f"  👥 Participants: {len(set(self.processed_data[activity]['participant_ids']))}")
            
            if self.research_mode and activity in self.data_quality_metrics:
                # Quality metrics summary
                quality_data = self.data_quality_metrics[activity]
                avg_snr = np.mean([q['imu_snr'] for q in quality_data])
                avg_force_peak = np.mean([q['force_peak'] for q in quality_data])
                
                logger.info(f"  📶 Avg IMU SNR: {avg_snr:.1f} dB")
                logger.info(f"  ⚡ Avg Peak Force: {avg_force_peak:.1f} N")
                
                # Correlation analysis summary
                if activity in self.correlation_analysis:
                    corr_data = self.correlation_analysis[activity]
                    best_correlations = []
                    for corr in corr_data:
                        best_corr = max(corr.items(), key=lambda x: abs(x[1]['correlation']) if isinstance(x[1], dict) else 0)
                        if isinstance(best_corr[1], dict):
                            best_correlations.append(best_corr[1]['correlation'])
                    
                    if best_correlations:
                        logger.info(f"  🔗 Avg Best Correlation: {np.mean(best_correlations):.3f}")
                
                # Arm swing analysis (for wrist sensors)
                if not self.use_waist and activity in self.arm_swing_analysis and self.arm_swing_analysis[activity]:
                    swing_data = self.arm_swing_analysis[activity]
                    avg_swing_range = np.mean([s['swing_range_x'] for s in swing_data if 'swing_range_x' in s])
                    avg_coordination = np.mean([s['xy_coordination'] for s in swing_data if 'xy_coordination' in s and not np.isnan(s['xy_coordination'])])
                    
                    logger.info(f"  🤲 Avg Arm Swing Range: {avg_swing_range:.3f}g")
                    logger.info(f"  🔄 Avg XY Coordination: {avg_coordination:.3f}")

    # Include all the visualization and export methods from your original code...
    # (keeping the rest of your methods unchanged)

    def generate_research_visualizations(self, save_path: Path = None):
        """Generate comprehensive visualizations for research publication."""
        if not self.research_mode:
            logger.warning("Research mode not enabled. Limited visualizations available.")
            return
        
        save_path = save_path or Path("visualizations")
        save_path.mkdir(exist_ok=True)
        
        logger.info("🎨 Generating research visualizations...")
        
        # 1. Data Quality Heatmap
        self._plot_data_quality_heatmap(save_path)
        
        # 2. Correlation Analysis
        self._plot_correlation_analysis(save_path)
        
        # 3. Arm Swing Analysis (if wrist sensor)
        if not self.use_waist:
            self._plot_arm_swing_analysis(save_path)
        
        # 4. Activity-specific distributions
        self._plot_activity_distributions(save_path)
        
        # 5. Sample signals visualization
        self._plot_sample_signals(save_path)
        
        logger.info(f"✅ Visualizations saved to {save_path}")
    
    def _plot_data_quality_heatmap(self, save_path: Path):
        """Create data quality heatmap across activities and participants."""
        plt.figure(figsize=(12, 8))
        
        quality_matrix = []
        activity_labels = []
        
        for activity in ACTIVITIES:
            if activity in self.data_quality_metrics and self.data_quality_metrics[activity]:
                quality_data = self.data_quality_metrics[activity]
                
                # Aggregate quality metrics
                snr_scores = [q['imu_snr'] for q in quality_data]
                force_peaks = [q['force_peak'] / (q.get('participant_weight', 70) * 9.81) for q in quality_data]  # Normalize by BW
                missing_data = [q['imu_missing_pct'] for q in quality_data]
                
                activity_quality = [
                    np.mean(snr_scores),
                    np.mean(force_peaks),
                    100 - np.mean(missing_data),  # Data completeness
                    len(quality_data)  # Sample size
                ]
                
                quality_matrix.append(activity_quality)
                activity_labels.append(activity.capitalize())
        
        if quality_matrix:
            quality_df = pd.DataFrame(
                quality_matrix,
                columns=['SNR (dB)', 'Peak Force (BW)', 'Data Completeness (%)', 'Sample Size'],
                index=activity_labels
            )
            
            # Normalize for visualization
            quality_normalized = quality_df.copy()
            for col in quality_df.columns:
                quality_normalized[col] = (quality_df[col] - quality_df[col].min()) / (quality_df[col].max() - quality_df[col].min())
            
            sns.heatmap(quality_normalized, annot=quality_df.values, fmt='.1f', 
                       cmap='viridis', cbar_kws={'label': 'Normalized Quality Score'})
            plt.title('Data Quality Assessment Across Activities')
            plt.tight_layout()
            plt.savefig(save_path / 'data_quality_heatmap.png', dpi=300, bbox_inches='tight')
            plt.close()
    
    def _plot_correlation_analysis(self, save_path: Path):
        """Visualize correlation patterns between IMU and GRF data."""
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for idx, activity in enumerate(ACTIVITIES):
            if idx >= len(axes):
                break
                
            if activity not in self.correlation_analysis or not self.correlation_analysis[activity]:
                axes[idx].text(0.5, 0.5, f'No data\nfor {activity}', ha='center', va='center')
                axes[idx].set_title(f'{activity.capitalize()}')
                continue
            
            corr_data = self.correlation_analysis[activity]
            
            # Extract correlation values for each IMU channel
            channels = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'acc_magnitude']
            correlations = {channel: [] for channel in channels}
            
            for trial_corr in corr_data:
                for channel in channels:
                    if channel in trial_corr and isinstance(trial_corr[channel], dict):
                        correlations[channel].append(trial_corr[channel]['correlation'])
            
            # Create box plot
            valid_correlations = [(ch, corrs) for ch, corrs in correlations.items() if corrs]
            if valid_correlations:
                channel_names, corr_values = zip(*valid_correlations)
                
                box_data = [np.abs(corrs) for corrs in corr_values]  # Use absolute correlations
                axes[idx].boxplot(box_data, labels=[ch.replace('_', '\n') for ch in channel_names])
                axes[idx].set_title(f'{activity.capitalize()}')
                axes[idx].set_ylabel('|Correlation with GRF|')
                axes[idx].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.savefig(save_path / 'correlation_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    def _plot_arm_swing_analysis(self, save_path: Path):
        """Visualize arm swing characteristics (wrist sensor only)."""
        if self.use_waist:
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # Collect arm swing data across all activities
        all_swing_data = []
        activity_labels = []
        
        for activity in ACTIVITIES:
            if activity in self.arm_swing_analysis and self.arm_swing_analysis[activity]:
                for swing_data in self.arm_swing_analysis[activity]:
                    all_swing_data.append(swing_data)
                    activity_labels.append(activity)
        
        if not all_swing_data:
            return
        
        swing_df = pd.DataFrame(all_swing_data)
        swing_df['activity'] = activity_labels
        
        # 1. Swing Range Analysis
        if 'swing_range_x' in swing_df.columns:
            axes[0, 0].boxplot([swing_df[swing_df.activity == act]['swing_range_x'].dropna().values 
                               for act in ACTIVITIES if act in activity_labels])
            axes[0, 0].set_xticklabels([act.capitalize() for act in ACTIVITIES if act in activity_labels])
            axes[0, 0].set_title('Arm Swing Range (X-axis)')
            axes[0, 0].set_ylabel('Range (g)')
        
        # 2. Swing Frequency Analysis
        if 'swing_frequency_x' in swing_df.columns:
            axes[0, 1].boxplot([swing_df[swing_df.activity == act]['swing_frequency_x'].dropna().values 
                               for act in ACTIVITIES if act in activity_labels])
            axes[0, 1].set_xticklabels([act.capitalize() for act in ACTIVITIES if act in activity_labels])
            axes[0, 1].set_title('Arm Swing Frequency')
            axes[0, 1].set_ylabel('Frequency (Hz)')
        
        # 3. Coordination Analysis
        if 'xy_coordination' in swing_df.columns:
            coord_data = swing_df['xy_coordination'].dropna()
            axes[1, 0].hist(coord_data, bins=20, alpha=0.7, edgecolor='black')
            axes[1, 0].set_title('XY Coordination Distribution')
            axes[1, 0].set_xlabel('Correlation Coefficient')
            axes[1, 0].set_ylabel('Frequency')
        
        # 4. Swing Regularity
        if 'swing_regularity' in swing_df.columns:
            axes[1, 1].boxplot([swing_df[swing_df.activity == act]['swing_regularity'].dropna().values 
                               for act in ACTIVITIES if act in activity_labels])
            axes[1, 1].set_xticklabels([act.capitalize() for act in ACTIVITIES if act in activity_labels])
            axes[1, 1].set_title('Swing Regularity')
            axes[1, 1].set_ylabel('Regularity Score')
        
        plt.tight_layout()
        plt.savefig(save_path / 'arm_swing_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    def _plot_activity_distributions(self, save_path: Path):
        """Plot GRF distributions for each activity."""
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for idx, activity in enumerate(ACTIVITIES):
            if idx >= len(axes):
                break
                
            y_data = self.processed_data[activity]["y"]
            if not y_data:
                axes[idx].text(0.5, 0.5, f'No data\nfor {activity}', ha='center', va='center')
                axes[idx].set_title(f'{activity.capitalize()}')
                continue
            
            # Flatten all trials for this activity
            all_grf = np.concatenate(y_data)
            
            # Create histogram
            axes[idx].hist(all_grf, bins=50, alpha=0.7, edgecolor='black', density=True)
            axes[idx].set_title(f'{activity.capitalize()} GRF Distribution')
            axes[idx].set_xlabel('GRF (Body Weight)')
            axes[idx].set_ylabel('Density')
            
            # Add statistics
            mean_grf = np.mean(all_grf)
            std_grf = np.std(all_grf)
            max_grf = np.max(all_grf)
            
            axes[idx].axvline(mean_grf, color='red', linestyle='--', label=f'Mean: {mean_grf:.2f}')
            axes[idx].text(0.02, 0.98, f'Max: {max_grf:.2f}\nStd: {std_grf:.2f}', 
                          transform=axes[idx].transAxes, verticalalignment='top',
                          bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            axes[idx].legend()
        
        plt.tight_layout()
        plt.savefig(save_path / 'grf_distributions.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    def _plot_sample_signals(self, save_path: Path, n_samples: int = 3):
        """Plot sample signals from each activity."""
        fig, axes = plt.subplots(len(ACTIVITIES), 2, figsize=(15, 3*len(ACTIVITIES)))
        
        for activity_idx, activity in enumerate(ACTIVITIES):
            X_data = self.processed_data[activity]["X"]
            y_data = self.processed_data[activity]["y"]
            
            if not X_data or not y_data:
                axes[activity_idx, 0].text(0.5, 0.5, f'No data for {activity}', ha='center', va='center')
                axes[activity_idx, 1].text(0.5, 0.5, f'No data for {activity}', ha='center', va='center')
                continue
            
            # Select random samples
            n_trials = min(n_samples, len(X_data))
            sample_indices = np.random.choice(len(X_data), n_trials, replace=False)
            
            # Plot IMU signals
            for i, idx in enumerate(sample_indices):
                X_sample = X_data[idx]
                
                # Plot first 3 channels (accelerometer)
                for ch in range(min(3, X_sample.shape[1])):
                    axes[activity_idx, 0].plot(X_sample[:, ch], alpha=0.7, 
                                             label=f'Trial {i+1}, Ch {ch+1}' if i == 0 else "")
                
                axes[activity_idx, 0].set_title(f'{activity.capitalize()} - IMU Signals')
                axes[activity_idx, 0].set_xlabel('Time (samples)')
                axes[activity_idx, 0].set_ylabel('Acceleration (normalized)')
                if activity_idx == 0:
                    axes[activity_idx, 0].legend()
                
                # Plot GRF signals
                y_sample = y_data[idx]
                axes[activity_idx, 1].plot(y_sample, alpha=0.7, label=f'Trial {i+1}' if i == 0 else "")
            
            axes[activity_idx, 1].set_title(f'{activity.capitalize()} - Ground Reaction Force')
            axes[activity_idx, 1].set_xlabel('Time (samples)')
            axes[activity_idx, 1].set_ylabel('GRF (Body Weight)')
            if activity_idx == 0:
                axes[activity_idx, 1].legend()
        
        plt.tight_layout()
        plt.savefig(save_path / 'sample_signals.png', dpi=300, bbox_inches='tight')
        plt.close()

    def create_comprehensive_force_visualizations(self, output_path: Path = None):
        """Create comprehensive force plate visualizations like reference figures."""
        output_path = output_path or Path("comprehensive_force_analysis")
        output_path.mkdir(exist_ok=True)
        
        logger.info("🎨 Creating comprehensive force plate visualizations...")
        
        # Prepare data for visualization
        self._prepare_force_visualization_data()
        
        # Create first figure - Overview Analysis (4-panel)
        self._create_overview_analysis_figure(output_path)
        
        # Create second figure - Distribution Analysis (4-panel)
        self._create_distribution_analysis_figure(output_path)
        
        # Print comprehensive summary
        self._print_force_analysis_summary()
        
        logger.info(f"✅ Comprehensive force visualizations saved to {output_path}")
    
    def _prepare_force_visualization_data(self):
        """Prepare data for comprehensive force visualization."""
        self.force_analysis_data = {
            'all_peak_forces': [],
            'all_activities': [],
            'all_participants': [],
            'all_body_weights': [],
            'all_processing_methods': [],
            'participant_data': {},
            'activity_data': {activity: [] for activity in ACTIVITIES}
        }
        
        # Extract data from processed results
        for activity in ACTIVITIES:
            if self.processed_data[activity]["y"]:
                for i, y_data in enumerate(self.processed_data[activity]["y"]):
                    peak_force = np.max(y_data)
                    participant_id = self.processed_data[activity]["participant_ids"][i]
                    
                    # Get weight from metadata if available
                    weight = 70.0  # Default
                    if i < len(self.processed_data[activity]["metadata"]):
                        weight = self.processed_data[activity]["metadata"][i].get('weight_kg', 70.0)
                    
                    # Store data
                    self.force_analysis_data['all_peak_forces'].append(peak_force)
                    self.force_analysis_data['all_activities'].append(activity)
                    self.force_analysis_data['all_participants'].append(participant_id)
                    self.force_analysis_data['all_body_weights'].append(weight)
                    self.force_analysis_data['all_processing_methods'].append('enhanced_biomech')
                    self.force_analysis_data['activity_data'][activity].append(peak_force)
                    
                    # Store participant data
                    if participant_id not in self.force_analysis_data['participant_data']:
                        self.force_analysis_data['participant_data'][participant_id] = {
                            'peak_forces': [], 'activities': [], 'body_weight': weight
                        }
                    self.force_analysis_data['participant_data'][participant_id]['peak_forces'].append(peak_force)
                    self.force_analysis_data['participant_data'][participant_id]['activities'].append(activity)
    
    def _create_overview_analysis_figure(self, output_path: Path):
        """Create overview analysis figure (4-panel)."""
        fig1 = plt.figure(figsize=(16, 12))
        
        # 1. Peak Force by Activity (Box Plot) - Top Left
        ax1 = plt.subplot(2, 2, 1)
        
        # Prepare data for box plot
        activity_forces = []
        activity_labels = []
        
        for activity in ACTIVITIES:
            if activity in self.force_analysis_data['activity_data'] and self.force_analysis_data['activity_data'][activity]:
                activity_forces.append(self.force_analysis_data['activity_data'][activity])
                activity_labels.append(activity)
        
        if activity_forces:
            bp = ax1.boxplot(activity_forces, labels=activity_labels, patch_artist=True)
            
            # Color the boxes
            colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow', 'lightpink']
            for patch, color in zip(bp['boxes'], colors[:len(bp['boxes'])]):
                patch.set_facecolor(color)
        
        ax1.set_ylabel('Peak Force (BW)')
        ax1.set_title('Peak Force by Activity', fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        plt.setp(ax1.get_xticklabels(), rotation=45)
        
        # 2. Force vs Body Weight Correlation - Top Right
        ax2 = plt.subplot(2, 2, 2)
        
        # Create scatter plot with different colors for activities
        activity_colors = {'walking': 'blue', 'running': 'red', 'jogging': 'green', 
                          'heel': 'orange', 'drop': 'purple'}
        
        for activity in ACTIVITIES:
            activity_mask = [act == activity for act in self.force_analysis_data['all_activities']]
            if any(activity_mask):
                activity_forces = [f for f, mask in zip(self.force_analysis_data['all_peak_forces'], activity_mask) if mask]
                activity_weights = [w for w, mask in zip(self.force_analysis_data['all_body_weights'], activity_mask) if mask]
                
                ax2.scatter(activity_weights, activity_forces, 
                           c=activity_colors.get(activity, 'gray'), 
                           label=activity.capitalize(), alpha=0.6, s=50)
        
        # Add correlation line
        if len(self.force_analysis_data['all_peak_forces']) > 1:
            corr_coef = np.corrcoef(self.force_analysis_data['all_body_weights'], 
                                   self.force_analysis_data['all_peak_forces'])[0, 1]
            
            # Fit line
            z = np.polyfit(self.force_analysis_data['all_body_weights'], 
                          self.force_analysis_data['all_peak_forces'], 1)
            p = np.poly1d(z)
            ax2.plot(sorted(self.force_analysis_data['all_body_weights']), 
                    p(sorted(self.force_analysis_data['all_body_weights'])), 
                    "r--", alpha=0.8, linewidth=2)
            
            ax2.text(0.05, 0.95, f'r = {corr_coef:.3f}', 
                    transform=ax2.transAxes, fontsize=12, 
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        ax2.set_xlabel('Body Weight (kg)')
        ax2.set_ylabel('Peak Force (BW)')
        ax2.set_title('Peak Force vs Body Weight', fontsize=14, fontweight='bold')
        ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax2.grid(True, alpha=0.3)
        
        # 3. Trial Count by Participant - Bottom Left
        ax3 = plt.subplot(2, 2, 3)
        
        participant_counts = []
        participant_ids = []
        
        for pid, data in self.force_analysis_data['participant_data'].items():
            participant_counts.append(len(data['peak_forces']))
            participant_ids.append(pid)
        
        # Sort by participant ID
        sorted_data = sorted(zip(participant_ids, participant_counts))
        sorted_ids, sorted_counts = zip(*sorted_data) if sorted_data else ([], [])
        
        bars = ax3.bar(range(len(sorted_counts)), sorted_counts, color='lightsteelblue')
        ax3.set_xlabel('Participant ID')
        ax3.set_ylabel('Number of Trials')
        ax3.set_title('Trial Count by Participant', fontsize=14, fontweight='bold')
        ax3.set_xticks(range(len(sorted_ids)))
        ax3.set_xticklabels([f'P{pid}' for pid in sorted_ids])
        ax3.grid(True, alpha=0.3, axis='y')
        
        # Add count labels on bars
        for bar, count in zip(bars, sorted_counts):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{count}', ha='center', va='bottom', fontsize=10)
        
        # 4. Activity Distribution (Pie Chart) - Bottom Right
        ax4 = plt.subplot(2, 2, 4)
        
        activity_counts = {}
        for activity in self.force_analysis_data['all_activities']:
            activity_counts[activity] = activity_counts.get(activity, 0) + 1
        
        if activity_counts:
            labels = [act.capitalize() for act in activity_counts.keys()]
            sizes = list(activity_counts.values())
            colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow', 'lightpink']
            
            wedges, texts, autotexts = ax4.pie(sizes, labels=labels, colors=colors[:len(sizes)], 
                                              autopct='%1.1f%%', startangle=90)
            
            # Enhance text
            for autotext in autotexts:
                autotext.set_color('black')
                autotext.set_fontweight('bold')
        
        ax4.set_title('Activity Distribution', fontsize=14, fontweight='bold')
        
        plt.suptitle('Comprehensive Ground Reaction Force Analysis Overview', 
                     fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        
        # Save the overview
        overview_path = output_path / "force_overview_analysis.png"
        plt.savefig(overview_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        logger.info(f"✅ Overview analysis saved: {overview_path}")
    
    def _create_distribution_analysis_figure(self, output_path: Path):
        """Create distribution analysis figure (4-panel)."""
        fig2 = plt.figure(figsize=(16, 12))
        
        all_forces = self.force_analysis_data['all_peak_forces']
        
        # 1. Overall Peak Force Distribution (Top Left)
        ax1 = plt.subplot(2, 2, 1)
        n, bins, patches = ax1.hist(all_forces, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        
        mean_force = np.mean(all_forces)
        median_force = np.median(all_forces)
        
        ax1.axvline(mean_force, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_force:.2f} BW')
        ax1.axvline(median_force, color='green', linestyle='--', linewidth=2, label=f'Median: {median_force:.2f} BW')
        
        ax1.set_xlabel('Peak Force (BW)')
        ax1.set_ylabel('Frequency')
        ax1.set_title('Overall Peak Force Distribution', fontsize=14, fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. Mean Peak Force by Participant (Top Right)
        ax2 = plt.subplot(2, 2, 2)
        participant_means = []
        participant_ids = []
        
        for pid, data in self.force_analysis_data['participant_data'].items():
            if data['peak_forces']:
                participant_means.append(np.mean(data['peak_forces']))
                participant_ids.append(pid)
        
        # Sort by mean force
        sorted_data = sorted(zip(participant_ids, participant_means), key=lambda x: x[1])
        sorted_ids, sorted_means = zip(*sorted_data) if sorted_data else ([], [])
        
        bars = ax2.bar(range(len(sorted_means)), sorted_means, color='lightcoral')
        ax2.set_xlabel('Participant (sorted by mean force)')
        ax2.set_ylabel('Mean Peak Force (BW)')
        ax2.set_title('Mean Peak Force by Participant', fontsize=14, fontweight='bold')
        ax2.set_xticks(range(len(sorted_ids)))
        ax2.set_xticklabels([f'P{pid}' for pid in sorted_ids])
        ax2.grid(True, alpha=0.3, axis='y')
        
        # 3. Peak Force Distribution by Activity (Violin Plot) (Bottom Left)
        ax3 = plt.subplot(2, 2, 3)
        
        # Prepare data for violin plot
        violin_data = []
        violin_labels = []
        
        for activity in ACTIVITIES:
            if activity in self.force_analysis_data['activity_data'] and self.force_analysis_data['activity_data'][activity]:
                violin_data.append(self.force_analysis_data['activity_data'][activity])
                violin_labels.append(activity)
        
        if violin_data:
            parts = ax3.violinplot(violin_data, positions=range(len(violin_labels)), showmeans=True, showmedians=True)
            
            # Color the violin plots
            colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
            for i, pc in enumerate(parts['bodies']):
                pc.set_facecolor(colors[i % len(colors)])
                pc.set_alpha(0.7)
            
            ax3.set_xticks(range(len(violin_labels)))
            ax3.set_xticklabels(violin_labels, rotation=45)
            ax3.set_ylabel('Peak Force (BW)')
            ax3.set_title('Peak Force Distribution by Activity (Violin Plot)', fontsize=14, fontweight='bold')
            ax3.grid(True, alpha=0.3)
        
        # 4. Peak Force Range by Activity (Bottom Right)
        ax4 = plt.subplot(2, 2, 4)
        
        activity_stats = {}
        for activity in ACTIVITIES:
            if activity in self.force_analysis_data['activity_data'] and self.force_analysis_data['activity_data'][activity]:
                forces = self.force_analysis_data['activity_data'][activity]
                activity_stats[activity] = {
                    'min': np.min(forces),
                    'max': np.max(forces),
                    'mean': np.mean(forces),
                    'std': np.std(forces)
                }
        
        if activity_stats:
            activities = list(activity_stats.keys())
            mins = [activity_stats[act]['min'] for act in activities]
            maxs = [activity_stats[act]['max'] for act in activities]
            means = [activity_stats[act]['mean'] for act in activities]
            
            x_pos = range(len(activities))
            
            # Plot error bars showing range
            ax4.errorbar(x_pos, means, 
                        yerr=[np.array(means) - np.array(mins), np.array(maxs) - np.array(means)],
                        fmt='o', capsize=5, capthick=2, markersize=8)
            
            ax4.set_xticks(x_pos)
            ax4.set_xticklabels([act.capitalize() for act in activities], rotation=45)
            ax4.set_ylabel('Peak Force (BW)')
            ax4.set_title('Peak Force Range by Activity', fontsize=14, fontweight='bold')
            ax4.grid(True, alpha=0.3)
        
        plt.suptitle('Ground Reaction Force Distribution Analysis', 
                     fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        
        # Save the distribution analysis
        distribution_path = output_path / "force_distribution_analysis.png"
        plt.savefig(distribution_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        logger.info(f"✅ Distribution analysis saved: {distribution_path}")
    
    def _print_force_analysis_summary(self):
        """Print comprehensive force analysis summary."""
        all_forces = self.force_analysis_data['all_peak_forces']
        
        if not all_forces:
            logger.warning("No force data available for summary")
            return
        
        mean_force = np.mean(all_forces)
        median_force = np.median(all_forces)
        
        logger.info("\n" + "="*60)
        logger.info("📊 COMPREHENSIVE FORCE ANALYSIS SUMMARY")
        logger.info("="*60)
        logger.info(f"Total trials analyzed: {len(all_forces)}")
        logger.info(f"Mean peak force: {mean_force:.3f} ± {np.std(all_forces):.3f} BW")
        logger.info(f"Median peak force: {median_force:.3f} BW")
        logger.info(f"Range: {min(all_forces):.3f} - {max(all_forces):.3f} BW")
        
        logger.info(f"\n📊 Activity breakdown:")
        activity_counts = pd.Series(self.force_analysis_data['all_activities']).value_counts()
        for activity, count in activity_counts.items():
            activity_forces = self.force_analysis_data['activity_data'][activity]
            logger.info(f"  {activity.capitalize()}: {count} trials, mean = {np.mean(activity_forces):.3f} BW")
        
        logger.info(f"\n📊 Participant breakdown:")
        for pid, data in sorted(self.force_analysis_data['participant_data'].items()):
            if data['peak_forces']:
                mean_participant_force = np.mean(data['peak_forces'])
                logger.info(f"  P{pid:02d}: {len(data['peak_forces'])} trials, mean = {mean_participant_force:.3f} BW")
    
    def export_research_data(self, export_path: Path = None):
        """Export processed data and analytics for external analysis."""
        export_path = export_path or Path("research_export")
        export_path.mkdir(exist_ok=True)
        
        logger.info(f"📤 Exporting research data to {export_path}")
        
        # 1. Export processed features and targets
        for activity in ACTIVITIES:
            if self.processed_data[activity]["X"]:
                activity_data = {
                    'X': np.array(self.processed_data[activity]["X"]),
                    'y': np.array(self.processed_data[activity]["y"]),
                    'trial_ids': self.processed_data[activity]["trial_ids"],
                    'participant_ids': self.processed_data[activity]["participant_ids"]
                }
                np.savez_compressed(export_path / f'{activity}_processed_data.npz', **activity_data)
        
        # 2. Export analytics summaries
        if self.research_mode:
            analytics = {
                'data_quality_metrics': self.data_quality_metrics,
                'correlation_analysis': self.correlation_analysis,
                'arm_swing_analysis': self.arm_swing_analysis
            }
            
            with open(export_path / 'research_analytics.json', 'w') as f:
                import json
                json.dump(analytics, f, indent=2, default=str)  # default=str for numpy types
        
        # 3. Export configuration
        config_data = {
            'base_path': str(self.base_path),
            'use_waist': self.use_waist,
            'activity_params': ACTIVITY_PARAMS,
            'activities': ACTIVITIES,
            'total_trials': sum(len(self.processed_data[activity]["X"]) for activity in ACTIVITIES)
        }
        
        with open(export_path / 'dataset_config.json', 'w') as f:
            import json
            json.dump(config_data, f, indent=2)
        
        logger.info("✅ Research data export complete")
    
    def generate_publication_summary(self) -> Dict:
        """Generate comprehensive summary for publication."""
        summary = {
            'dataset_overview': {},
            'data_quality': {},
            'technical_specifications': {},
            'research_insights': {}
        }
        
        # Dataset overview
        total_trials = sum(len(self.processed_data[activity]["X"]) for activity in ACTIVITIES)
        total_participants = len(set(
            pid for activity in ACTIVITIES 
            for pid in self.processed_data[activity]["participant_ids"]
        ))
        
        summary['dataset_overview'] = {
            'total_trials': total_trials,
            'total_participants': total_participants,
            'activities': ACTIVITIES,
            'sensor_location': 'waist' if self.use_waist else 'wrist',
            'trials_per_activity': {
                activity: len(self.processed_data[activity]["X"]) 
                for activity in ACTIVITIES
            }
        }
        
        # Data quality summary
        if self.research_mode and self.data_quality_metrics:
            quality_summary = {}
            for activity in ACTIVITIES:
                if activity in self.data_quality_metrics and self.data_quality_metrics[activity]:
                    quality_data = self.data_quality_metrics[activity]
                    quality_summary[activity] = {
                        'avg_snr': np.mean([q['imu_snr'] for q in quality_data]),
                        'avg_force_peak': np.mean([q['force_peak'] for q in quality_data]),
                        'data_completeness': 100 - np.mean([q['imu_missing_pct'] for q in quality_data])
                    }
            summary['data_quality'] = quality_summary
        
        # Technical specifications
        summary['technical_specifications'] = {
            'target_lengths': {activity: ACTIVITY_PARAMS[activity]['TARGET_LEN'] for activity in ACTIVITIES},
            'sampling_rates': {
                'force_plate': '1000 Hz',
                'imu': '~100 Hz'
            },
            'preprocessing': {
                'normalization': 'StandardScaler for locomotion, RobustScaler for impacts',
                'filtering': 'Activity-specific thresholds',
                'resampling': 'Linear interpolation to target length'
            }
        }
        
        return summary

# ============================================================================
# USAGE EXAMPLE AND TESTING
# ============================================================================

def run_enhanced_analysis():
    """Run the enhanced data analysis pipeline with comprehensive visualizations."""
    logger.info("🚀 Starting Enhanced GRF Dataset Analysis")
    
    # Initialize processor
    processor = EnhancedDataProcessor(
        base_path=BASE_PATH,
        use_waist=USE_WAIST,
        research_mode=True
    )
    
    # Load and process all data
    success = processor.load_all_data()
    
    if not success:
        logger.error("❌ Data loading failed")
        return None
    
    # Generate research visualizations
    processor.generate_research_visualizations()
    
    # NEW: Generate comprehensive force plate visualizations
    logger.info("🎨 Creating comprehensive force plate analysis...")
    processor.create_comprehensive_force_visualizations()
    
    # Export research data
    processor.export_research_data()
    
    # Generate publication summary
    pub_summary = processor.generate_publication_summary()
    
    logger.info("\n" + "="*80)
    logger.info("📊 PUBLICATION SUMMARY")
    logger.info("="*80)
    logger.info(f"Total Trials: {pub_summary['dataset_overview']['total_trials']}")
    logger.info(f"Total Participants: {pub_summary['dataset_overview']['total_participants']}")
    logger.info(f"Sensor Location: {pub_summary['dataset_overview']['sensor_location']}")
    
    logger.info("\n✅ COMPLETE ANALYSIS FINISHED!")
    logger.info("📊 Generated visualizations:")
    logger.info("  1. Research visualizations (data quality, correlations, etc.)")
    logger.info("  2. Comprehensive force analysis (overview + distribution)")
    logger.info("  3. Sample signals and activity distributions")
    logger.info("📄 All files saved in respective output directories")
    
    return processor

# Run analysis if executed directly
if __name__ == "__main__":
    processor = run_enhanced_analysis()

2025-10-28 12:01:48,193 - INFO - 🚀 Starting Enhanced GRF Dataset Analysis
2025-10-28 12:01:48,196 - INFO - 🔄 Loading data from /kaggle/input/dataset10/Dataset
2025-10-28 12:01:48,196 - INFO - 📊 Sensor location: Wrist
2025-10-28 12:01:48,197 - INFO - 🔬 Research mode: Enabled
2025-10-28 12:01:48,214 - INFO - Parsed info - P09, Year/Trial: 1992, Height: 167.0cm, Weight: 60.2kg
2025-10-28 12:01:48,215 - INFO - 
👤 Processing Participant 9 (Weight: 60.2kg, Height: 167.0cm)
2025-10-28 12:01:48,262 - INFO - 🔍 Processing running trial run0002_f_7.csv (P09)
2025-10-28 12:01:48,328 - INFO -   ✅ Final running features: (50, 6)
2025-10-28 12:01:48,329 - INFO -     📊 X range: [-2.189, 2.339]
2025-10-28 12:01:48,329 - INFO -     🎯 y range: [0.178, 2.049] BW (peak: 2.049)
2025-10-28 12:01:48,330 - INFO - 🔍 Processing walking trial walk0005_f_6.csv (P09)
2025-10-28 12:01:48,386 - INFO -   ✅ Final walking features: (100, 6)
2025-10-28 12:01:48,386 - INFO -     📊 X range: [-2.215, 2.393]
2025-10-28 12:01

In [ ]:
/kaggle/input/dataset10/Dataset

In [2]:
#!/usr/bin/env python3
"""
Scientific 3-Phase Synchronization & Analysis
Phase 1: Waist → Force Plate (Reference Mapping)
Phase 2: Wrist → Waist (Transfer Learning)
Phase 3: Wrist → Force Plate (End-to-End Deployment)

Author: Scientific Biomechanics Pipeline
"""

import os
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import correlate, find_peaks
from scipy.stats import pearsonr

# ==================== CONFIGURATION ====================
BASE_PATH = Path("/kaggle/input/dataset10/Dataset")
OUT_DIR = Path("scientific_sync_analysis")
OUT_DIR.mkdir(exist_ok=True)

# Create subdirectories for each phase
PHASE_DIRS = {
    'phase1': OUT_DIR / "phase1_waist_to_fp",
    'phase2': OUT_DIR / "phase2_wrist_to_waist", 
    'phase3': OUT_DIR / "phase3_wrist_to_fp"
}

for phase_dir in PHASE_DIRS.values():
    phase_dir.mkdir(exist_ok=True)

# ==================== FILE DETECTION ====================
def is_force_plate_csv(path: Path):
    """Detect force plate by Force_Z column."""
    try:
        for skip in [26, 27, 28, 0]:
            try:
                df = pd.read_csv(path, skiprows=skip, nrows=1)
                cols = [c.lower() for c in df.columns]
                if any(("force" in c and "z" in c) for c in cols):
                    return True
            except:
                continue
        return False
    except:
        return False

def is_watch_csv(path: Path):
    """Detect watch CSV by IMU columns."""
    try:
        df = pd.read_csv(path, nrows=1)
        needed = {'userAccelerationX', 'userAccelerationY', 'userAccelerationZ'}
        return needed.issubset(set(df.columns))
    except:
        return False

def infer_activity(name: str) -> str:
    """Infer activity from filename."""
    name_lower = name.lower()
    patterns = {
        'heel': ['heel'],
        'drop': ['drop'],
        'running': ['run'],
        'jogging': ['jog'],
        'walking': ['walk']
    }
    
    for activity, keywords in patterns.items():
        if any(kw in name_lower for kw in keywords):
            return activity
    return "unknown"

# ==================== SIGNAL PROCESSING ====================
def zscore_safe(x):
    """Z-score normalization with safety."""
    x = np.asarray(x, dtype=float)
    mu = np.nanmean(x)
    sd = np.nanstd(x)
    return (x - mu) / (sd + 1e-10)

def read_force_plate_full(path: Path):
    """Read complete force plate data."""
    df = None
    for skip in [26, 27, 28, 25, 0]:
        try:
            df = pd.read_csv(path, skiprows=skip)
            df.columns = [c.strip() for c in df.columns]
            if any("force" in c.lower() and "z" in c.lower() for c in df.columns):
                break
        except:
            continue
    
    if df is None:
        raise ValueError(f"Cannot read {path.name}")
    
    fz_col = next((c for c in df.columns if "force" in c.lower() and "z" in c.lower()), None)
    if fz_col is None:
        raise ValueError(f"No Force_Z in {path.name}")
    
    force_z = pd.to_numeric(df[fz_col], errors="coerce").fillna(0.0).to_numpy()
    
    # Get time info
    time_col = next((c for c in df.columns if c.lower().strip() in ("time", "time (s)", "time(s)")), None)
    
    if time_col:
        time = pd.to_numeric(df[time_col], errors="coerce").to_numpy()
        time = time[time > 0]
        if len(time) > 1:
            dt = np.median(np.diff(time))
            fs = 1.0 / dt if dt > 0 else 1000.0
        else:
            fs = 1000.0
    else:
        fs = 1000.0
    
    return force_z, fs

def read_watch_full(path: Path):
    """Read complete IMU data with all channels."""
    df = pd.read_csv(path)
    
    # Check for required columns
    acc_cols = ['userAccelerationX', 'userAccelerationY', 'userAccelerationZ']
    gyro_cols = ['rotationRateX', 'rotationRateY', 'rotationRateZ']
    
    for c in acc_cols:
        if c not in df.columns:
            raise ValueError(f"Missing {c}")
    
    # Get acceleration
    acc = df[acc_cols].to_numpy()
    acc_mag = np.sqrt((acc ** 2).sum(axis=1))
    
    # Get gyroscope if available
    has_gyro = all(c in df.columns for c in gyro_cols)
    if has_gyro:
        gyro = df[gyro_cols].to_numpy()
        gyro_mag = np.sqrt((gyro ** 2).sum(axis=1))
    else:
        gyro_mag = None
    
    # Get vertical acceleration (Z-axis most relevant for GRF)
    acc_z = acc[:, 2]
    
    # Estimate sampling rate
    if 'time' in df.columns or 'Time' in df.columns:
        time_col = 'time' if 'time' in df.columns else 'Time'
        time = pd.to_numeric(df[time_col], errors='coerce').dropna().to_numpy()
        if len(time) > 1:
            dt = np.median(np.diff(time))
            fs = 1.0 / dt if dt > 0 else 100.0
        else:
            fs = 100.0
    else:
        fs = 100.0
    
    return {
        'acc_mag': acc_mag,
        'acc_z': acc_z,
        'acc_xyz': acc,
        'gyro_mag': gyro_mag,
        'fs': fs
    }

def align_signals_advanced(force_sig, imu_dict, fs_force):
    """
    Advanced signal alignment with multiple strategies.
    Returns aligned signals and metadata.
    """
    fs_imu = imu_dict['fs']
    
    # Calculate durations
    duration_force = len(force_sig) / fs_force
    duration_imu = len(imu_dict['acc_mag']) / fs_imu
    
    # Use shorter duration
    common_duration = min(duration_force, duration_imu)
    
    # Use higher sampling rate as target
    target_fs = max(fs_force, fs_imu)
    target_samples = int(common_duration * target_fs)
    
    # Resample function
    def resample_signal(sig, target_len):
        if len(sig) == target_len:
            return sig
        old_idx = np.linspace(0, len(sig)-1, len(sig))
        new_idx = np.linspace(0, len(sig)-1, target_len)
        return np.interp(new_idx, old_idx, sig)
    
    # Truncate to common duration
    force_samples = int(common_duration * fs_force)
    imu_samples = int(common_duration * fs_imu)
    
    force_truncated = force_sig[:force_samples]
    
    # Resample force to target
    force_aligned = resample_signal(force_truncated, target_samples)
    
    # Resample all IMU signals
    aligned_imu = {}
    for key in ['acc_mag', 'acc_z']:
        imu_truncated = imu_dict[key][:imu_samples]
        aligned_imu[key] = resample_signal(imu_truncated, target_samples)
    
    if imu_dict['gyro_mag'] is not None:
        gyro_truncated = imu_dict['gyro_mag'][:imu_samples]
        aligned_imu['gyro_mag'] = resample_signal(gyro_truncated, target_samples)
    
    return force_aligned, aligned_imu, target_fs

def find_optimal_lag_dtw(ref, sig, max_lag=200):
    """
    Find optimal lag using Dynamic Time Warping approach.
    More robust than simple cross-correlation for biomechanical signals.
    """
    n = min(len(ref), len(sig))
    if n < 50:
        return 0, 0.0, 0.0
    
    # Normalize signals
    ref_z = zscore_safe(ref[:n])
    sig_z = zscore_safe(sig[:n])
    
    # Cross-correlation
    corr = correlate(sig_z, ref_z, mode='full')
    lags = np.arange(-n+1, n)
    
    # Limit search window
    mask = (lags >= -max_lag) & (lags <= max_lag)
    corr_valid = corr[mask]
    lags_valid = lags[mask]
    
    # Find best lag
    best_idx = np.argmax(corr_valid)
    best_lag = int(lags_valid[best_idx])
    
    # Normalize correlation coefficient
    max_corr = float(corr_valid[best_idx] / (np.linalg.norm(ref_z) * np.linalg.norm(sig_z) + 1e-9))
    
    # Also calculate Pearson correlation at best lag
    if best_lag == 0:
        pearson_r, _ = pearsonr(ref_z, sig_z)
    elif best_lag > 0:
        pearson_r, _ = pearsonr(ref_z[:-best_lag], sig_z[best_lag:])
    else:
        pearson_r, _ = pearsonr(ref_z[-best_lag:], sig_z[:best_lag])
    
    return best_lag, max_corr, pearson_r

# ==================== DISCOVERY ====================
def discover_triads(base: Path):
    """Find complete triads (FP + Wrist + Waist)."""
    triads = []
    
    print("🔍 Scanning directories...")
    
    for participant_dir in sorted(d for d in base.iterdir() if d.is_dir()):
        print(f"  📁 {participant_dir.name}")
        csv_files = list(participant_dir.glob("*.csv"))
        
        fp_files = []
        wrist_files = []
        waist_files = []
        
        for filepath in csv_files:
            fname_lower = filepath.name.lower()
            
            if is_force_plate_csv(filepath):
                fp_files.append(filepath)
            elif is_watch_csv(filepath):
                if 'wrist' in fname_lower:
                    wrist_files.append(filepath)
                elif 'waist' in fname_lower:
                    waist_files.append(filepath)
        
        print(f"     FP: {len(fp_files)}, Wrist: {len(wrist_files)}, Waist: {len(waist_files)}")
        
        # Match by activity
        for fp_path in fp_files:
            activity = infer_activity(fp_path.name)
            if activity == "unknown":
                continue
            
            wrist_match = next((w for w in wrist_files if infer_activity(w.name) == activity), None)
            waist_match = next((w for w in waist_files if infer_activity(w.name) == activity), None)
            
            if wrist_match and waist_match:
                triads.append({
                    "participant": participant_dir.name,
                    "activity": activity,
                    "fp": fp_path,
                    "wrist": wrist_match,
                    "waist": waist_match
                })
                print(f"     ✓ {activity}: {fp_path.name[:40]}...")
    
    return triads

# ==================== PHASE ANALYSIS ====================
def analyze_phase1_waist_to_fp(force_sig, waist_dict, fs, participant, activity):
    """Phase 1: Establish ground truth mapping from Waist to Force Plate."""
    results = {}
    
    # 1. Acceleration magnitude
    lag_mag, corr_mag, pearson_mag = find_optimal_lag_dtw(force_sig, waist_dict['acc_mag'])
    results['acc_mag'] = {'lag': lag_mag, 'corr_xcorr': corr_mag, 'corr_pearson': pearson_mag}
    
    # 2. Vertical acceleration (Z-axis)
    lag_z, corr_z, pearson_z = find_optimal_lag_dtw(force_sig, waist_dict['acc_z'])
    results['acc_z'] = {'lag': lag_z, 'corr_xcorr': corr_z, 'corr_pearson': pearson_z}
    
    # 3. Gyroscope if available
    if waist_dict.get('gyro_mag') is not None:
        lag_gyro, corr_gyro, pearson_gyro = find_optimal_lag_dtw(force_sig, waist_dict['gyro_mag'])
        results['gyro_mag'] = {'lag': lag_gyro, 'corr_xcorr': corr_gyro, 'corr_pearson': pearson_gyro}
    
    # Select best feature
    best_feature = max(results.items(), key=lambda x: abs(x[1]['corr_pearson']))
    
    return results, best_feature

def analyze_phase2_wrist_to_waist(waist_dict, wrist_dict, fs, participant, activity):
    """Phase 2: Learn transformation from Wrist to Waist."""
    results = {}
    
    for feature in ['acc_mag', 'acc_z']:
        lag, corr_xcorr, corr_pearson = find_optimal_lag_dtw(waist_dict[feature], wrist_dict[feature])
        results[feature] = {
            'lag': lag,
            'corr_xcorr': corr_xcorr,
            'corr_pearson': corr_pearson
        }
    
    if waist_dict.get('gyro_mag') is not None and wrist_dict.get('gyro_mag') is not None:
        lag, corr_xcorr, corr_pearson = find_optimal_lag_dtw(waist_dict['gyro_mag'], wrist_dict['gyro_mag'])
        results['gyro_mag'] = {
            'lag': lag,
            'corr_xcorr': corr_xcorr,
            'corr_pearson': corr_pearson
        }
    
    best_feature = max(results.items(), key=lambda x: abs(x[1]['corr_pearson']))
    
    return results, best_feature

def analyze_phase3_wrist_to_fp(force_sig, wrist_dict, fs, participant, activity):
    """Phase 3: Direct end-to-end mapping from Wrist to Force Plate."""
    results = {}
    
    for feature in ['acc_mag', 'acc_z']:
        lag, corr_xcorr, corr_pearson = find_optimal_lag_dtw(force_sig, wrist_dict[feature])
        results[feature] = {
            'lag': lag,
            'corr_xcorr': corr_xcorr,
            'corr_pearson': corr_pearson
        }
    
    if wrist_dict.get('gyro_mag') is not None:
        lag, corr_xcorr, corr_pearson = find_optimal_lag_dtw(force_sig, wrist_dict['gyro_mag'])
        results['gyro_mag'] = {
            'lag': lag,
            'corr_xcorr': corr_xcorr,
            'corr_pearson': corr_pearson
        }
    
    best_feature = max(results.items(), key=lambda x: abs(x[1]['corr_pearson']))
    
    return results, best_feature

# ==================== VISUALIZATION ====================
def create_three_phase_visualization(force_sig, waist_dict, wrist_dict,
                                    phase1_results, phase2_results, phase3_results,
                                    fs, participant, activity, out_dir):
    """Create comprehensive 3-phase visualization."""
    fig = plt.figure(figsize=(18, 12))
    
    t = np.arange(len(force_sig)) / fs
    
    # Normalize signals
    force_z = zscore_safe(force_sig)
    waist_mag_z = zscore_safe(waist_dict['acc_mag'])
    wrist_mag_z = zscore_safe(wrist_dict['acc_mag'])
    
    # Phase 1: WAIST → FORCE PLATE
    ax1 = plt.subplot(3, 2, 1)
    ax1.plot(t, force_z, linewidth=2.5, label="Force Plate", color="#FF6B35", alpha=0.9)
    ax1.plot(t, waist_mag_z, linewidth=1.8, label="Waist IMU", color="#00A878", alpha=0.8)
    
    best_waist = phase1_results[1]
    r_waist = best_waist[1]['corr_pearson']
    lag_waist = best_waist[1]['lag']
    quality_waist = "🟢" if abs(r_waist) > 0.7 else "🟡" if abs(r_waist) > 0.5 else "🔴"
    
    ax1.set_title(f"{quality_waist} Phase 1: Waist → FP | r={r_waist:.3f}, lag={lag_waist/fs*1000:.1f}ms",
                 fontsize=12, fontweight='bold')
    ax1.set_ylabel("Z-scored amplitude", fontsize=10)
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # Zoomed view
    ax2 = plt.subplot(3, 2, 2)
    zoom_samples = min(int(3 * fs), len(force_sig))
    t_zoom = t[:zoom_samples]
    ax2.plot(t_zoom, force_z[:zoom_samples], linewidth=2.5, color="#FF6B35", alpha=0.9)
    ax2.plot(t_zoom, waist_mag_z[:zoom_samples], linewidth=1.8, color="#00A878", alpha=0.8)
    ax2.set_title("Phase 1: Zoomed (3s)", fontsize=11, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Phase 2: WRIST → WAIST
    ax3 = plt.subplot(3, 2, 3)
    ax3.plot(t, waist_mag_z, linewidth=2.5, label="Waist", color="#00A878", alpha=0.9)
    ax3.plot(t, wrist_mag_z, linewidth=1.8, label="Wrist", color="#004E89", alpha=0.8)
    
    best_wrist_waist = phase2_results[1]
    r_wrist_waist = best_wrist_waist[1]['corr_pearson']
    lag_wrist_waist = best_wrist_waist[1]['lag']
    quality_transfer = "🟢" if abs(r_wrist_waist) > 0.7 else "🟡" if abs(r_wrist_waist) > 0.5 else "🔴"
    
    ax3.set_title(f"{quality_transfer} Phase 2: Wrist → Waist | r={r_wrist_waist:.3f}, lag={lag_wrist_waist/fs*1000:.1f}ms",
                 fontsize=12, fontweight='bold')
    ax3.set_ylabel("Z-scored amplitude", fontsize=10)
    ax3.legend(fontsize=9)
    ax3.grid(True, alpha=0.3)
    
    # Zoomed view
    ax4 = plt.subplot(3, 2, 4)
    ax4.plot(t_zoom, waist_mag_z[:zoom_samples], linewidth=2.5, color="#00A878", alpha=0.9)
    ax4.plot(t_zoom, wrist_mag_z[:zoom_samples], linewidth=1.8, color="#004E89", alpha=0.8)
    ax4.set_title("Phase 2: Zoomed (3s)", fontsize=11, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    # Phase 3: WRIST → FORCE PLATE
    ax5 = plt.subplot(3, 2, 5)
    ax5.plot(t, force_z, linewidth=2.5, label="Force Plate", color="#FF6B35", alpha=0.9)
    ax5.plot(t, wrist_mag_z, linewidth=1.8, label="Wrist", color="#004E89", alpha=0.8)
    
    best_wrist_fp = phase3_results[1]
    r_wrist_fp = best_wrist_fp[1]['corr_pearson']
    lag_wrist_fp = best_wrist_fp[1]['lag']
    quality_deploy = "🟢" if abs(r_wrist_fp) > 0.7 else "🟡" if abs(r_wrist_fp) > 0.5 else "🔴"
    
    ax5.set_title(f"{quality_deploy} Phase 3: Wrist → FP (Deployment) | r={r_wrist_fp:.3f}, lag={lag_wrist_fp/fs*1000:.1f}ms",
                 fontsize=12, fontweight='bold')
    ax5.set_xlabel("Time (s)", fontsize=10)
    ax5.set_ylabel("Z-scored amplitude", fontsize=10)
    ax5.legend(fontsize=9)
    ax5.grid(True, alpha=0.3)
    
    # Zoomed view
    ax6 = plt.subplot(3, 2, 6)
    ax6.plot(t_zoom, force_z[:zoom_samples], linewidth=2.5, color="#FF6B35", alpha=0.9)
    ax6.plot(t_zoom, wrist_mag_z[:zoom_samples], linewidth=1.8, color="#004E89", alpha=0.8)
    ax6.set_title("Phase 3: Zoomed (3s)", fontsize=11, fontweight='bold')
    ax6.set_xlabel("Time (s)", fontsize=10)
    ax6.grid(True, alpha=0.3)
    
    plt.suptitle(f"3-Phase Scientific Analysis: {activity.capitalize()} | {participant}\n"
                f"Duration: {t[-1]:.1f}s | Samples: {len(force_sig)} | Rate: {fs:.0f}Hz",
                fontsize=14, fontweight='bold', y=0.98)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    out_name = f"three_phase_{participant}_{activity}.png"
    out_path = out_dir / out_name
    fig.savefig(out_path, dpi=200, facecolor="white", bbox_inches="tight")
    plt.close(fig)
    
    return out_path

# ==================== PROCESSING ====================
def process_triad_three_phases(triad):
    """Process single triad through all 3 phases."""
    try:
        # Read all data
        force_sig, fs_force = read_force_plate_full(triad["fp"])
        waist_data = read_watch_full(triad["waist"])
        wrist_data = read_watch_full(triad["wrist"])
        
        # Align all signals
        force_aligned, waist_aligned, fs_common = align_signals_advanced(force_sig, waist_data, fs_force)
        _, wrist_aligned, _ = align_signals_advanced(force_sig, wrist_data, fs_force)
        
        # Ensure same length
        min_len = min(len(force_aligned), len(waist_aligned['acc_mag']), len(wrist_aligned['acc_mag']))
        if min_len < 100:
            raise ValueError(f"Signals too short: {min_len}")
        
        force_final = force_aligned[:min_len]
        waist_final = {k: v[:min_len] for k, v in waist_aligned.items()}
        wrist_final = {k: v[:min_len] for k, v in wrist_aligned.items()}
        
        # Phase 1: Waist → FP
        phase1_results, phase1_best = analyze_phase1_waist_to_fp(
            force_final, waist_final, fs_common,
            triad['participant'], triad['activity']
        )
        
        # Phase 2: Wrist → Waist
        phase2_results, phase2_best = analyze_phase2_wrist_to_waist(
            waist_final, wrist_final, fs_common,
            triad['participant'], triad['activity']
        )
        
        # Phase 3: Wrist → FP
        phase3_results, phase3_best = analyze_phase3_wrist_to_fp(
            force_final, wrist_final, fs_common,
            triad['participant'], triad['activity']
        )
        
        # Create visualization
        plot_path = create_three_phase_visualization(
            force_final, waist_final, wrist_final,
            (phase1_results, phase1_best),
            (phase2_results, phase2_best),
            (phase3_results, phase3_best),
            fs_common, triad['participant'], triad['activity'],
            OUT_DIR
        )
        
        return {
            "participant": triad["participant"],
            "activity": triad["activity"],
            "fp_file": triad["fp"].name,
            "wrist_file": triad["wrist"].name,
            "waist_file": triad["waist"].name,
            
            # Phase 1
            "phase1_feature": phase1_best[0],
            "phase1_lag_ms": (phase1_best[1]['lag'] / fs_common) * 1000,
            "phase1_corr_xcorr": phase1_best[1]['corr_xcorr'],
            "phase1_corr_pearson": phase1_best[1]['corr_pearson'],
            
            # Phase 2
            "phase2_feature": phase2_best[0],
            "phase2_lag_ms": (phase2_best[1]['lag'] / fs_common) * 1000,
            "phase2_corr_xcorr": phase2_best[1]['corr_xcorr'],
            "phase2_corr_pearson": phase2_best[1]['corr_pearson'],
            
            # Phase 3
            "phase3_feature": phase3_best[0],
            "phase3_lag_ms": (phase3_best[1]['lag'] / fs_common) * 1000,
            "phase3_corr_xcorr": phase3_best[1]['corr_xcorr'],
            "phase3_corr_pearson": phase3_best[1]['corr_pearson'],
            
            "duration_s": len(force_final) / fs_common,
            "samples": len(force_final),
            "sampling_rate": fs_common,
            "plot_path": str(plot_path),
            "status": "success"
        }
        
    except Exception as e:
        return {
            "participant": triad["participant"],
            "activity": triad["activity"],
            "fp_file": triad.get("fp", Path("")).name,
            "wrist_file": triad.get("wrist", Path("")).name,
            "waist_file": triad.get("waist", Path("")).name,
            "status": "failed",
            "error": str(e)
        }

# ==================== BATCH PROCESSING ====================
def run_three_phase_batch(base=BASE_PATH, max_trials=None):
    """Run 3-phase analysis on all triads."""
    print("="*80)
    print("🎯 SCIENTIFIC 3-PHASE SYNCHRONIZATION ANALYSIS")
    print("="*80)
    print("Phase 1: Waist → Force Plate (Reference Ground Truth)")
    print("Phase 2: Wrist → Waist (Transfer Learning)")
    print("Phase 3: Wrist → Force Plate (End-to-End Deployment)")
    print("="*80)
    print(f"Base path: {base}")
    print(f"Output: {OUT_DIR}\n")
    
    triads = discover_triads(base)
    
    if not triads:
        print("\n❌ No triads found!")
        return None
    
    total = len(triads)
    n_process = min(total, max_trials) if max_trials else total
    
    print(f"\n✅ Found {total} complete triads")
    print(f"   Processing {n_process}...\n")
    
    # Activity distribution
    by_activity = {}
    for t in triads:
        by_activity[t['activity']] = by_activity.get(t['activity'], 0) + 1
    
    print("Distribution:")
    for act, count in sorted(by_activity.items()):
        print(f"  {act:10s}: {count:3d}")
    print()
    
    # Process all triads
    results = []
    for i, triad in enumerate(triads[:n_process]):
        print(f"[{i+1:3d}/{n_process}] Processing {triad['participant']} - {triad['activity']}...")
        result = process_triad_three_phases(triad)
        results.append(result)
        
        if result['status'] == 'success':
            p1_q = "✓" if abs(result['phase1_corr_pearson']) > 0.7 else "~"
            p2_q = "✓" if abs(result['phase2_corr_pearson']) > 0.7 else "~"
            p3_q = "✓" if abs(result['phase3_corr_pearson']) > 0.7 else "~"
            
            print(f"  {p1_q} Phase 1 (Waist→FP):   r={result['phase1_corr_pearson']:+.3f}")
            print(f"  {p2_q} Phase 2 (Wrist→Waist): r={result['phase2_corr_pearson']:+.3f}")
            print(f"  {p3_q} Phase 3 (Wrist→FP):   r={result['phase3_corr_pearson']:+.3f}")
        else:
            print(f"  ✗ Failed: {result.get('error', 'Unknown')[:60]}")
        print()
    
    # Save results
    df = pd.DataFrame(results)
    csv_path = OUT_DIR / "three_phase_results.csv"
    df.to_csv(csv_path, index=False)
    
    # Generate statistics
    success = df[df['status'] == 'success']
    
    if len(success) > 0:
        print("\n" + "="*80)
        print("📊 THREE-PHASE ANALYSIS STATISTICS")
        print("="*80)
        print(f"Success rate: {len(success)}/{len(df)} ({100*len(success)/len(df):.1f}%)\n")
        
        # Phase 1 Statistics
        print("🟢 PHASE 1: Waist → Force Plate (Reference Mapping)")
        print(f"   Mean correlation: {success['phase1_corr_pearson'].mean():.3f} ± {success['phase1_corr_pearson'].std():.3f}")
        print(f"   Median lag: {success['phase1_lag_ms'].median():.1f}ms")
        print(f"   Best features: {success['phase1_feature'].value_counts().to_dict()}")
        
        # Phase 2 Statistics
        print("\n🟡 PHASE 2: Wrist → Waist (Transfer Learning)")
        print(f"   Mean correlation: {success['phase2_corr_pearson'].mean():.3f} ± {success['phase2_corr_pearson'].std():.3f}")
        print(f"   Median lag: {success['phase2_lag_ms'].median():.1f}ms")
        print(f"   Best features: {success['phase2_feature'].value_counts().to_dict()}")
        
        # Phase 3 Statistics
        print("\n🔴 PHASE 3: Wrist → Force Plate (Deployment)")
        print(f"   Mean correlation: {success['phase3_corr_pearson'].mean():.3f} ± {success['phase3_corr_pearson'].std():.3f}")
        print(f"   Median lag: {success['phase3_lag_ms'].median():.1f}ms")
        print(f"   Best features: {success['phase3_feature'].value_counts().to_dict()}")
        
        # Comparison analysis
        print("\n📈 PHASE COMPARISON:")
        print(f"   Phase 1 vs Phase 3 correlation gap: {(success['phase1_corr_pearson'].mean() - success['phase3_corr_pearson'].mean()):.3f}")
        print(f"   Phase 2 quality (transfer): {success['phase2_corr_pearson'].mean():.3f}")
        
        # By activity breakdown
        print("\n📊 BY ACTIVITY:")
        for act in sorted(success['activity'].unique()):
            act_data = success[success['activity'] == act]
            print(f"\n  {act.capitalize()}:")
            print(f"    n = {len(act_data)}")
            print(f"    Phase 1: r={act_data['phase1_corr_pearson'].mean():.3f} (Waist→FP)")
            print(f"    Phase 2: r={act_data['phase2_corr_pearson'].mean():.3f} (Wrist→Waist)")
            print(f"    Phase 3: r={act_data['phase3_corr_pearson'].mean():.3f} (Wrist→FP)")
        
        # Quality tiers
        print("\n🎯 DEPLOYMENT READINESS (Phase 3):")
        excellent = success[success['phase3_corr_pearson'].abs() > 0.8]
        good = success[(success['phase3_corr_pearson'].abs() > 0.6) & (success['phase3_corr_pearson'].abs() <= 0.8)]
        moderate = success[(success['phase3_corr_pearson'].abs() > 0.4) & (success['phase3_corr_pearson'].abs() <= 0.6)]
        poor = success[success['phase3_corr_pearson'].abs() <= 0.4]
        
        print(f"   🟢 Excellent (r > 0.8): {len(excellent)} ({100*len(excellent)/len(success):.1f}%)")
        print(f"   🟡 Good (r > 0.6):      {len(good)} ({100*len(good)/len(success):.1f}%)")
        print(f"   🟠 Moderate (r > 0.4):  {len(moderate)} ({100*len(moderate)/len(success):.1f}%)")
        print(f"   🔴 Poor (r ≤ 0.4):      {len(poor)} ({100*len(poor)/len(success):.1f}%)")
        
        # Transfer learning effectiveness
        print("\n🔄 TRANSFER LEARNING EFFECTIVENESS:")
        combined_potential = (success['phase1_corr_pearson'].abs() + success['phase2_corr_pearson'].abs()) / 2
        direct_performance = success['phase3_corr_pearson'].abs()
        transfer_gap = combined_potential - direct_performance
        
        print(f"   Combined potential (P1+P2)/2: {combined_potential.mean():.3f}")
        print(f"   Direct performance (P3):      {direct_performance.mean():.3f}")
        print(f"   Transfer gap:                 {transfer_gap.mean():.3f}")
        
        if transfer_gap.mean() < 0.1:
            print(f"   ✓ Good transfer learning (gap < 0.1)")
        else:
            print(f"   ⚠ Transfer learning needs improvement (gap = {transfer_gap.mean():.3f})")
    
    print(f"\n✅ Results saved: {csv_path}")
    print(f"   Plots saved: {OUT_DIR}/")
    print(f"\n💡 RESEARCH INSIGHTS:")
    print(f"   • Phase 1 shows the theoretical best performance (Waist at CoM)")
    print(f"   • Phase 2 shows how well Wrist can approximate Waist")
    print(f"   • Phase 3 shows real-world deployment feasibility (Wrist-only)")
    print(f"   • Gap between Phase 1 and Phase 3 indicates room for improvement")
    
    return df

# ==================== ADVANCED ANALYSIS ====================
def generate_phase_comparison_plot(df: pd.DataFrame):
    """Generate comparison plot across all three phases."""
    success = df[df['status'] == 'success']
    
    if len(success) == 0:
        print("No successful trials to plot")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Correlation comparison across phases
    ax1 = axes[0, 0]
    phases = ['Phase 1\n(Waist→FP)', 'Phase 2\n(Wrist→Waist)', 'Phase 3\n(Wrist→FP)']
    correlations = [
        success['phase1_corr_pearson'].abs().values,
        success['phase2_corr_pearson'].abs().values,
        success['phase3_corr_pearson'].abs().values
    ]
    
    bp = ax1.boxplot(correlations, labels=phases, patch_artist=True)
    colors = ['#00A878', '#FFB627', '#FF6B35']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax1.set_ylabel('|Pearson Correlation|', fontsize=11)
    ax1.set_title('Correlation Strength Across Phases', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.axhline(0.7, color='green', linestyle='--', alpha=0.5, label='Good threshold')
    ax1.axhline(0.5, color='orange', linestyle='--', alpha=0.5, label='Moderate threshold')
    ax1.legend(fontsize=9)
    
    # 2. Lag comparison
    ax2 = axes[0, 1]
    lags = [
        success['phase1_lag_ms'].abs().values,
        success['phase2_lag_ms'].abs().values,
        success['phase3_lag_ms'].abs().values
    ]
    
    bp2 = ax2.boxplot(lags, labels=phases, patch_artist=True)
    for patch, color in zip(bp2['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax2.set_ylabel('|Lag| (ms)', fontsize=11)
    ax2.set_title('Time Lag Across Phases', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # 3. Phase 1 vs Phase 3 scatter
    ax3 = axes[1, 0]
    activity_colors = {'walking': 'blue', 'running': 'red', 'jogging': 'green', 
                       'heel': 'orange', 'drop': 'purple'}
    
    for activity in success['activity'].unique():
        act_data = success[success['activity'] == activity]
        ax3.scatter(act_data['phase1_corr_pearson'].abs(), 
                   act_data['phase3_corr_pearson'].abs(),
                   c=activity_colors.get(activity, 'gray'),
                   label=activity.capitalize(),
                   alpha=0.6, s=80)
    
    # Add diagonal line (perfect transfer)
    max_val = max(success['phase1_corr_pearson'].abs().max(), 
                  success['phase3_corr_pearson'].abs().max())
    ax3.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Perfect transfer')
    
    ax3.set_xlabel('Phase 1: |r| (Waist→FP)', fontsize=11)
    ax3.set_ylabel('Phase 3: |r| (Wrist→FP)', fontsize=11)
    ax3.set_title('Transfer Learning Effectiveness', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.legend(fontsize=9)
    
    # 4. By activity comparison
    ax4 = axes[1, 1]
    activities = sorted(success['activity'].unique())
    x = np.arange(len(activities))
    width = 0.25
    
    p1_means = [success[success['activity'] == act]['phase1_corr_pearson'].abs().mean() for act in activities]
    p2_means = [success[success['activity'] == act]['phase2_corr_pearson'].abs().mean() for act in activities]
    p3_means = [success[success['activity'] == act]['phase3_corr_pearson'].abs().mean() for act in activities]
    
    ax4.bar(x - width, p1_means, width, label='Phase 1 (Waist→FP)', color='#00A878', alpha=0.8)
    ax4.bar(x, p2_means, width, label='Phase 2 (Wrist→Waist)', color='#FFB627', alpha=0.8)
    ax4.bar(x + width, p3_means, width, label='Phase 3 (Wrist→FP)', color='#FF6B35', alpha=0.8)
    
    ax4.set_ylabel('Mean |Correlation|', fontsize=11)
    ax4.set_title('Performance by Activity', fontsize=12, fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels([a.capitalize() for a in activities], rotation=45)
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    
    summary_path = OUT_DIR / "phase_comparison_summary.png"
    fig.savefig(summary_path, dpi=300, facecolor='white', bbox_inches='tight')
    plt.close()
    
    print(f"\n📊 Phase comparison plot saved: {summary_path}")

# ==================== MAIN ====================
if __name__ == "__main__":
    # Run 3-phase analysis
    df = run_three_phase_batch(BASE_PATH, max_trials=None)
    
    # Generate comparison plots
    if df is not None:
        generate_phase_comparison_plot(df)
        
        print("\n" + "="*80)
        print("✅ COMPLETE 3-PHASE ANALYSIS FINISHED!")
        print("="*80)
        print("\n📁 Output Structure:")
        print(f"   {OUT_DIR}/")
        print(f"   ├── three_phase_*.png         (Individual trial visualizations)")
        print(f"   ├── three_phase_results.csv   (Comprehensive results)")
        print(f"   └── phase_comparison_summary.png (Statistical summary)")
        print("\n🎯 Next Steps:")
        print("   1. Review Phase 1 results → Ground truth quality")
        print("   2. Analyze Phase 2 → Transfer learning potential")
        print("   3. Evaluate Phase 3 → Deployment readiness")
        print("   4. Build predictive model: Wrist features → GRF estimation")

🎯 SCIENTIFIC 3-PHASE SYNCHRONIZATION ANALYSIS
Phase 1: Waist → Force Plate (Reference Ground Truth)
Phase 2: Wrist → Waist (Transfer Learning)
Phase 3: Wrist → Force Plate (End-to-End Deployment)
Base path: /kaggle/input/dataset10/Dataset
Output: scientific_sync_analysis

🔍 Scanning directories...
  📁 P1-1990-186.5-95.8
     FP: 70, Wrist: 36, Waist: 46
     ✓ walking: walk0005_f_6.csv...
     ✓ drop: drop0008_f_8.csv...
     ✓ drop: drop0001_f_8.csv...
     ✓ walking: walk0001_f_8.csv...
     ✓ heel: heel0003_f_8.csv...
     ✓ walking: walk0006_f_7.csv...
     ✓ walking: walk0007_f_8.csv...
     ✓ drop: drop0003_f_8.csv...
     ✓ heel: heel0007_f_8.csv...
     ✓ drop: drop0010_f_8.csv...
     ✓ jogging: jog0003_f_7.csv...
     ✓ drop: drop0007_f_8.csv...
     ✓ jogging: jog0004_f_8.csv...
     ✓ walking: walk0004_f_8.csv...
     ✓ jogging: jog0004_f_7.csv...
     ✓ walking: walk0002_f_6.csv...
     ✓ walking: walk0001_f_7.csv...
     ✓ jogging: jog0001_f_6.csv...
     ✓ walking: walk0

In [ ]:
full Trial Synchronise

In [1]:
#!/usr/bin/env python3
"""
Fixed Synchronization - Uses FULL trial data (no window extraction)
Properly aligns IMU and Force Plate for entire recording
"""

import os
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import correlate, butter, sosfiltfilt

# ==================== CONFIGURATION ====================
BASE_PATH = Path("/kaggle/input/dataset10/Dataset")
OUT_DIR = Path("sync_plots_full_trial")
OUT_DIR.mkdir(exist_ok=True)

# ==================== FILE DETECTION ====================
def is_force_plate_csv(path: Path):
    """Detect force plate CSV by Force_Z column."""
    try:
        for skip in [26, 27, 28, 0]:
            try:
                df = pd.read_csv(path, skiprows=skip, nrows=1)
                cols = [c.lower() for c in df.columns]
                if any(("force" in c and "z" in c) for c in cols):
                    return True
            except:
                continue
        return False
    except:
        return False

def is_watch_csv(path: Path):
    """Detect watch CSV by IMU columns."""
    try:
        df = pd.read_csv(path, nrows=1)
        needed = {'userAccelerationX', 'userAccelerationY', 'userAccelerationZ'}
        return needed.issubset(set(df.columns))
    except:
        return False

def infer_activity(name: str) -> str:
    """Infer activity from filename."""
    name_lower = name.lower()
    patterns = {
        'heel': ['heel'],
        'drop': ['drop'],
        'running': ['run'],
        'jogging': ['jog'],
        'walking': ['walk']
    }
    
    for activity, keywords in patterns.items():
        if any(kw in name_lower for kw in keywords):
            return activity
    return "unknown"

# ==================== SIGNAL PROCESSING ====================
def zscore_safe(x):
    """Z-score normalization."""
    x = np.asarray(x, dtype=float)
    mu = np.nanmean(x)
    sd = np.nanstd(x)
    return (x - mu) / (sd + 1e-10)

def read_force_plate_full(path: Path):
    """
    Read FULL force plate data (no windowing).
    Returns: (force_z_array, sampling_rate)
    """
    df = None
    for skip in [26, 27, 28, 25, 0]:
        try:
            df = pd.read_csv(path, skiprows=skip)
            df.columns = [c.strip() for c in df.columns]
            if any("force" in c.lower() and "z" in c.lower() for c in df.columns):
                break
        except:
            continue
    
    if df is None:
        raise ValueError(f"Cannot read {path.name}")
    
    # Find Force_Z column
    fz_col = next((c for c in df.columns if "force" in c.lower() and "z" in c.lower()), None)
    if fz_col is None:
        raise ValueError(f"No Force_Z in {path.name}")
    
    force_z = pd.to_numeric(df[fz_col], errors="coerce").fillna(0.0).to_numpy()
    
    # Get sampling rate from time column if available
    time_col = None
    for c in df.columns:
        if c.lower().strip() in ("time", "time (s)", "time(s)"):
            time_col = c
            break
    
    if time_col:
        time = pd.to_numeric(df[time_col], errors="coerce").to_numpy()
        time = time[time > 0]
        if len(time) > 1:
            dt = np.median(np.diff(time))
            fs = 1.0 / dt if dt > 0 else 1000.0
        else:
            fs = 1000.0
    else:
        fs = 1000.0  # AMTI default
    
    return force_z, fs

def read_watch_full(path: Path):
    """
    Read FULL IMU data (no windowing).
    Returns: (acc_magnitude_array, sampling_rate)
    """
    df = pd.read_csv(path)
    cols = ['userAccelerationX', 'userAccelerationY', 'userAccelerationZ']
    
    for c in cols:
        if c not in df.columns:
            raise ValueError(f"Missing {c}")
    
    acc = df[cols].to_numpy()
    acc_mag = np.sqrt((acc ** 2).sum(axis=1))
    
    # Estimate sampling rate
    if 'time' in df.columns or 'Time' in df.columns:
        time_col = 'time' if 'time' in df.columns else 'Time'
        time = pd.to_numeric(df[time_col], errors='coerce').dropna().to_numpy()
        if len(time) > 1:
            dt = np.median(np.diff(time))
            fs = 1.0 / dt if dt > 0 else 100.0
        else:
            fs = 100.0
    else:
        fs = 100.0  # Apple Watch typical rate
    
    return acc_mag, fs

def align_signals(force_sig, imu_sig, fs_force, fs_imu):
    """
    Align force and IMU signals by resampling to common rate and length.
    Uses the shorter duration to avoid extrapolation.
    """
    # Calculate durations
    duration_force = len(force_sig) / fs_force
    duration_imu = len(imu_sig) / fs_imu
    
    # Use shorter duration to avoid extrapolation
    common_duration = min(duration_force, duration_imu)
    
    # Use higher sampling rate as target (for better resolution)
    target_fs = max(fs_force, fs_imu)
    target_samples = int(common_duration * target_fs)
    
    # Resample both to common length
    def resample_signal(sig, target_len):
        if len(sig) == target_len:
            return sig
        old_idx = np.linspace(0, len(sig)-1, len(sig))
        new_idx = np.linspace(0, len(sig)-1, target_len)
        return np.interp(new_idx, old_idx, sig)
    
    # Truncate to common duration first
    force_samples = int(common_duration * fs_force)
    imu_samples = int(common_duration * fs_imu)
    
    force_truncated = force_sig[:force_samples]
    imu_truncated = imu_sig[:imu_samples]
    
    # Resample to common length
    force_aligned = resample_signal(force_truncated, target_samples)
    imu_aligned = resample_signal(imu_truncated, target_samples)
    
    return force_aligned, imu_aligned, target_fs

def find_best_lag(ref, sig, max_lag=100):
    """
    Cross-correlation lag finder for synchronized signals.
    """
    n = min(len(ref), len(sig))
    if n < 20:
        return 0, 0.0
    
    ref_z = zscore_safe(ref[:n])
    sig_z = zscore_safe(sig[:n])
    
    corr = correlate(sig_z, ref_z, mode='full')
    lags = np.arange(-n+1, n)
    
    # Limit search window
    mask = (lags >= -max_lag) & (lags <= max_lag)
    corr_valid = corr[mask]
    lags_valid = lags[mask]
    
    best_idx = np.argmax(corr_valid)
    best_lag = int(lags_valid[best_idx])
    max_corr = float(corr_valid[best_idx] / (np.linalg.norm(ref_z) * np.linalg.norm(sig_z) + 1e-9))
    
    return best_lag, max_corr

# ==================== DISCOVERY ====================
def discover_triads(base: Path):
    """Find FP + Wrist + Waist triads."""
    triads = []
    
    print("🔍 Scanning directories...")
    
    for participant_dir in sorted(d for d in base.iterdir() if d.is_dir()):
        print(f"  📁 {participant_dir.name}")
        csv_files = list(participant_dir.glob("*.csv"))
        
        # Classify files
        fp_files = []
        wrist_files = []
        waist_files = []
        
        for filepath in csv_files:
            fname_lower = filepath.name.lower()
            
            if is_force_plate_csv(filepath):
                fp_files.append(filepath)
            elif is_watch_csv(filepath):
                if 'wrist' in fname_lower:
                    wrist_files.append(filepath)
                elif 'waist' in fname_lower:
                    waist_files.append(filepath)
        
        print(f"     FP: {len(fp_files)}, Wrist: {len(wrist_files)}, Waist: {len(waist_files)}")
        
        # Match by activity
        for fp_path in fp_files:
            activity = infer_activity(fp_path.name)
            if activity == "unknown":
                continue
            
            wrist_match = None
            waist_match = None
            
            for wrist_path in wrist_files:
                if infer_activity(wrist_path.name) == activity:
                    wrist_match = wrist_path
                    break
            
            for waist_path in waist_files:
                if infer_activity(waist_path.name) == activity:
                    waist_match = waist_path
                    break
            
            if wrist_match and waist_match:
                triads.append({
                    "participant": participant_dir.name,
                    "activity": activity,
                    "trial_id": 0,
                    "fp": fp_path,
                    "wrist": wrist_match,
                    "waist": waist_match
                })
                print(f"     ✓ {activity}: {fp_path.name[:40]}...")
    
    return triads

# ==================== PLOTTING ====================
def create_sync_plot(fp_sig, wrist_sig, waist_sig,
                    lag_wrist_ms, lag_waist_ms,
                    corr_wrist, corr_waist,
                    participant, activity, trial_id,
                    fs, out_path):
    """Create full trial synchronization visualization."""
    
    t = np.arange(len(fp_sig)) / fs
    
    # Normalize for visualization
    fp_z = zscore_safe(fp_sig)
    wrist_z = zscore_safe(wrist_sig)
    waist_z = zscore_safe(waist_sig)
    
    # Plot
    fig, axes = plt.subplots(2, 1, figsize=(16, 8))
    
    # Top panel: Full trial overview
    axes[0].plot(t, fp_z, linewidth=2, label="Force Plate GRF-Z",
                color="#FF6B35", alpha=0.9, zorder=3)
    axes[0].plot(t, wrist_z, linewidth=1.5,
                label=f"Wrist |acc| (lag={lag_wrist_ms:.1f}ms, r={corr_wrist:.3f})",
                color="#004E89", alpha=0.7, zorder=2)
    axes[0].plot(t, waist_z, linewidth=1.5,
                label=f"Waist |acc| (lag={lag_waist_ms:.1f}ms, r={corr_waist:.3f})",
                color="#00A878", alpha=0.7, zorder=1)
    
    wrist_q = "🟢" if corr_wrist > 0.7 else "🟡" if corr_wrist > 0.5 else "🔴"
    waist_q = "🟢" if corr_waist > 0.7 else "🟡" if corr_waist > 0.5 else "🔴"
    
    axes[0].set_title(f"{wrist_q}{waist_q} Full Trial Sync: {activity.capitalize()} | {participant}",
                     fontsize=14, fontweight='bold')
    axes[0].set_xlabel("Time (s)", fontsize=11)
    axes[0].set_ylabel("Z-scored amplitude", fontsize=11)
    axes[0].grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    axes[0].legend(loc="upper right", fontsize=10, framealpha=0.9)
    
    # Bottom panel: Zoomed view of first 5 seconds
    zoom_samples = min(int(5 * fs), len(fp_sig))
    t_zoom = t[:zoom_samples]
    
    axes[1].plot(t_zoom, fp_z[:zoom_samples], linewidth=2,
                label="Force Plate", color="#FF6B35", alpha=0.9)
    axes[1].plot(t_zoom, wrist_z[:zoom_samples], linewidth=1.5,
                label="Wrist", color="#004E89", alpha=0.7)
    axes[1].plot(t_zoom, waist_z[:zoom_samples], linewidth=1.5,
                label="Waist", color="#00A878", alpha=0.7)
    
    axes[1].set_title("Zoomed View (First 5 seconds)", fontsize=12, fontweight='bold')
    axes[1].set_xlabel("Time (s)", fontsize=11)
    axes[1].set_ylabel("Z-scored amplitude", fontsize=11)
    axes[1].grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    axes[1].legend(loc="upper right", fontsize=9)
    
    # Add info box
    duration_text = (f"Full Trial | Duration: {t[-1]:.2f}s | Samples: {len(fp_sig)} | "
                    f"Rate: {fs:.1f}Hz | Original Data (No Windowing)")
    fig.text(0.5, 0.01, duration_text, ha='center', fontsize=9,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.6))
    
    fig.tight_layout(rect=[0, 0.03, 1, 1])
    fig.savefig(out_path, dpi=200, facecolor="white", bbox_inches="tight")
    plt.close(fig)

# ==================== PROCESSING ====================
def process_triad(triad):
    """Process a single triad with full trial data."""
    try:
        # Read FULL force plate data
        fp_sig, fs_fp = read_force_plate_full(triad["fp"])
        
        # Read FULL IMU data
        wrist_sig, fs_wrist = read_watch_full(triad["wrist"])
        waist_sig, fs_waist = read_watch_full(triad["waist"])
        
        # Align all signals to common sampling rate and length
        fp_aligned, wrist_aligned, fs_common = align_signals(fp_sig, wrist_sig, fs_fp, fs_wrist)
        fp_aligned2, waist_aligned, _ = align_signals(fp_sig, waist_sig, fs_fp, fs_waist)
        
        # Use the alignment that resulted in longer duration
        if len(fp_aligned) >= len(fp_aligned2):
            fp_final = fp_aligned
            wrist_final = wrist_aligned
            # Resample waist to match
            waist_final = np.interp(
                np.linspace(0, len(waist_aligned)-1, len(fp_aligned)),
                np.arange(len(waist_aligned)),
                waist_aligned
            )
        else:
            fp_final = fp_aligned2
            waist_final = waist_aligned
            # Resample wrist to match
            wrist_final = np.interp(
                np.linspace(0, len(wrist_aligned)-1, len(fp_aligned2)),
                np.arange(len(wrist_aligned)),
                wrist_aligned
            )
        
        # Ensure all same length
        min_len = min(len(fp_final), len(wrist_final), len(waist_final))
        fp_final = fp_final[:min_len]
        wrist_final = wrist_final[:min_len]
        waist_final = waist_final[:min_len]
        
        if min_len < 100:
            raise ValueError(f"Aligned signals too short: {min_len} samples")
        
        # Find alignment
        lag_wrist, corr_wrist = find_best_lag(fp_final, wrist_final)
        lag_waist, corr_waist = find_best_lag(fp_final, waist_final)
        
        lag_wrist_ms = (lag_wrist / fs_common) * 1000
        lag_waist_ms = (lag_waist / fs_common) * 1000
        
        # Create plot
        out_name = f"sync_{triad['participant']}_{triad['activity']}.png"
        out_path = OUT_DIR / out_name
        
        create_sync_plot(fp_final, wrist_final, waist_final,
                        lag_wrist_ms, lag_waist_ms,
                        corr_wrist, corr_waist,
                        triad['participant'], triad['activity'], triad['trial_id'],
                        fs_common, out_path)
        
        return {
            "participant": triad["participant"],
            "activity": triad["activity"],
            "trial_id": triad["trial_id"],
            "fp_file": triad["fp"].name,
            "wrist_file": triad["wrist"].name,
            "waist_file": triad["waist"].name,
            "lag_wrist_ms": lag_wrist_ms,
            "lag_waist_ms": lag_waist_ms,
            "corr_wrist": corr_wrist,
            "corr_waist": corr_waist,
            "duration_s": len(fp_final) / fs_common,
            "samples": len(fp_final),
            "sampling_rate": fs_common,
            "fp_original_samples": len(fp_sig),
            "wrist_original_samples": len(wrist_sig),
            "waist_original_samples": len(waist_sig),
            "status": "success"
        }
        
    except Exception as e:
        return {
            "participant": triad["participant"],
            "activity": triad["activity"],
            "trial_id": triad["trial_id"],
            "fp_file": triad.get("fp", Path("")).name,
            "status": "failed",
            "error": str(e)
        }

# ==================== BATCH ====================
def run_batch(base=BASE_PATH, max_trials=None):
    """Run batch synchronization analysis on FULL trials."""
    print("="*70)
    print("🔄 FULL TRIAL SYNCHRONIZATION (No Windowing)")
    print("="*70)
    print(f"Base path: {base}")
    print(f"Output: {OUT_DIR}\n")
    
    triads = discover_triads(base)
    
    if not triads:
        print("\n❌ No triads found!")
        print("\nTroubleshooting:")
        print("  1. Check BASE_PATH is correct")
        print("  2. Verify CSV files exist")
        print("  3. Ensure force plate files have Force_Z column")
        print("  4. Ensure IMU files have userAcceleration columns")
        return None
    
    total = len(triads)
    n_process = min(total, max_trials) if max_trials else total
    
    print(f"\n✅ Found {total} triads")
    print(f"   Processing {n_process}...\n")
    
    # Activity distribution
    by_activity = {}
    for t in triads:
        by_activity[t['activity']] = by_activity.get(t['activity'], 0) + 1
    
    print("Distribution:")
    for act, count in sorted(by_activity.items()):
        print(f"  {act:10s}: {count:3d}")
    print()
    
    # Process
    results = []
    for i, triad in enumerate(triads[:n_process]):
        result = process_triad(triad)
        results.append(result)
        
        if result['status'] == 'success':
            quality = "✓" if result['corr_wrist'] > 0.6 and result['corr_waist'] > 0.6 else "~"
            print(f"{quality} [{i+1:3d}/{n_process}] {result['participant']:20s} "
                  f"{result['activity']:8s} {result['duration_s']:5.1f}s "
                  f"(wrist r={result['corr_wrist']:.2f}, waist r={result['corr_waist']:.2f})")
        else:
            print(f"✗ [{i+1:3d}/{n_process}] {triad['participant']} {triad['activity']}: "
                  f"{result.get('error', 'Unknown')[:50]}")
    
    # Save results
    df = pd.DataFrame(results)
    csv_path = OUT_DIR / "sync_results_full_trial.csv"
    df.to_csv(csv_path, index=False)
    
    # Statistics
    success = df[df['status'] == 'success']
    if len(success) > 0:
        print("\n" + "="*70)
        print("📊 STATISTICS (Full Trial Analysis)")
        print("="*70)
        print(f"Success rate: {len(success)}/{len(df)} ({100*len(success)/len(df):.1f}%)")
        print(f"\nTrial durations: {success['duration_s'].mean():.1f}±{success['duration_s'].std():.1f}s")
        print(f"Sample counts: {int(success['samples'].mean())}±{int(success['samples'].std())} samples")
        print(f"\nWrist: lag={success['lag_wrist_ms'].mean():.1f}±{success['lag_wrist_ms'].std():.1f}ms, "
              f"r={success['corr_wrist'].mean():.3f}±{success['corr_wrist'].std():.3f}")
        print(f"Waist: lag={success['lag_waist_ms'].mean():.1f}±{success['lag_waist_ms'].std():.1f}ms, "
              f"r={success['corr_waist'].mean():.3f}±{success['corr_waist'].std():.3f}")
        
        print(f"\nBy activity:")
        for act in sorted(success['activity'].unique()):
            act_data = success[success['activity'] == act]
            print(f"  {act:10s}: n={len(act_data):3d}, dur={act_data['duration_s'].mean():5.1f}s, "
                  f"wrist_r={act_data['corr_wrist'].mean():.3f}, "
                  f"waist_r={act_data['corr_waist'].mean():.3f}")
        
        # Quality breakdown
        high_q = success[(success['corr_wrist'] > 0.7) & (success['corr_waist'] > 0.7)]
        med_q = success[((success['corr_wrist'] > 0.5) | (success['corr_waist'] > 0.5)) & 
                       ((success['corr_wrist'] <= 0.7) | (success['corr_waist'] <= 0.7))]
        low_q = success[(success['corr_wrist'] <= 0.5) & (success['corr_waist'] <= 0.5)]
        
        print(f"\nQuality:")
        print(f"  🟢 High (r>0.7): {len(high_q)} ({100*len(high_q)/len(success):.1f}%)")
        print(f"  🟡 Med (r>0.5):  {len(med_q)} ({100*len(med_q)/len(success):.1f}%)")
        print(f"  🔴 Low (r≤0.5):  {len(low_q)} ({100*len(low_q)/len(success):.1f}%)")
    
    print(f"\n✅ Results saved: {csv_path}")
    print(f"   Plots saved: {OUT_DIR}/")
    print(f"\n💡 Note: This analysis uses FULL trial data (no windowing)")
    print(f"   Original force plate samples preserved in alignment")
    
    return df

if __name__ == "__main__":
    df = run_batch(BASE_PATH, max_trials=None)  # Process all triads

🔄 FULL TRIAL SYNCHRONIZATION (No Windowing)
Base path: /kaggle/input/dataset10/Dataset
Output: sync_plots_full_trial

🔍 Scanning directories...
  📁 P1-1990-186.5-95.8
     FP: 70, Wrist: 36, Waist: 46
     ✓ walking: walk0005_f_6.csv...
     ✓ drop: drop0008_f_8.csv...
     ✓ drop: drop0001_f_8.csv...
     ✓ walking: walk0001_f_8.csv...
     ✓ heel: heel0003_f_8.csv...
     ✓ walking: walk0006_f_7.csv...
     ✓ walking: walk0007_f_8.csv...
     ✓ drop: drop0003_f_8.csv...
     ✓ heel: heel0007_f_8.csv...
     ✓ drop: drop0010_f_8.csv...
     ✓ jogging: jog0003_f_7.csv...
     ✓ drop: drop0007_f_8.csv...
     ✓ jogging: jog0004_f_8.csv...
     ✓ walking: walk0004_f_8.csv...
     ✓ jogging: jog0004_f_7.csv...
     ✓ walking: walk0002_f_6.csv...
     ✓ walking: walk0001_f_7.csv...
     ✓ jogging: jog0001_f_6.csv...
     ✓ walking: walk0006_f_6.csv...
     ✓ running: run0001_f_7.csv...
     ✓ running: run0009_f_6.csv...
     ✓ walking: walk0005_f_7.csv...
     ✓ walking: walk0009_f_6.csv..

/tmp/ipykernel_10/1911113477.py:327: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  fig.tight_layout(rect=[0, 0.03, 1, 1])
/tmp/ipykernel_10/1911113477.py:328: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  fig.savefig(out_path, dpi=200, facecolor="white", bbox_inches="tight")


~ [  1/753] P1-1990-186.5-95.8   walking    6.9s (wrist r=-0.01, waist r=0.01)


/tmp/ipykernel_10/1911113477.py:327: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  fig.tight_layout(rect=[0, 0.03, 1, 1])
/tmp/ipykernel_10/1911113477.py:328: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  fig.savefig(out_path, dpi=200, facecolor="white", bbox_inches="tight")


~ [  2/753] P1-1990-186.5-95.8   drop       4.3s (wrist r=-0.05, waist r=0.41)


...

~ [751/753] P9-1992-167-60.2     drop       5.0s (wrist r=0.30, waist r=-0.17)


/tmp/ipykernel_10/1911113477.py:327: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  fig.tight_layout(rect=[0, 0.03, 1, 1])
/tmp/ipykernel_10/1911113477.py:328: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  fig.savefig(out_path, dpi=200, facecolor="white", bbox_inches="tight")


~ [752/753] P9-1992-167-60.2     running    6.4s (wrist r=0.11, waist r=0.20)


/tmp/ipykernel_10/1911113477.py:327: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  fig.tight_layout(rect=[0, 0.03, 1, 1])
/tmp/ipykernel_10/1911113477.py:328: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  fig.savefig(out_path, dpi=200, facecolor="white", bbox_inches="tight")


~ [753/753] P9-1992-167-60.2     drop       5.0s (wrist r=-0.09, waist r=-0.28)

📊 STATISTICS (Full Trial Analysis)
Success rate: 753/753 (100.0%)

Trial durations: 6.8±1.7s
Sample counts: 6833±1688 samples

Wrist: lag=-0.0±83.5ms, r=0.143±0.178
Waist: lag=8.0±78.2ms, r=0.166±0.149

By activity:
  drop      : n= 86, dur=  4.8s, wrist_r=0.243, waist_r=0.263
  heel      : n= 85, dur=  4.8s, wrist_r=0.153, waist_r=0.132
  jogging   : n=135, dur=  7.1s, wrist_r=0.168, waist_r=0.124
  running   : n=221, dur=  6.8s, wrist_r=0.121, waist_r=0.169
  walking   : n=226, dur=  8.3s, wrist_r=0.108, waist_r=0.163

Quality:
  🟢 High (r>0.7): 1 (0.1%)
  🟡 Med (r>0.5):  43 (5.7%)
  🔴 Low (r≤0.5):  709 (94.2%)

✅ Results saved: sync_plots_full_trial/sync_results_full_trial.csv
   Plots saved: sync_plots_full_trial/

💡 Note: This analysis uses FULL trial data (no windowing)
   Original force plate samples preserved in alignment


In [ ]:
3Phases results scientific results

In [10]:
#!/usr/bin/env python3
"""
================================================================================
COMPLETE 3-PHASE SYNCHRONIZATION ANALYSIS WITH PUBLICATION FIGURES
Works with corrected trial matching (proper trial ID matching)
================================================================================
Usage: Copy and paste this entire script into a Jupyter notebook cell
================================================================================
"""

import os
import re
import io
import json
import warnings
from pathlib import Path
from typing import Optional, Tuple, Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import correlate
from scipy.stats import pearsonr, sem, t
from scipy.interpolate import interp1d

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - EDIT THIS SECTION
# ============================================================================

BASE_PATH = Path("/kaggle/input/dataset10/Dataset")
OUTPUT_DIR = Path("improved_sync_analysis")
PUB_FIG_DIR = OUTPUT_DIR / "publication_figures"

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def extract_participant_info(folder_name: str) -> Tuple[Optional[int], Optional[int], Optional[float], Optional[float]]:
    """Extract participant ID, year, height, weight from folder name."""
    pats = [
        r'^P(\d+)-(\d+)-([\d.]+)-([\d.]+)$',
        r'^P(\d+)_(\d+)_([\d.]+)_([\d.]+)$',
    ]
    for p in pats:
        m = re.match(p, folder_name)
        if m:
            return int(m[1]), int(m[2]), float(m[3]), float(m[4])
    return None, None, None, None

def infer_activity(name: str) -> Optional[str]:
    """Infer activity type from filename."""
    n = name.lower()
    if "heel" in n:
        return "heel"
    if "drop" in n:
        return "drop"
    if "run" in n:
        return "running"
    if "jog" in n:
        return "jogging"
    if "walk" in n:
        return "walking"
    return None

def load_force_z_from_csv(fp_path: Path) -> Tuple[np.ndarray, Dict]:
    """Load Force Z channel from force plate CSV."""
    meta = {}
    with open(fp_path, "rb") as f:
        raw = f.read().decode("utf-8-sig", errors="replace")
    
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    
    # Extract metadata
    for ln in lines[:20]:
        if "," in ln:
            k, *v = [x.strip() for x in ln.split(",")]
            if k.upper() in {"NO_OF_SAMPLES", "FREQUENCY", "TIME_STAMP"}:
                meta[k] = v[0] if v else ""
    
    # Find header row with "force z"
    header_idx = None
    for i, ln in enumerate(lines):
        if "force" in ln.lower() and "z" in ln.lower():
            header_idx = i
            break
    
    # If not found, look for numeric header
    if header_idx is None:
        for i, ln in enumerate(lines):
            if all(re.fullmatch(r'\d+', c) for c in ln.split(",")[:6]):
                header_idx = i
                break
    
    # Default to row 26 if still not found
    if header_idx is None:
        header_idx = 26
    
    data_start = header_idx + 1
    header_cols = [c.strip() for c in lines[header_idx].split(",")]
    data_block = "\n".join(lines[data_start:])
    csv_str = ",".join(header_cols) + "\n" + data_block
    
    df = pd.read_csv(io.StringIO(csv_str))
    df.columns = [c.strip() for c in df.columns]
    
    # Find Force Z column
    fz_col = next((c for c in df.columns if "z" in c.lower() and "force" in c.lower()), None)
    if fz_col is None and "5" in df.columns:
        fz_col = "5"
    if fz_col is None:
        raise ValueError("Force Z not found")
    
    fz = pd.to_numeric(df[fz_col], errors="coerce").dropna().to_numpy()
    return fz, meta

def load_watch_matrix(csv_path: Path) -> np.ndarray:
    """Load IMU data from watch CSV."""
    IMU_COLS = [
        "userAccelerationX", "userAccelerationY", "userAccelerationZ",
        "rotationRateX", "rotationRateY", "rotationRateZ"
    ]
    
    df = pd.read_csv(csv_path)
    
    # Check if all columns exist
    if not all(c in df.columns for c in IMU_COLS):
        # Try alternative names
        alt_map = {
            "userAccelerationX": ["AccX", "accel_x"],
            "userAccelerationY": ["AccY", "accel_y"],
            "userAccelerationZ": ["AccZ", "accel_z"],
            "rotationRateX": ["GyroX", "gyro_x"],
            "rotationRateY": ["GyroY", "gyro_y"],
            "rotationRateZ": ["GyroZ", "gyro_z"],
        }
        
        for std_col, alts in alt_map.items():
            if std_col not in df.columns:
                for alt in alts:
                    if alt in df.columns:
                        df[std_col] = df[alt]
                        break
    
    return df[IMU_COLS].to_numpy()

def zscore_safe(x):
    """Z-score normalization with safety."""
    x = np.asarray(x, dtype=float)
    mu = np.nanmean(x)
    sd = np.nanstd(x)
    return (x - mu) / (sd + 1e-10)

def find_optimal_lag_xcorr(ref, sig, max_lag_s=0.25, fs=1000.0) -> Dict:
    """Find optimal lag using cross-correlation."""
    n = min(len(ref), len(sig))
    
    if n < 50:
        return {"lag_samples": 0, "correlation": 0.0, "pearson_r": 0.0}
    
    ref_z = zscore_safe(ref[:n])
    sig_z = zscore_safe(sig[:n])
    
    max_lag = int(round(max_lag_s * fs))
    corr = correlate(sig_z, ref_z, mode='full')
    lags = np.arange(-n + 1, n)
    mask = (lags >= -max_lag) & (lags <= max_lag)
    corr_valid = corr[mask]
    lags_valid = lags[mask]
    
    k = int(np.argmax(np.abs(corr_valid)))
    best_lag = int(lags_valid[k])
    
    # Compute Pearson r at the best lag
    if best_lag == 0:
        r, _ = pearsonr(ref_z, sig_z)
    elif best_lag > 0:
        overlap = min(len(ref_z) - best_lag, len(sig_z))
        if overlap > 10:
            r, _ = pearsonr(ref_z[:-best_lag], sig_z[best_lag:best_lag + overlap])
        else:
            r = 0.0
    else:  # best_lag < 0
        overlap = min(len(ref_z), len(sig_z) + best_lag)
        if overlap > 10:
            r, _ = pearsonr(ref_z[-best_lag:], sig_z[:overlap])
        else:
            r = 0.0
    
    return {
        "lag_samples": best_lag,
        "correlation": float(np.max(np.abs(corr_valid))),
        "pearson_r": float(r)
    }

# ============================================================================
# CORRECTED RESULTS ANALYZER - GENERATES PUBLICATION FIGURES
# ============================================================================

class CorrectedResultsAnalyzer:
    """Analyzer for corrected results - creates publication figures."""
    
    def __init__(self, df: pd.DataFrame):
        """Initialize with results dataframe."""
        self.df = df
        self.success_df = df[df['status'] == 'success'].copy() if 'status' in df.columns else df.copy()
        
        # Add quality metrics
        self.success_df['phase1_quality'] = self.success_df['phase1_corr'].abs()
        self.success_df['phase3_quality'] = self.success_df['phase3_corr'].abs()
        
        # Ensure output directory exists
        PUB_FIG_DIR.mkdir(parents=True, exist_ok=True)
        
        print(f"\n📊 Analyzer initialized with {len(self.success_df)} successful trials")
    
    def _get_quality_tier(self, corr):
        """Assign quality tier based on correlation."""
        abs_corr = abs(corr)
        if abs_corr > 0.7:
            return 'Excellent'
        elif abs_corr > 0.5:
            return 'Good'
        elif abs_corr > 0.3:
            return 'Moderate'
        else:
            return 'Poor'
    
    def create_manifest(self):
        """Create and save trial manifest."""
        manifest_data = []
        
        for _, row in self.success_df.iterrows():
            quality_tier = self._get_quality_tier(row['phase3_corr'])
            
            manifest_data.append({
                'trial_id': f"{row.get('participant', 'N/A')}_{row.get('activity', 'N/A')}",
                'participant': row.get('participant', 'N/A'),
                'activity': row.get('activity', 'N/A'),
                'phase1_correlation': row.get('phase1_corr', np.nan),
                'phase2_correlation': row.get('phase2_corr', np.nan),
                'phase3_correlation': row.get('phase3_corr', np.nan),
                'quality_tier': quality_tier,
            })
        
        manifest_df = pd.DataFrame(manifest_data)
        manifest_path = PUB_FIG_DIR / "trial_manifest.csv"
        manifest_df.to_csv(manifest_path, index=False)
        
        print(f"\n📄 Manifest saved: {manifest_path}")
        print(f"   Total trials: {len(manifest_df)}")
        print(f"   Participants: {manifest_df['participant'].nunique()}")
        
        return manifest_df
    
    def create_overview_figure(self):
        """Create 4-panel overview figure."""
        fig = plt.figure(figsize=(16, 12))
        
        activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
        activity_labels = ['Walking', 'Jogging', 'Running', 'Heel Drop', 'Step Drop']
        activity_colors = ['#6BAED6', '#FC8D62', '#FDB462', '#B3B3E6', '#8DD3C7']
        
        # (A) Quality Distribution by Activity - Box Plot
        ax1 = plt.subplot(2, 2, 1)
        box_data = []
        valid_activities = []
        valid_colors = []
        
        for i, activity in enumerate(activity_order):
            activity_data = self.success_df[self.success_df['activity'] == activity]['phase3_quality']
            if len(activity_data) > 0:
                box_data.append(activity_data.values)
                valid_activities.append(activity_labels[i])
                valid_colors.append(activity_colors[i])
        
        if box_data:
            bp = ax1.boxplot(box_data, labels=valid_activities, patch_artist=True,
                            widths=0.6, showfliers=True,
                            medianprops=dict(color='black', linewidth=2),
                            boxprops=dict(linewidth=1.5))
            
            for patch, color in zip(bp['boxes'], valid_colors):
                patch.set_facecolor(color)
                patch.set_alpha(0.7)
        
        ax1.set_ylabel('Correlation |r|', fontsize=12, fontweight='bold')
        ax1.set_title('(A) Wrist-FP Correlation by Activity', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='y')
        ax1.tick_params(axis='x', rotation=45)
        ax1.axhline(0.7, color='green', linestyle='--', alpha=0.5, label='Excellent (>0.7)')
        ax1.axhline(0.5, color='orange', linestyle='--', alpha=0.5, label='Good (>0.5)')
        ax1.legend(fontsize=9)
        
        # (B) Quality Distribution Pie
        ax2 = plt.subplot(2, 2, 2)
        
        quality_tiers = []
        for _, row in self.success_df.iterrows():
            abs_corr = abs(row['phase3_corr'])
            if abs_corr > 0.7:
                quality_tiers.append('Excellent')
            elif abs_corr > 0.5:
                quality_tiers.append('Good')
            elif abs_corr > 0.3:
                quality_tiers.append('Moderate')
            else:
                quality_tiers.append('Poor')
        
        quality_counts = pd.Series(quality_tiers).value_counts()
        colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']
        
        wedges, texts, autotexts = ax2.pie(quality_counts.values, 
                                           labels=quality_counts.index,
                                           colors=colors_pie[:len(quality_counts)],
                                           autopct='%1.1f%%',
                                           startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax2.set_title('(B) Quality Distribution (Phase 3)', fontsize=13, fontweight='bold')
        
        # (C) Phase 1 vs Phase 3 Scatter
        ax3 = plt.subplot(2, 2, 3)
        
        for activity in activity_order:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                color = activity_colors[activity_order.index(activity)]
                ax3.scatter(act_data['phase1_quality'], act_data['phase3_quality'],
                           c=color, label=activity.capitalize(), alpha=0.6, s=50)
        
        max_val = max(self.success_df['phase1_quality'].max(), self.success_df['phase3_quality'].max())
        ax3.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Perfect Transfer')
        
        ax3.set_xlabel('Phase 1: Waist→FP |r|', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Phase 3: Wrist→FP |r|', fontsize=12, fontweight='bold')
        ax3.set_title('(C) Transfer Learning Effectiveness', fontsize=13, fontweight='bold')
        ax3.grid(True, alpha=0.3)
        ax3.legend(fontsize=9, loc='lower right')
        ax3.set_xlim([0, 1])
        ax3.set_ylim([0, 1])
        
        # (D) Trial Count by Activity
        ax4 = plt.subplot(2, 2, 4)
        
        trial_counts = []
        trial_labels = []
        trial_colors = []
        
        for i, activity in enumerate(activity_order):
            count = len(self.success_df[self.success_df['activity'] == activity])
            if count > 0:
                trial_counts.append(count)
                trial_labels.append(activity_labels[i])
                trial_colors.append(activity_colors[i])
        
        bars = ax4.bar(trial_labels, trial_counts, color=trial_colors, 
                      alpha=0.8, edgecolor='black', linewidth=1.5)
        
        for bar, count in zip(bars, trial_counts):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(count)}', ha='center', va='bottom', fontweight='bold')
        
        ax4.set_ylabel('Number of Trials', fontsize=12, fontweight='bold')
        ax4.set_title('(D) Successfully Analyzed Trials', fontsize=13, fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        ax4.tick_params(axis='x', rotation=45)
        
        fig.suptitle('3-Phase Synchronization Analysis - Dataset Overview', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        save_path = PUB_FIG_DIR / 'dataset_overview.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Overview figure saved: {save_path}")
    
    def create_statistical_summary(self):
        """Create statistical summary figure."""
        fig = plt.figure(figsize=(16, 12))
        
        activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
        activity_labels = ['Walking', 'Jogging', 'Running', 'Heel Drop', 'Step Drop']
        
        # (A) Mean with 95% CI
        ax1 = plt.subplot(2, 2, 1)
        
        means_p1, cis_p1 = [], []
        means_p3, cis_p3 = [], []
        valid_labels = []
        
        for i, activity in enumerate(activity_order):
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 1:
                mean_p1 = act_data['phase1_corr'].mean()
                sem_p1 = sem(act_data['phase1_corr'].dropna())
                ci_p1 = sem_p1 * t.ppf(0.975, len(act_data) - 1)
                
                mean_p3 = act_data['phase3_corr'].mean()
                sem_p3 = sem(act_data['phase3_corr'].dropna())
                ci_p3 = sem_p3 * t.ppf(0.975, len(act_data) - 1)
                
                means_p1.append(mean_p1)
                cis_p1.append(ci_p1)
                means_p3.append(mean_p3)
                cis_p3.append(ci_p3)
                valid_labels.append(activity_labels[i])
        
        if valid_labels:
            x = np.arange(len(valid_labels))
            width = 0.35
            
            ax1.bar(x - width/2, means_p1, width, yerr=cis_p1,
                   label='Phase 1 (Waist)', color='#3498DB', alpha=0.8, capsize=5)
            ax1.bar(x + width/2, means_p3, width, yerr=cis_p3,
                   label='Phase 3 (Wrist)', color='#9B59B6', alpha=0.8, capsize=5)
            
            ax1.set_ylabel('Mean Correlation r', fontsize=12, fontweight='bold')
            ax1.set_title('(A) Mean Correlation with 95% CI', fontsize=13, fontweight='bold')
            ax1.set_xticks(x)
            ax1.set_xticklabels(valid_labels, rotation=45, ha='right')
            ax1.legend()
            ax1.grid(True, alpha=0.3, axis='y')
        
        # (B) Summary Statistics Table
        ax2 = plt.subplot(2, 2, 2)
        ax2.axis('off')
        
        summary_data = []
        for activity in activity_order:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                summary_data.append([
                    activity_labels[activity_order.index(activity)],
                    len(act_data),
                    f"{act_data['phase1_corr'].mean():.3f}",
                    f"{act_data['phase1_corr'].std():.3f}",
                    f"{act_data['phase3_corr'].mean():.3f}",
                    f"{act_data['phase3_corr'].std():.3f}"
                ])
        
        if summary_data:
            table = ax2.table(cellText=summary_data,
                             colLabels=['Activity', 'N', 'P1 Mean', 'P1 SD', 'P3 Mean', 'P3 SD'],
                             cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
            
            table.auto_set_font_size(False)
            table.set_fontsize(10)
            table.scale(1, 2.5)
            
            for i in range(6):
                table[(0, i)].set_facecolor('#4472C4')
                table[(0, i)].set_text_props(weight='bold', color='white')
        
        ax2.set_title('(B) Summary Statistics', fontsize=13, fontweight='bold', pad=20)
        
        # (C) Distribution Histogram
        ax3 = plt.subplot(2, 2, 3)
        
        phase3_quality = self.success_df['phase3_corr'].abs()
        ax3.hist(phase3_quality, bins=30, alpha=0.7,
                color='#9ECAE1', edgecolor='black', linewidth=1.5)
        
        mean_corr = phase3_quality.mean()
        ax3.axvline(mean_corr, color='red', linestyle='--', linewidth=2.5,
                   label=f'Mean: {mean_corr:.3f}')
        
        ax3.set_xlabel('Correlation |r|', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax3.set_title('(C) Overall Quality Distribution', fontsize=13, fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3, axis='y')
        
        # (D) Deployment Readiness Pie
        ax4 = plt.subplot(2, 2, 4)
        
        phase3_quality = self.success_df['phase3_corr'].abs()
        excellent = len(self.success_df[phase3_quality > 0.7])
        good = len(self.success_df[(phase3_quality > 0.5) & (phase3_quality <= 0.7)])
        moderate = len(self.success_df[(phase3_quality > 0.3) & (phase3_quality <= 0.5)])
        poor = len(self.success_df[phase3_quality <= 0.3])
        
        sizes = [excellent, good, moderate, poor]
        labels = ['Excellent\n(>0.7)', 'Good\n(>0.5)', 'Moderate\n(>0.3)', 'Poor\n(≤0.3)']
        colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']
        
        wedges, texts, autotexts = ax4.pie(sizes, labels=labels, colors=colors_pie,
                                           autopct=lambda p: f'{p:.1f}%\n({int(p*sum(sizes)/100)})',
                                           startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax4.set_title('(D) Deployment Readiness', fontsize=13, fontweight='bold')
        
        fig.suptitle('Statistical Summary - 3-Phase Analysis', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        save_path = PUB_FIG_DIR / 'statistical_summary.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Statistical summary saved: {save_path}")
    
    def print_summary_statistics(self):
        """Print summary statistics to console."""
        if len(self.success_df) == 0:
            print("No successful trials to analyze")
            return
        
        print("\n" + "="*80)
        print("📊 SUMMARY STATISTICS")
        print("="*80)
        
        print(f"\nPhase 1 (Waist→FP):")
        print(f"  Mean r: {self.success_df['phase1_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase1_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase1_corr'].min():.3f}, {self.success_df['phase1_corr'].max():.3f}]")
        
        print(f"\nPhase 2 (Waist→Wrist):")
        print(f"  Mean r: {self.success_df['phase2_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase2_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase2_corr'].min():.3f}, {self.success_df['phase2_corr'].max():.3f}]")
        
        print(f"\nPhase 3 (Wrist→FP):")
        print(f"  Mean r: {self.success_df['phase3_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase3_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase3_corr'].min():.3f}, {self.success_df['phase3_corr'].max():.3f}]")
        
        # By activity
        print(f"\nBy Activity:")
        for activity in ['walking', 'jogging', 'running', 'heel', 'drop']:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                print(f"\n  {activity.capitalize()}:")
                print(f"    Count: {len(act_data)}")
                print(f"    Phase 1 Mean r: {act_data['phase1_corr'].mean():.3f}")
                print(f"    Phase 3 Mean r: {act_data['phase3_corr'].mean():.3f}")
                print(f"    Phase 3 Std:   {act_data['phase3_corr'].std():.3f}")
    
    def create_all_figures(self):
        """Generate all publication figures."""
        print("\n🎨 Creating publication-quality figures...")
        print("="*80)
        
        self.create_manifest()
        self.create_overview_figure()
        self.create_statistical_summary()
        self.print_summary_statistics()
        
        print("\n" + "="*80)
        print("✅ ALL FIGURES GENERATED")
        print("="*80)

# ============================================================================
# MAIN ANALYSIS PIPELINE
# ============================================================================

def run_three_phase_analysis():
    """Run complete 3-phase analysis with corrected trial matching."""
    
    print("\n" + "="*80)
    print("🎯 IMPROVED 3-PHASE SYNCHRONIZATION ANALYSIS")
    print("="*80)
    print(f"Base path: {BASE_PATH}")
    print(f"Output: {OUTPUT_DIR}")
    
    # Check if saved results exist
    results_file = OUTPUT_DIR / "three_phase_results.csv"
    
    if results_file.exists():
        print(f"\n✅ Loading saved results from: {results_file}")
        df = pd.read_csv(results_file)
        print(f"   Loaded {len(df)} trials")
    else:
        print("\n🔍 Running corrected 3-phase analysis...")
        print("   This may take several minutes...")
        
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        results = []
        trial_count = 0
        
        # Scan all participants
        for folder_name in sorted(os.listdir(BASE_PATH)):
            pdir = BASE_PATH / folder_name
            
            if not pdir.is_dir():
                continue
            
            # Extract participant info
            pid, year, height_cm, weight_kg = extract_participant_info(folder_name)
            
            if pid is None or weight_kg is None:
                continue
            
            print(f"\n👤 {folder_name} (Weight: {weight_kg}kg)")
            
            # List all files
            files = [f for f in os.listdir(pdir) if f.endswith(".csv")]
            
            # Find force plate files
            fp_files = [f for f in files if re.match(r'^(walk|jog|run|heel|drop)\d+_f_\d+\.csv$', f, re.IGNORECASE)]
            
            # Find watch files
            watch_files = [f for f in files if "Trial" in f and ("Waist" in f or "Wrist" in f)]
            
            # Build mapping: trial ID -> watch files
            id_to_watch = {}
            for wf in watch_files:
                m = re.search(r'ID_(\d+)', wf)
                if m:
                    trial_id = int(m.group(1))
                    id_to_watch.setdefault(trial_id, []).append(wf)
            
            # Process each force plate file
            for fp in sorted(fp_files):
                activity = infer_activity(fp)
                
                if not activity:
                    continue
                
                # Extract trial number from force plate filename
                m = re.match(r'^(?:walk|jog|run|heel|drop)(\d+)_f_\d+\.csv$', fp, re.IGNORECASE)
                if not m:
                    continue
                
                trial_num = int(m.group(1))
                candidates = id_to_watch.get(trial_num, [])
                
                # Find matching wrist and waist files
                wrist_f = next((x for x in candidates if "Wrist" in x and activity.capitalize() in x), None)
                waist_f = next((x for x in candidates if "Waist" in x and activity.capitalize() in x), None)
                
                if not (wrist_f and waist_f):
                    continue
                
                try:
                    # Load data
                    fz, _ = load_force_z_from_csv(pdir / fp)
                    waist_X = load_watch_matrix(pdir / waist_f)
                    wrist_X = load_watch_matrix(pdir / wrist_f)
                    
                    # Ensure same length
                    n = min(len(fz), len(waist_X), len(wrist_X))
                    if n < 100:
                        continue
                    
                    fz_short = fz[:n]
                    waist_mag = np.sqrt((waist_X[:n, :3]**2).sum(axis=1))
                    wrist_mag = np.sqrt((wrist_X[:n, :3]**2).sum(axis=1))
                    
                    # Phase 1: Waist → FP
                    p1 = find_optimal_lag_xcorr(fz_short, waist_mag, fs=1000.0)
                    
                    # Phase 2: Waist → Wrist
                    p2 = find_optimal_lag_xcorr(waist_mag, wrist_mag, fs=100.0)
                    
                    # Phase 3: Wrist → FP
                    p3 = find_optimal_lag_xcorr(fz_short, wrist_mag, fs=1000.0)
                    
                    result = {
                        'participant': folder_name,
                        'activity': activity,
                        'trial_id': trial_num,
                        'phase1_corr': p1['pearson_r'],
                        'phase2_corr': p2['pearson_r'],
                        'phase3_corr': p3['pearson_r'],
                        'status': 'success'
                    }
                    
                    results.append(result)
                    trial_count += 1
                    
                    print(f"  ✅ {activity:8s} (trial {trial_num:2d}): "
                          f"P1={p1['pearson_r']:.3f}, P2={p2['pearson_r']:.3f}, P3={p3['pearson_r']:.3f}")
                    
                    if trial_count % 50 == 0:
                        print(f"     [{trial_count} trials processed]")
                
                except Exception as e:
                    print(f"  ❌ {activity} trial {trial_num}: {str(e)[:50]}")
                    continue
        
        # Save results
        df = pd.DataFrame(results)
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        df.to_csv(results_file, index=False)
        
        print(f"\n✅ Analysis complete: {trial_count} trials")
        print(f"   Saved to: {results_file}")
    
    return df

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution."""
    
    # Run analysis
    df = run_three_phase_analysis()
    
    if df is not None and len(df) > 0:
        # Create analyzer and generate all figures
        print("\n" + "="*80)
        print("🎨 GENERATING PUBLICATION FIGURES")
        print("="*80)
        
        analyzer = CorrectedResultsAnalyzer(df)
        analyzer.create_all_figures()
        
        # Final summary
        print("\n" + "="*80)
        print("✅ COMPLETE 3-PHASE ANALYSIS FINISHED!")
        print("="*80)
        print(f"\n📁 Output Directory: {OUTPUT_DIR}")
        print(f"📁 Publication Figures: {PUB_FIG_DIR}")
        print(f"\n📊 Generated Files:")
        print(f"   • three_phase_results.csv")
        print(f"   • publication_figures/")
        print(f"     ├── dataset_overview.png")
        print(f"     ├── statistical_summary.png")
        print(f"     └── trial_manifest.csv")
        
    else:
        print("\n❌ Analysis failed - no valid trials found!")

if __name__ == "__main__":
    main()


🎯 IMPROVED 3-PHASE SYNCHRONIZATION ANALYSIS
Base path: /kaggle/input/dataset10/Dataset
Output: improved_sync_analysis

✅ Loading saved results from: improved_sync_analysis/three_phase_results.csv
   Loaded 387 trials

🎨 GENERATING PUBLICATION FIGURES

📊 Analyzer initialized with 387 successful trials

🎨 Creating publication-quality figures...

📄 Manifest saved: improved_sync_analysis/publication_figures/trial_manifest.csv
   Total trials: 387
   Participants: 10
✅ Overview figure saved: improved_sync_analysis/publication_figures/dataset_overview.png
✅ Statistical summary saved: improved_sync_analysis/publication_figures/statistical_summary.png

📊 SUMMARY STATISTICS

Phase 1 (Waist→FP):
  Mean r: 0.385
  Std:    0.403
  Range:  [-0.632, 0.838]

Phase 2 (Waist→Wrist):
  Mean r: 0.162
  Std:    0.316
  Range:  [-0.617, 0.830]

Phase 3 (Wrist→FP):
  Mean r: -0.040
  Std:    0.581
  Range:  [-0.800, 0.889]

By Activity:

  Walking:
    Count: 84
    Phase 1 Mean r: 0.451
    Phase 3 Mean r

In [7]:
#!/usr/bin/env python3
"""
================================================================================
COMPLETE 3-PHASE SYNCHRONIZATION ANALYSIS
Handles flat file structure: [Activity]_[Placement]_Trial_[UUID]_ID_[NUMBER]_[DATE]_[TIME].csv
================================================================================
Usage: Copy and paste this entire script into a Jupyter notebook cell
================================================================================
"""

import os
import re
import io
import json
import warnings
from pathlib import Path
from typing import Optional, Tuple, Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import correlate
from scipy.stats import pearsonr, sem, t

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - EDIT THIS SECTION
# ============================================================================

BASE_PATH = Path("/kaggle/input/dataset10/Dataset")
OUTPUT_DIR = Path("improved_sync_analysis")
PUB_FIG_DIR = OUTPUT_DIR / "publication_figures"

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def extract_participant_info(folder_name: str) -> Tuple[Optional[int], Optional[int], Optional[float], Optional[float]]:
    """Extract participant ID, year, height, weight from folder name."""
    pats = [
        r'^P(\d+)-(\d+)-([\d.]+)-([\d.]+)$',
        r'^P(\d+)_(\d+)_([\d.]+)_([\d.]+)$',
    ]
    for p in pats:
        m = re.match(p, folder_name)
        if m:
            return int(m[1]), int(m[2]), float(m[3]), float(m[4])
    return None, None, None, None

def extract_trial_info(filename: str) -> Tuple[Optional[str], Optional[str], Optional[int]]:
    """
    Extract activity, placement (Waist/Wrist), and trial ID from filename.
    Format: [Activity]_[Placement]_Trial_[UUID]_ID_[NUMBER]_[DATE]_[TIME].csv
    """
    # Extract activity (before first underscore or before _Waist/_Wrist)
    m = re.search(r'^(\w+)_(Waist|Wrist)_Trial.*_ID_(\d+)_', filename)
    if m:
        activity = m.group(1).lower()
        placement = m.group(2)
        trial_id = int(m.group(3))
        return activity, placement, trial_id
    return None, None, None

def load_force_z_from_csv(fp_path: Path) -> Tuple[np.ndarray, Dict]:
    """Load Force Z channel from force plate CSV."""
    meta = {}
    with open(fp_path, "rb") as f:
        raw = f.read().decode("utf-8-sig", errors="replace")
    
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    
    # Extract metadata
    for ln in lines[:20]:
        if "," in ln:
            k, *v = [x.strip() for x in ln.split(",")]
            if k.upper() in {"NO_OF_SAMPLES", "FREQUENCY", "TIME_STAMP"}:
                meta[k] = v[0] if v else ""
    
    # Find header row with "force z"
    header_idx = None
    for i, ln in enumerate(lines):
        if "force" in ln.lower() and "z" in ln.lower():
            header_idx = i
            break
    
    # If not found, look for numeric header
    if header_idx is None:
        for i, ln in enumerate(lines):
            if all(re.fullmatch(r'\d+', c) for c in ln.split(",")[:6]):
                header_idx = i
                break
    
    # Default to row 26 if still not found
    if header_idx is None:
        header_idx = 26
    
    data_start = header_idx + 1
    header_cols = [c.strip() for c in lines[header_idx].split(",")]
    data_block = "\n".join(lines[data_start:])
    csv_str = ",".join(header_cols) + "\n" + data_block
    
    df = pd.read_csv(io.StringIO(csv_str))
    df.columns = [c.strip() for c in df.columns]
    
    # Find Force Z column
    fz_col = next((c for c in df.columns if "z" in c.lower() and "force" in c.lower()), None)
    if fz_col is None and "5" in df.columns:
        fz_col = "5"
    if fz_col is None:
        raise ValueError("Force Z not found")
    
    fz = pd.to_numeric(df[fz_col], errors="coerce").dropna().to_numpy()
    return fz, meta

def load_watch_matrix(csv_path: Path) -> np.ndarray:
    """Load IMU data from watch CSV."""
    IMU_COLS = [
        "userAccelerationX", "userAccelerationY", "userAccelerationZ",
        "rotationRateX", "rotationRateY", "rotationRateZ"
    ]
    
    df = pd.read_csv(csv_path)
    
    # Check if all columns exist
    if not all(c in df.columns for c in IMU_COLS):
        # Try alternative names
        alt_map = {
            "userAccelerationX": ["AccX", "accel_x"],
            "userAccelerationY": ["AccY", "accel_y"],
            "userAccelerationZ": ["AccZ", "accel_z"],
            "rotationRateX": ["GyroX", "gyro_x"],
            "rotationRateY": ["GyroY", "gyro_y"],
            "rotationRateZ": ["GyroZ", "gyro_z"],
        }
        
        for std_col, alts in alt_map.items():
            if std_col not in df.columns:
                for alt in alts:
                    if alt in df.columns:
                        df[std_col] = df[alt]
                        break
    
    return df[IMU_COLS].to_numpy()

def zscore_safe(x):
    """Z-score normalization with safety."""
    x = np.asarray(x, dtype=float)
    mu = np.nanmean(x)
    sd = np.nanstd(x)
    return (x - mu) / (sd + 1e-10)

def find_optimal_lag_xcorr(ref, sig, max_lag_s=0.25, fs=1000.0) -> Dict:
    """Find optimal lag using cross-correlation."""
    n = min(len(ref), len(sig))
    
    if n < 50:
        return {"lag_samples": 0, "correlation": 0.0, "pearson_r": 0.0}
    
    ref_z = zscore_safe(ref[:n])
    sig_z = zscore_safe(sig[:n])
    
    max_lag = int(round(max_lag_s * fs))
    corr = correlate(sig_z, ref_z, mode='full')
    lags = np.arange(-n + 1, n)
    mask = (lags >= -max_lag) & (lags <= max_lag)
    corr_valid = corr[mask]
    lags_valid = lags[mask]
    
    k = int(np.argmax(np.abs(corr_valid)))
    best_lag = int(lags_valid[k])
    
    # Compute Pearson r at the best lag
    if best_lag == 0:
        r, _ = pearsonr(ref_z, sig_z)
    elif best_lag > 0:
        overlap = min(len(ref_z) - best_lag, len(sig_z))
        if overlap > 10:
            r, _ = pearsonr(ref_z[:-best_lag], sig_z[best_lag:best_lag + overlap])
        else:
            r = 0.0
    else:  # best_lag < 0
        overlap = min(len(ref_z), len(sig_z) + best_lag)
        if overlap > 10:
            r, _ = pearsonr(ref_z[-best_lag:], sig_z[:overlap])
        else:
            r = 0.0
    
    return {
        "lag_samples": best_lag,
        "correlation": float(np.max(np.abs(corr_valid))),
        "pearson_r": float(r)
    }

# ============================================================================
# CORRECTED RESULTS ANALYZER - GENERATES PUBLICATION FIGURES
# ============================================================================

class CorrectedResultsAnalyzer:
    """Analyzer for corrected results - creates publication figures."""
    
    def __init__(self, df: pd.DataFrame):
        """Initialize with results dataframe."""
        self.df = df
        self.success_df = df[df['status'] == 'success'].copy() if 'status' in df.columns else df.copy()
        
        # Add quality metrics
        self.success_df['phase1_quality'] = self.success_df['phase1_corr'].abs()
        self.success_df['phase3_quality'] = self.success_df['phase3_corr'].abs()
        
        # Ensure output directory exists
        PUB_FIG_DIR.mkdir(parents=True, exist_ok=True)
        
        print(f"\n📊 Analyzer initialized with {len(self.success_df)} successful trials")
    
    def _get_quality_tier(self, corr):
        """Assign quality tier based on correlation."""
        abs_corr = abs(corr)
        if abs_corr > 0.7:
            return 'Excellent'
        elif abs_corr > 0.5:
            return 'Good'
        elif abs_corr > 0.3:
            return 'Moderate'
        else:
            return 'Poor'
    
    def create_manifest(self):
        """Create and save trial manifest."""
        manifest_data = []
        
        for _, row in self.success_df.iterrows():
            quality_tier = self._get_quality_tier(row['phase3_corr'])
            
            manifest_data.append({
                'trial_id': f"{row.get('participant', 'N/A')}_{row.get('activity', 'N/A')}_ID{row.get('trial_num', 'N/A')}",
                'participant': row.get('participant', 'N/A'),
                'activity': row.get('activity', 'N/A'),
                'trial_num': row.get('trial_num', 'N/A'),
                'phase1_correlation': row.get('phase1_corr', np.nan),
                'phase2_correlation': row.get('phase2_corr', np.nan),
                'phase3_correlation': row.get('phase3_corr', np.nan),
                'quality_tier': quality_tier,
            })
        
        manifest_df = pd.DataFrame(manifest_data)
        manifest_path = PUB_FIG_DIR / "trial_manifest.csv"
        manifest_df.to_csv(manifest_path, index=False)
        
        print(f"\n📄 Manifest saved: {manifest_path}")
        print(f"   Total trials: {len(manifest_df)}")
        print(f"   Participants: {manifest_df['participant'].nunique()}")
        
        return manifest_df
    
    def create_overview_figure(self):
        """Create 4-panel overview figure."""
        if len(self.success_df) == 0:
            print("⚠️  No successful trials to visualize")
            return
        
        fig = plt.figure(figsize=(16, 12))
        
        activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
        activity_labels = ['Walking', 'Jogging', 'Running', 'Heel Drop', 'Step Drop']
        activity_colors = ['#6BAED6', '#FC8D62', '#FDB462', '#B3B3E6', '#8DD3C7']
        
        # (A) Quality Distribution by Activity - Box Plot
        ax1 = plt.subplot(2, 2, 1)
        box_data = []
        valid_activities = []
        valid_colors = []
        
        for i, activity in enumerate(activity_order):
            activity_data = self.success_df[self.success_df['activity'] == activity]['phase3_quality']
            if len(activity_data) > 0:
                box_data.append(activity_data.values)
                valid_activities.append(activity_labels[i])
                valid_colors.append(activity_colors[i])
        
        if box_data:
            bp = ax1.boxplot(box_data, labels=valid_activities, patch_artist=True,
                            widths=0.6, showfliers=True,
                            medianprops=dict(color='black', linewidth=2),
                            boxprops=dict(linewidth=1.5))
            
            for patch, color in zip(bp['boxes'], valid_colors):
                patch.set_facecolor(color)
                patch.set_alpha(0.7)
        
        ax1.set_ylabel('Correlation |r|', fontsize=12, fontweight='bold')
        ax1.set_title('(A) Wrist-FP Correlation by Activity', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='y')
        ax1.tick_params(axis='x', rotation=45)
        ax1.axhline(0.7, color='green', linestyle='--', alpha=0.5, label='Excellent (>0.7)')
        ax1.axhline(0.5, color='orange', linestyle='--', alpha=0.5, label='Good (>0.5)')
        ax1.legend(fontsize=9)
        
        # (B) Quality Distribution Pie
        ax2 = plt.subplot(2, 2, 2)
        
        quality_tiers = []
        for _, row in self.success_df.iterrows():
            abs_corr = abs(row['phase3_corr'])
            if abs_corr > 0.7:
                quality_tiers.append('Excellent')
            elif abs_corr > 0.5:
                quality_tiers.append('Good')
            elif abs_corr > 0.3:
                quality_tiers.append('Moderate')
            else:
                quality_tiers.append('Poor')
        
        quality_counts = pd.Series(quality_tiers).value_counts()
        colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']
        
        wedges, texts, autotexts = ax2.pie(quality_counts.values, 
                                           labels=quality_counts.index,
                                           colors=colors_pie[:len(quality_counts)],
                                           autopct='%1.1f%%',
                                           startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax2.set_title('(B) Quality Distribution (Phase 3)', fontsize=13, fontweight='bold')
        
        # (C) Phase 1 vs Phase 3 Scatter
        ax3 = plt.subplot(2, 2, 3)
        
        for activity in activity_order:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                color = activity_colors[activity_order.index(activity)]
                ax3.scatter(act_data['phase1_quality'], act_data['phase3_quality'],
                           c=color, label=activity.capitalize(), alpha=0.6, s=50)
        
        if len(self.success_df) > 0:
            max_val = max(self.success_df['phase1_quality'].max(), self.success_df['phase3_quality'].max())
            ax3.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Perfect Transfer')
        
        ax3.set_xlabel('Phase 1: Waist→FP |r|', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Phase 3: Wrist→FP |r|', fontsize=12, fontweight='bold')
        ax3.set_title('(C) Transfer Learning Effectiveness', fontsize=13, fontweight='bold')
        ax3.grid(True, alpha=0.3)
        ax3.legend(fontsize=9, loc='lower right')
        ax3.set_xlim([0, 1])
        ax3.set_ylim([0, 1])
        
        # (D) Trial Count by Activity
        ax4 = plt.subplot(2, 2, 4)
        
        trial_counts = []
        trial_labels = []
        trial_colors = []
        
        for i, activity in enumerate(activity_order):
            count = len(self.success_df[self.success_df['activity'] == activity])
            if count > 0:
                trial_counts.append(count)
                trial_labels.append(activity_labels[i])
                trial_colors.append(activity_colors[i])
        
        bars = ax4.bar(trial_labels, trial_counts, color=trial_colors, 
                      alpha=0.8, edgecolor='black', linewidth=1.5)
        
        for bar, count in zip(bars, trial_counts):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(count)}', ha='center', va='bottom', fontweight='bold')
        
        ax4.set_ylabel('Number of Trials', fontsize=12, fontweight='bold')
        ax4.set_title('(D) Successfully Analyzed Trials', fontsize=13, fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        ax4.tick_params(axis='x', rotation=45)
        
        fig.suptitle('3-Phase Synchronization Analysis - Dataset Overview', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        save_path = PUB_FIG_DIR / 'dataset_overview.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Overview figure saved: {save_path}")
    
    def create_statistical_summary(self):
        """Create statistical summary figure."""
        if len(self.success_df) == 0:
            return
        
        fig = plt.figure(figsize=(16, 12))
        
        activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
        activity_labels = ['Walking', 'Jogging', 'Running', 'Heel Drop', 'Step Drop']
        
        # (A) Mean with 95% CI
        ax1 = plt.subplot(2, 2, 1)
        
        means_p1, cis_p1 = [], []
        means_p3, cis_p3 = [], []
        valid_labels = []
        
        for i, activity in enumerate(activity_order):
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 1:
                mean_p1 = act_data['phase1_corr'].mean()
                sem_p1 = sem(act_data['phase1_corr'].dropna())
                ci_p1 = sem_p1 * t.ppf(0.975, len(act_data) - 1)
                
                mean_p3 = act_data['phase3_corr'].mean()
                sem_p3 = sem(act_data['phase3_corr'].dropna())
                ci_p3 = sem_p3 * t.ppf(0.975, len(act_data) - 1)
                
                means_p1.append(mean_p1)
                cis_p1.append(ci_p1)
                means_p3.append(mean_p3)
                cis_p3.append(ci_p3)
                valid_labels.append(activity_labels[i])
        
        if valid_labels:
            x = np.arange(len(valid_labels))
            width = 0.35
            
            ax1.bar(x - width/2, means_p1, width, yerr=cis_p1,
                   label='Phase 1 (Waist)', color='#3498DB', alpha=0.8, capsize=5)
            ax1.bar(x + width/2, means_p3, width, yerr=cis_p3,
                   label='Phase 3 (Wrist)', color='#9B59B6', alpha=0.8, capsize=5)
            
            ax1.set_ylabel('Mean Correlation r', fontsize=12, fontweight='bold')
            ax1.set_title('(A) Mean Correlation with 95% CI', fontsize=13, fontweight='bold')
            ax1.set_xticks(x)
            ax1.set_xticklabels(valid_labels, rotation=45, ha='right')
            ax1.legend()
            ax1.grid(True, alpha=0.3, axis='y')
        
        # (B) Summary Statistics Table
        ax2 = plt.subplot(2, 2, 2)
        ax2.axis('off')
        
        summary_data = []
        for activity in activity_order:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                summary_data.append([
                    activity_labels[activity_order.index(activity)],
                    len(act_data),
                    f"{act_data['phase1_corr'].mean():.3f}",
                    f"{act_data['phase1_corr'].std():.3f}",
                    f"{act_data['phase3_corr'].mean():.3f}",
                    f"{act_data['phase3_corr'].std():.3f}"
                ])
        
        if summary_data:
            table = ax2.table(cellText=summary_data,
                             colLabels=['Activity', 'N', 'P1 Mean', 'P1 SD', 'P3 Mean', 'P3 SD'],
                             cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
            
            table.auto_set_font_size(False)
            table.set_fontsize(10)
            table.scale(1, 2.5)
            
            for i in range(6):
                table[(0, i)].set_facecolor('#4472C4')
                table[(0, i)].set_text_props(weight='bold', color='white')
        
        ax2.set_title('(B) Summary Statistics', fontsize=13, fontweight='bold', pad=20)
        
        # (C) Distribution Histogram
        ax3 = plt.subplot(2, 2, 3)
        
        phase3_quality = self.success_df['phase3_corr'].abs()
        ax3.hist(phase3_quality, bins=30, alpha=0.7,
                color='#9ECAE1', edgecolor='black', linewidth=1.5)
        
        mean_corr = phase3_quality.mean()
        ax3.axvline(mean_corr, color='red', linestyle='--', linewidth=2.5,
                   label=f'Mean: {mean_corr:.3f}')
        
        ax3.set_xlabel('Correlation |r|', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax3.set_title('(C) Overall Quality Distribution', fontsize=13, fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3, axis='y')
        
        # (D) Deployment Readiness Pie
        ax4 = plt.subplot(2, 2, 4)
        
        phase3_quality = self.success_df['phase3_corr'].abs()
        excellent = len(self.success_df[phase3_quality > 0.7])
        good = len(self.success_df[(phase3_quality > 0.5) & (phase3_quality <= 0.7)])
        moderate = len(self.success_df[(phase3_quality > 0.3) & (phase3_quality <= 0.5)])
        poor = len(self.success_df[phase3_quality <= 0.3])
        
        sizes = [excellent, good, moderate, poor]
        labels = ['Excellent\n(>0.7)', 'Good\n(>0.5)', 'Moderate\n(>0.3)', 'Poor\n(≤0.3)']
        colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']
        
        wedges, texts, autotexts = ax4.pie(sizes, labels=labels, colors=colors_pie,
                                           autopct=lambda p: f'{p:.1f}%\n({int(p*sum(sizes)/100)})',
                                           startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax4.set_title('(D) Deployment Readiness', fontsize=13, fontweight='bold')
        
        fig.suptitle('Statistical Summary - 3-Phase Analysis', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        save_path = PUB_FIG_DIR / 'statistical_summary.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Statistical summary saved: {save_path}")
    
    def print_summary_statistics(self):
        """Print summary statistics to console."""
        if len(self.success_df) == 0:
            print("No successful trials to analyze")
            return
        
        print("\n" + "="*80)
        print("📊 SUMMARY STATISTICS")
        print("="*80)
        
        print(f"\nPhase 1 (Waist→FP):")
        print(f"  Mean r: {self.success_df['phase1_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase1_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase1_corr'].min():.3f}, {self.success_df['phase1_corr'].max():.3f}]")
        
        print(f"\nPhase 2 (Waist→Wrist):")
        print(f"  Mean r: {self.success_df['phase2_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase2_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase2_corr'].min():.3f}, {self.success_df['phase2_corr'].max():.3f}]")
        
        print(f"\nPhase 3 (Wrist→FP):")
        print(f"  Mean r: {self.success_df['phase3_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase3_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase3_corr'].min():.3f}, {self.success_df['phase3_corr'].max():.3f}]")
        
        # By activity
        print(f"\nBy Activity:")
        for activity in ['walking', 'jogging', 'running', 'heel', 'drop']:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                print(f"\n  {activity.capitalize()}:")
                print(f"    Count: {len(act_data)}")
                print(f"    Phase 1 Mean r: {act_data['phase1_corr'].mean():.3f}")
                print(f"    Phase 3 Mean r: {act_data['phase3_corr'].mean():.3f}")
                print(f"    Phase 3 Std:   {act_data['phase3_corr'].std():.3f}")
    
    def create_all_figures(self):
        """Generate all publication figures."""
        print("\n🎨 Creating publication-quality figures...")
        print("="*80)
        
        self.create_manifest()
        self.create_overview_figure()
        self.create_statistical_summary()
        self.print_summary_statistics()
        
        print("\n" + "="*80)
        print("✅ ALL FIGURES GENERATED")
        print("="*80)

# ============================================================================
# MAIN ANALYSIS PIPELINE
# ============================================================================

def run_three_phase_analysis():
    """Run complete 3-phase analysis with corrected trial matching."""
    
    print("\n" + "="*80)
    print("🎯 IMPROVED 3-PHASE SYNCHRONIZATION ANALYSIS")
    print("="*80)
    print(f"Base path: {BASE_PATH}")
    print(f"Output: {OUTPUT_DIR}")
    
    # Check if saved results exist
    results_file = OUTPUT_DIR / "three_phase_results.csv"
    
    if results_file.exists():
        print(f"\n✅ Loading saved results from: {results_file}")
        df = pd.read_csv(results_file)
        print(f"   Loaded {len(df)} trials")
    else:
        print("\n🔍 Running corrected 3-phase analysis...")
        print("   This may take several minutes...")
        
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        results = []
        trial_count = 0
        
        # Scan all participants
        for folder_name in sorted(os.listdir(BASE_PATH)):
            pdir = BASE_PATH / folder_name
            
            if not pdir.is_dir():
                continue
            
            # Extract participant info
            pid, year, height_cm, weight_kg = extract_participant_info(folder_name)
            
            if pid is None or weight_kg is None:
                print(f"⚠️  Skipped {folder_name}: Could not extract participant info")
                continue
            
            print(f"\n👤 {folder_name} (Weight: {weight_kg}kg)")
            
            # List all CSV files
            files = [f for f in os.listdir(pdir) if f.endswith(".csv")]
            
            # Build mapping: (activity, trial_id) -> {waist_file, wrist_file, fp_file}
            trial_map = {}
            
            for filename in files:
                activity, placement, trial_id = extract_trial_info(filename)
                
                if activity is None or trial_id is None:
                    continue
                
                key = (activity, trial_id)
                if key not in trial_map:
                    trial_map[key] = {}
                
                if placement == "Waist":
                    trial_map[key]['waist'] = filename
                elif placement == "Wrist":
                    trial_map[key]['wrist'] = filename
            
            # Now find force plate files (different format)
            # They should match by activity and trial ID
            fp_files = [f for f in files if "_f_" in f.lower()]  # Force plate indicator
            
            for fp_file in fp_files:
                # Try to extract activity from force plate filename
                # They might be named like: walk01_f_X.csv, Drop_FP_Trial_...csv, etc.
                activity = None
                trial_id = None
                
                # Pattern 1: [activity][number]_f_X.csv
                m = re.match(r'^([a-z]+)(\d+)_f_\d+\.csv$', fp_file, re.IGNORECASE)
                if m:
                    act_str = m.group(1).lower()
                    trial_id = int(m.group(2))
                    # Map to standard activity names
                    if 'walk' in act_str:
                        activity = 'walking'
                    elif 'run' in act_str:
                        activity = 'running'
                    elif 'jog' in act_str:
                        activity = 'jogging'
                    elif 'heel' in act_str:
                        activity = 'heel'
                    elif 'drop' in act_str:
                        activity = 'drop'
                
                if activity is None or trial_id is None:
                    continue
                
                key = (activity, trial_id)
                if key in trial_map:
                    trial_map[key]['fp'] = fp_file
            
            # Process each complete trial (has all three files)
            for (activity, trial_id), files_dict in trial_map.items():
                if not all(k in files_dict for k in ['waist', 'wrist', 'fp']):
                    continue
                
                try:
                    # Load data
                    fz, _ = load_force_z_from_csv(pdir / files_dict['fp'])
                    waist_X = load_watch_matrix(pdir / files_dict['waist'])
                    wrist_X = load_watch_matrix(pdir / files_dict['wrist'])
                    
                    # Ensure same length
                    n = min(len(fz), len(waist_X), len(wrist_X))
                    if n < 100:
                        continue
                    
                    fz_short = fz[:n]
                    waist_mag = np.sqrt((waist_X[:n, :3]**2).sum(axis=1))
                    wrist_mag = np.sqrt((wrist_X[:n, :3]**2).sum(axis=1))
                    
                    # Phase 1: Waist → FP
                    p1 = find_optimal_lag_xcorr(fz_short, waist_mag, fs=1000.0)
                    
                    # Phase 2: Waist → Wrist
                    p2 = find_optimal_lag_xcorr(waist_mag, wrist_mag, fs=100.0)
                    
                    # Phase 3: Wrist → FP
                    p3 = find_optimal_lag_xcorr(fz_short, wrist_mag, fs=1000.0)
                    
                    result = {
                        'participant': folder_name,
                        'activity': activity,
                        'trial_num': trial_id,
                        'phase1_corr': p1['pearson_r'],
                        'phase2_corr': p2['pearson_r'],
                        'phase3_corr': p3['pearson_r'],
                        'status': 'success'
                    }
                    
                    results.append(result)
                    trial_count += 1
                    
                    print(f"  ✅ {activity:8s} (trial {trial_id:2d}): "
                          f"P1={p1['pearson_r']:.3f}, P2={p2['pearson_r']:.3f}, P3={p3['pearson_r']:.3f}")
                    
                    if trial_count % 50 == 0:
                        print(f"     [{trial_count} trials processed]")
                
                except Exception as e:
                    print(f"  ❌ {activity} trial {trial_id}: {str(e)[:60]}")
                    continue
        
        # Save results
        df = pd.DataFrame(results)
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        df.to_csv(results_file, index=False)
        
        print(f"\n✅ Analysis complete: {trial_count} trials")
        print(f"   Saved to: {results_file}")
    
    return df

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution."""
    
    # Run analysis
    df = run_three_phase_analysis()
    
    if df is not None and len(df) > 0:
        # Create analyzer and generate all figures
        print("\n" + "="*80)
        print("🎨 GENERATING PUBLICATION FIGURES")
        print("="*80)
        
        analyzer = CorrectedResultsAnalyzer(df)
        analyzer.create_all_figures()
        
        # Final summary
        print("\n" + "="*80)
        print("✅ COMPLETE 3-PHASE ANALYSIS FINISHED!")
        print("="*80)
        print(f"\n📁 Output Directory: {OUTPUT_DIR}")
        print(f"📁 Publication Figures: {PUB_FIG_DIR}")
        print(f"\n📊 Generated Files:")
        print(f"   • three_phase_results.csv")
        print(f"   • publication_figures/")
        print(f"     ├── dataset_overview.png")
        print(f"     ├── statistical_summary.png")
        print(f"     └── trial_manifest.csv")
        
    else:
        print("\n❌ Analysis failed - no valid trials found!")

if __name__ == "__main__":
    main()


🎯 IMPROVED 3-PHASE SYNCHRONIZATION ANALYSIS
Base path: /kaggle/input/dataset10/Dataset
Output: improved_sync_analysis

🔍 Running corrected 3-phase analysis...
   This may take several minutes...

👤 P1-1990-186.5-95.8 (Weight: 95.8kg)
  ✅ running  (trial  5): P1=0.640, P2=0.494, P3=0.741
  ✅ heel     (trial  1): P1=0.430, P2=-0.137, P3=-0.581
  ✅ drop     (trial  1): P1=-0.517, P2=-0.264, P3=-0.723
  ✅ walking  (trial  8): P1=0.677, P2=0.459, P3=0.574
  ✅ drop     (trial  2): P1=0.573, P2=0.305, P3=0.634
  ✅ drop     (trial  9): P1=0.401, P2=-0.272, P3=-0.675
  ✅ heel     (trial  6): P1=0.680, P2=0.404, P3=0.828
  ✅ walking  (trial  2): P1=0.714, P2=0.606, P3=0.677
  ✅ drop     (trial  4): P1=0.505, P2=-0.177, P3=-0.528
  ✅ heel     (trial  4): P1=0.520, P2=0.189, P3=-0.616
  ✅ drop     (trial  7): P1=0.562, P2=-0.218, P3=-0.635
  ✅ heel     (trial 11): P1=0.502, P2=0.123, P3=-0.592
  ✅ walking  (trial  3): P1=0.451, P2=0.231, P3=0.639
  ✅ heel     (trial  8): P1=0.553, P2=0.159, P3=-0

In [ ]:
scientific_sync_analysis/three_phase_results.csv

In [5]:
#!/usr/bin/env python3
"""
Diagnostic script to inspect your CSV and fix column mapping issues
"""

import pandas as pd
from pathlib import Path

# Try to find your CSV file
possible_paths = [
    Path("/kaggle/working/improved_sync_analysis/three_phase_results.csv"),
    Path("/kaggle/working/improved_sync_analysis/three_phase_results.csv"),
    Path("/kaggle/working/improved_sync_analysis/three_phase_results.csv"),
]

csv_path = None
for p in possible_paths:
    if p.exists():
        csv_path = p
        break

if csv_path is None:
    print("❌ CSV file not found. Please provide the path to your three_phase_results.csv")
    print("\nTried these locations:")
    for p in possible_paths:
        print(f"  - {p}")
else:
    print(f"✅ Found CSV: {csv_path}")
    print(f"\n📊 Inspecting structure...")
    
    df = pd.read_csv(csv_path)
    
    print(f"\nShape: {df.shape}")
    print(f"\nColumns ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i}. {col}")
    
    print(f"\nFirst few rows:")
    print(df.head())
    
    print(f"\nData types:")
    print(df.dtypes)
    
    # Check for column name variations
    print(f"\n🔍 Checking for column variations:")
    
    correlation_cols = [c for c in df.columns if 'corr' in c.lower()]
    lag_cols = [c for c in df.columns if 'lag' in c.lower()]
    
    if correlation_cols:
        print(f"  Correlation columns: {correlation_cols}")
    else:
        print(f"  ⚠️  No correlation columns found!")
    
    if lag_cols:
        print(f"  Lag columns: {lag_cols}")
    else:
        print(f"  ⚠️  No lag columns found!")
    
    # Check what the actual column names are
    print(f"\n📋 All column names (for reference):")
    print(df.columns.tolist())

❌ CSV file not found. Please provide the path to your three_phase_results.csv

Tried these locations:
  - /kaggle/working/improved_sync_analysis/three_phase_results.csv
  - /kaggle/working/improved_sync_analysis/three_phase_results.csv
  - /kaggle/working/improved_sync_analysis/three_phase_results.csv


In [ ]:
waiting for results

In [7]:
#!/usr/bin/env python3
"""
Scientific 3-Phase Synchronization & Analysis
Phase 1: Waist → Force Plate (Reference Mapping)
Phase 2: Wrist → Waist (Transfer Learning)
Phase 3: Wrist → Force Plate (End-to-End Deployment)

Author: Scientific Biomechanics Pipeline
"""

import os
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import correlate, find_peaks
from scipy.stats import pearsonr

# ==================== CONFIGURATION ====================
BASE_PATH = Path("/kaggle/input/dataset10/Dataset")
OUT_DIR = Path("scientific_sync_analysis")
OUT_DIR.mkdir(exist_ok=True)

# Create subdirectories for each phase
PHASE_DIRS = {
    'phase1': OUT_DIR / "phase1_waist_to_fp",
    'phase2': OUT_DIR / "phase2_wrist_to_waist", 
    'phase3': OUT_DIR / "phase3_wrist_to_fp"
}

for phase_dir in PHASE_DIRS.values():
    phase_dir.mkdir(exist_ok=True)

# ==================== FILE DETECTION ====================
def is_force_plate_csv(path: Path):
    """Detect force plate by Force_Z column."""
    try:
        for skip in [26, 27, 28, 0]:
            try:
                df = pd.read_csv(path, skiprows=skip, nrows=1)
                cols = [c.lower() for c in df.columns]
                if any(("force" in c and "z" in c) for c in cols):
                    return True
            except:
                continue
        return False
    except:
        return False

def is_watch_csv(path: Path):
    """Detect watch CSV by IMU columns."""
    try:
        df = pd.read_csv(path, nrows=1)
        needed = {'userAccelerationX', 'userAccelerationY', 'userAccelerationZ'}
        return needed.issubset(set(df.columns))
    except:
        return False

def infer_activity(name: str) -> str:
    """Infer activity from filename."""
    name_lower = name.lower()
    patterns = {
        'heel': ['heel'],
        'drop': ['drop'],
        'running': ['run'],
        'jogging': ['jog'],
        'walking': ['walk']
    }
    
    for activity, keywords in patterns.items():
        if any(kw in name_lower for kw in keywords):
            return activity
    return "unknown"

# ==================== SIGNAL PROCESSING ====================
def zscore_safe(x):
    """Z-score normalization with safety."""
    x = np.asarray(x, dtype=float)
    mu = np.nanmean(x)
    sd = np.nanstd(x)
    return (x - mu) / (sd + 1e-10)

def read_force_plate_full(path: Path):
    """Read complete force plate data."""
    df = None
    for skip in [26, 27, 28, 25, 0]:
        try:
            df = pd.read_csv(path, skiprows=skip)
            df.columns = [c.strip() for c in df.columns]
            if any("force" in c.lower() and "z" in c.lower() for c in df.columns):
                break
        except:
            continue
    
    if df is None:
        raise ValueError(f"Cannot read {path.name}")
    
    fz_col = next((c for c in df.columns if "force" in c.lower() and "z" in c.lower()), None)
    if fz_col is None:
        raise ValueError(f"No Force_Z in {path.name}")
    
    force_z = pd.to_numeric(df[fz_col], errors="coerce").fillna(0.0).to_numpy()
    
    # Get time info
    time_col = next((c for c in df.columns if c.lower().strip() in ("time", "time (s)", "time(s)")), None)
    
    if time_col:
        time = pd.to_numeric(df[time_col], errors="coerce").to_numpy()
        time = time[time > 0]
        if len(time) > 1:
            dt = np.median(np.diff(time))
            fs = 1.0 / dt if dt > 0 else 1000.0
        else:
            fs = 1000.0
    else:
        fs = 1000.0
    
    return force_z, fs

def read_watch_full(path: Path):
    """Read complete IMU data with all channels."""
    df = pd.read_csv(path)
    
    # Check for required columns
    acc_cols = ['userAccelerationX', 'userAccelerationY', 'userAccelerationZ']
    gyro_cols = ['rotationRateX', 'rotationRateY', 'rotationRateZ']
    
    for c in acc_cols:
        if c not in df.columns:
            raise ValueError(f"Missing {c}")
    
    # Get acceleration
    acc = df[acc_cols].to_numpy()
    acc_mag = np.sqrt((acc ** 2).sum(axis=1))
    
    # Get gyroscope if available
    has_gyro = all(c in df.columns for c in gyro_cols)
    if has_gyro:
        gyro = df[gyro_cols].to_numpy()
        gyro_mag = np.sqrt((gyro ** 2).sum(axis=1))
    else:
        gyro_mag = None
    
    # Get vertical acceleration (Z-axis most relevant for GRF)
    acc_z = acc[:, 2]
    
    # Estimate sampling rate
    if 'time' in df.columns or 'Time' in df.columns:
        time_col = 'time' if 'time' in df.columns else 'Time'
        time = pd.to_numeric(df[time_col], errors='coerce').dropna().to_numpy()
        if len(time) > 1:
            dt = np.median(np.diff(time))
            fs = 1.0 / dt if dt > 0 else 100.0
        else:
            fs = 100.0
    else:
        fs = 100.0
    
    return {
        'acc_mag': acc_mag,
        'acc_z': acc_z,
        'acc_xyz': acc,
        'gyro_mag': gyro_mag,
        'fs': fs
    }

def align_signals_advanced(force_sig, imu_dict, fs_force):
    """
    Advanced signal alignment with multiple strategies.
    Returns aligned signals and metadata.
    """
    fs_imu = imu_dict['fs']
    
    # Calculate durations
    duration_force = len(force_sig) / fs_force
    duration_imu = len(imu_dict['acc_mag']) / fs_imu
    
    # Use shorter duration
    common_duration = min(duration_force, duration_imu)
    
    # Use higher sampling rate as target
    target_fs = max(fs_force, fs_imu)
    target_samples = int(common_duration * target_fs)
    
    # Resample function
    def resample_signal(sig, target_len):
        if len(sig) == target_len:
            return sig
        old_idx = np.linspace(0, len(sig)-1, len(sig))
        new_idx = np.linspace(0, len(sig)-1, target_len)
        return np.interp(new_idx, old_idx, sig)
    
    # Truncate to common duration
    force_samples = int(common_duration * fs_force)
    imu_samples = int(common_duration * fs_imu)
    
    force_truncated = force_sig[:force_samples]
    
    # Resample force to target
    force_aligned = resample_signal(force_truncated, target_samples)
    
    # Resample all IMU signals
    aligned_imu = {}
    for key in ['acc_mag', 'acc_z']:
        imu_truncated = imu_dict[key][:imu_samples]
        aligned_imu[key] = resample_signal(imu_truncated, target_samples)
    
    if imu_dict['gyro_mag'] is not None:
        gyro_truncated = imu_dict['gyro_mag'][:imu_samples]
        aligned_imu['gyro_mag'] = resample_signal(gyro_truncated, target_samples)
    
    return force_aligned, aligned_imu, target_fs

def find_optimal_lag_dtw(ref, sig, max_lag=200):
    """
    Find optimal lag using Dynamic Time Warping approach.
    More robust than simple cross-correlation for biomechanical signals.
    """
    n = min(len(ref), len(sig))
    if n < 50:
        return 0, 0.0, 0.0
    
    # Normalize signals
    ref_z = zscore_safe(ref[:n])
    sig_z = zscore_safe(sig[:n])
    
    # Cross-correlation
    corr = correlate(sig_z, ref_z, mode='full')
    lags = np.arange(-n+1, n)
    
    # Limit search window
    mask = (lags >= -max_lag) & (lags <= max_lag)
    corr_valid = corr[mask]
    lags_valid = lags[mask]
    
    # Find best lag
    best_idx = np.argmax(corr_valid)
    best_lag = int(lags_valid[best_idx])
    
    # Normalize correlation coefficient
    max_corr = float(corr_valid[best_idx] / (np.linalg.norm(ref_z) * np.linalg.norm(sig_z) + 1e-9))
    
    # Also calculate Pearson correlation at best lag
    if best_lag == 0:
        pearson_r, _ = pearsonr(ref_z, sig_z)
    elif best_lag > 0:
        pearson_r, _ = pearsonr(ref_z[:-best_lag], sig_z[best_lag:])
    else:
        pearson_r, _ = pearsonr(ref_z[-best_lag:], sig_z[:best_lag])
    
    return best_lag, max_corr, pearson_r

# ==================== DISCOVERY ====================
def discover_triads(base: Path):
    """Find complete triads (FP + Wrist + Waist)."""
    triads = []
    
    print("🔍 Scanning directories...")
    
    for participant_dir in sorted(d for d in base.iterdir() if d.is_dir()):
        print(f"  📁 {participant_dir.name}")
        csv_files = list(participant_dir.glob("*.csv"))
        
        fp_files = []
        wrist_files = []
        waist_files = []
        
        for filepath in csv_files:
            fname_lower = filepath.name.lower()
            
            if is_force_plate_csv(filepath):
                fp_files.append(filepath)
            elif is_watch_csv(filepath):
                if 'wrist' in fname_lower:
                    wrist_files.append(filepath)
                elif 'waist' in fname_lower:
                    waist_files.append(filepath)
        
        print(f"     FP: {len(fp_files)}, Wrist: {len(wrist_files)}, Waist: {len(waist_files)}")
        
        # Match by activity
        for fp_path in fp_files:
            activity = infer_activity(fp_path.name)
            if activity == "unknown":
                continue
            
            wrist_match = next((w for w in wrist_files if infer_activity(w.name) == activity), None)
            waist_match = next((w for w in waist_files if infer_activity(w.name) == activity), None)
            
            if wrist_match and waist_match:
                triads.append({
                    "participant": participant_dir.name,
                    "activity": activity,
                    "fp": fp_path,
                    "wrist": wrist_match,
                    "waist": waist_match
                })
                print(f"     ✓ {activity}: {fp_path.name[:40]}...")
    
    return triads

# ==================== PHASE ANALYSIS ====================
def analyze_phase1_waist_to_fp(force_sig, waist_dict, fs, participant, activity):
    """Phase 1: Establish ground truth mapping from Waist to Force Plate."""
    results = {}
    
    # 1. Acceleration magnitude
    lag_mag, corr_mag, pearson_mag = find_optimal_lag_dtw(force_sig, waist_dict['acc_mag'])
    results['acc_mag'] = {'lag': lag_mag, 'corr_xcorr': corr_mag, 'corr_pearson': pearson_mag}
    
    # 2. Vertical acceleration (Z-axis)
    lag_z, corr_z, pearson_z = find_optimal_lag_dtw(force_sig, waist_dict['acc_z'])
    results['acc_z'] = {'lag': lag_z, 'corr_xcorr': corr_z, 'corr_pearson': pearson_z}
    
    # 3. Gyroscope if available
    if waist_dict.get('gyro_mag') is not None:
        lag_gyro, corr_gyro, pearson_gyro = find_optimal_lag_dtw(force_sig, waist_dict['gyro_mag'])
        results['gyro_mag'] = {'lag': lag_gyro, 'corr_xcorr': corr_gyro, 'corr_pearson': pearson_gyro}
    
    # Select best feature
    best_feature = max(results.items(), key=lambda x: abs(x[1]['corr_pearson']))
    
    return results, best_feature

def analyze_phase2_wrist_to_waist(waist_dict, wrist_dict, fs, participant, activity):
    """Phase 2: Learn transformation from Wrist to Waist."""
    results = {}
    
    for feature in ['acc_mag', 'acc_z']:
        lag, corr_xcorr, corr_pearson = find_optimal_lag_dtw(waist_dict[feature], wrist_dict[feature])
        results[feature] = {
            'lag': lag,
            'corr_xcorr': corr_xcorr,
            'corr_pearson': corr_pearson
        }
    
    if waist_dict.get('gyro_mag') is not None and wrist_dict.get('gyro_mag') is not None:
        lag, corr_xcorr, corr_pearson = find_optimal_lag_dtw(waist_dict['gyro_mag'], wrist_dict['gyro_mag'])
        results['gyro_mag'] = {
            'lag': lag,
            'corr_xcorr': corr_xcorr,
            'corr_pearson': corr_pearson
        }
    
    best_feature = max(results.items(), key=lambda x: abs(x[1]['corr_pearson']))
    
    return results, best_feature

def analyze_phase3_wrist_to_fp(force_sig, wrist_dict, fs, participant, activity):
    """Phase 3: Direct end-to-end mapping from Wrist to Force Plate."""
    results = {}
    
    for feature in ['acc_mag', 'acc_z']:
        lag, corr_xcorr, corr_pearson = find_optimal_lag_dtw(force_sig, wrist_dict[feature])
        results[feature] = {
            'lag': lag,
            'corr_xcorr': corr_xcorr,
            'corr_pearson': corr_pearson
        }
    
    if wrist_dict.get('gyro_mag') is not None:
        lag, corr_xcorr, corr_pearson = find_optimal_lag_dtw(force_sig, wrist_dict['gyro_mag'])
        results['gyro_mag'] = {
            'lag': lag,
            'corr_xcorr': corr_xcorr,
            'corr_pearson': corr_pearson
        }
    
    best_feature = max(results.items(), key=lambda x: abs(x[1]['corr_pearson']))
    
    return results, best_feature

# ==================== VISUALIZATION ====================
def create_three_phase_visualization(force_sig, waist_dict, wrist_dict,
                                    phase1_results, phase2_results, phase3_results,
                                    fs, participant, activity, out_dir):
    """Create comprehensive 3-phase visualization."""
    fig = plt.figure(figsize=(18, 12))
    
    t = np.arange(len(force_sig)) / fs
    
    # Normalize signals
    force_z = zscore_safe(force_sig)
    waist_mag_z = zscore_safe(waist_dict['acc_mag'])
    wrist_mag_z = zscore_safe(wrist_dict['acc_mag'])
    
    # Phase 1: WAIST → FORCE PLATE
    ax1 = plt.subplot(3, 2, 1)
    ax1.plot(t, force_z, linewidth=2.5, label="Force Plate", color="#FF6B35", alpha=0.9)
    ax1.plot(t, waist_mag_z, linewidth=1.8, label="Waist IMU", color="#00A878", alpha=0.8)
    
    best_waist = phase1_results[1]
    r_waist = best_waist[1]['corr_pearson']
    lag_waist = best_waist[1]['lag']
    quality_waist = "🟢" if abs(r_waist) > 0.7 else "🟡" if abs(r_waist) > 0.5 else "🔴"
    
    ax1.set_title(f"{quality_waist} Phase 1: Waist → FP | r={r_waist:.3f}, lag={lag_waist/fs*1000:.1f}ms",
                 fontsize=12, fontweight='bold')
    ax1.set_ylabel("Z-scored amplitude", fontsize=10)
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # Zoomed view
    ax2 = plt.subplot(3, 2, 2)
    zoom_samples = min(int(3 * fs), len(force_sig))
    t_zoom = t[:zoom_samples]
    ax2.plot(t_zoom, force_z[:zoom_samples], linewidth=2.5, color="#FF6B35", alpha=0.9)
    ax2.plot(t_zoom, waist_mag_z[:zoom_samples], linewidth=1.8, color="#00A878", alpha=0.8)
    ax2.set_title("Phase 1: Zoomed (3s)", fontsize=11, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Phase 2: WRIST → WAIST
    ax3 = plt.subplot(3, 2, 3)
    ax3.plot(t, waist_mag_z, linewidth=2.5, label="Waist", color="#00A878", alpha=0.9)
    ax3.plot(t, wrist_mag_z, linewidth=1.8, label="Wrist", color="#004E89", alpha=0.8)
    
    best_wrist_waist = phase2_results[1]
    r_wrist_waist = best_wrist_waist[1]['corr_pearson']
    lag_wrist_waist = best_wrist_waist[1]['lag']
    quality_transfer = "🟢" if abs(r_wrist_waist) > 0.7 else "🟡" if abs(r_wrist_waist) > 0.5 else "🔴"
    
    ax3.set_title(f"{quality_transfer} Phase 2: Wrist → Waist | r={r_wrist_waist:.3f}, lag={lag_wrist_waist/fs*1000:.1f}ms",
                 fontsize=12, fontweight='bold')
    ax3.set_ylabel("Z-scored amplitude", fontsize=10)
    ax3.legend(fontsize=9)
    ax3.grid(True, alpha=0.3)
    
    # Zoomed view
    ax4 = plt.subplot(3, 2, 4)
    ax4.plot(t_zoom, waist_mag_z[:zoom_samples], linewidth=2.5, color="#00A878", alpha=0.9)
    ax4.plot(t_zoom, wrist_mag_z[:zoom_samples], linewidth=1.8, color="#004E89", alpha=0.8)
    ax4.set_title("Phase 2: Zoomed (3s)", fontsize=11, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    # Phase 3: WRIST → FORCE PLATE
    ax5 = plt.subplot(3, 2, 5)
    ax5.plot(t, force_z, linewidth=2.5, label="Force Plate", color="#FF6B35", alpha=0.9)
    ax5.plot(t, wrist_mag_z, linewidth=1.8, label="Wrist", color="#004E89", alpha=0.8)
    
    best_wrist_fp = phase3_results[1]
    r_wrist_fp = best_wrist_fp[1]['corr_pearson']
    lag_wrist_fp = best_wrist_fp[1]['lag']
    quality_deploy = "🟢" if abs(r_wrist_fp) > 0.7 else "🟡" if abs(r_wrist_fp) > 0.5 else "🔴"
    
    ax5.set_title(f"{quality_deploy} Phase 3: Wrist → FP (Deployment) | r={r_wrist_fp:.3f}, lag={lag_wrist_fp/fs*1000:.1f}ms",
                 fontsize=12, fontweight='bold')
    ax5.set_xlabel("Time (s)", fontsize=10)
    ax5.set_ylabel("Z-scored amplitude", fontsize=10)
    ax5.legend(fontsize=9)
    ax5.grid(True, alpha=0.3)
    
    # Zoomed view
    ax6 = plt.subplot(3, 2, 6)
    ax6.plot(t_zoom, force_z[:zoom_samples], linewidth=2.5, color="#FF6B35", alpha=0.9)
    ax6.plot(t_zoom, wrist_mag_z[:zoom_samples], linewidth=1.8, color="#004E89", alpha=0.8)
    ax6.set_title("Phase 3: Zoomed (3s)", fontsize=11, fontweight='bold')
    ax6.set_xlabel("Time (s)", fontsize=10)
    ax6.grid(True, alpha=0.3)
    
    plt.suptitle(f"3-Phase Scientific Analysis: {activity.capitalize()} | {participant}\n"
                f"Duration: {t[-1]:.1f}s | Samples: {len(force_sig)} | Rate: {fs:.0f}Hz",
                fontsize=14, fontweight='bold', y=0.98)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    out_name = f"three_phase_{participant}_{activity}.png"
    out_path = out_dir / out_name
    fig.savefig(out_path, dpi=200, facecolor="white", bbox_inches="tight")
    plt.close(fig)
    
    return out_path

# ==================== PROCESSING ====================
def process_triad_three_phases(triad):
    """Process single triad through all 3 phases."""
    try:
        # Read all data
        force_sig, fs_force = read_force_plate_full(triad["fp"])
        waist_data = read_watch_full(triad["waist"])
        wrist_data = read_watch_full(triad["wrist"])
        
        # Align all signals
        force_aligned, waist_aligned, fs_common = align_signals_advanced(force_sig, waist_data, fs_force)
        _, wrist_aligned, _ = align_signals_advanced(force_sig, wrist_data, fs_force)
        
        # Ensure same length
        min_len = min(len(force_aligned), len(waist_aligned['acc_mag']), len(wrist_aligned['acc_mag']))
        if min_len < 100:
            raise ValueError(f"Signals too short: {min_len}")
        
        force_final = force_aligned[:min_len]
        waist_final = {k: v[:min_len] for k, v in waist_aligned.items()}
        wrist_final = {k: v[:min_len] for k, v in wrist_aligned.items()}
        
        # Phase 1: Waist → FP
        phase1_results, phase1_best = analyze_phase1_waist_to_fp(
            force_final, waist_final, fs_common,
            triad['participant'], triad['activity']
        )
        
        # Phase 2: Wrist → Waist
        phase2_results, phase2_best = analyze_phase2_wrist_to_waist(
            waist_final, wrist_final, fs_common,
            triad['participant'], triad['activity']
        )
        
        # Phase 3: Wrist → FP
        phase3_results, phase3_best = analyze_phase3_wrist_to_fp(
            force_final, wrist_final, fs_common,
            triad['participant'], triad['activity']
        )
        
        # Create visualization
        plot_path = create_three_phase_visualization(
            force_final, waist_final, wrist_final,
            (phase1_results, phase1_best),
            (phase2_results, phase2_best),
            (phase3_results, phase3_best),
            fs_common, triad['participant'], triad['activity'],
            OUT_DIR
        )
        
        return {
            "participant": triad["participant"],
            "activity": triad["activity"],
            "fp_file": triad["fp"].name,
            "wrist_file": triad["wrist"].name,
            "waist_file": triad["waist"].name,
            
            # Phase 1
            "phase1_feature": phase1_best[0],
            "phase1_lag_ms": (phase1_best[1]['lag'] / fs_common) * 1000,
            "phase1_corr_xcorr": phase1_best[1]['corr_xcorr'],
            "phase1_corr_pearson": phase1_best[1]['corr_pearson'],
            
            # Phase 2
            "phase2_feature": phase2_best[0],
            "phase2_lag_ms": (phase2_best[1]['lag'] / fs_common) * 1000,
            "phase2_corr_xcorr": phase2_best[1]['corr_xcorr'],
            "phase2_corr_pearson": phase2_best[1]['corr_pearson'],
            
            # Phase 3
            "phase3_feature": phase3_best[0],
            "phase3_lag_ms": (phase3_best[1]['lag'] / fs_common) * 1000,
            "phase3_corr_xcorr": phase3_best[1]['corr_xcorr'],
            "phase3_corr_pearson": phase3_best[1]['corr_pearson'],
            
            "duration_s": len(force_final) / fs_common,
            "samples": len(force_final),
            "sampling_rate": fs_common,
            "plot_path": str(plot_path),
            "status": "success"
        }
        
    except Exception as e:
        return {
            "participant": triad["participant"],
            "activity": triad["activity"],
            "fp_file": triad.get("fp", Path("")).name,
            "wrist_file": triad.get("wrist", Path("")).name,
            "waist_file": triad.get("waist", Path("")).name,
            "status": "failed",
            "error": str(e)
        }

# ==================== BATCH PROCESSING ====================
def run_three_phase_batch(base=BASE_PATH, max_trials=None):
    """Run 3-phase analysis on all triads."""
    print("="*80)
    print("🎯 SCIENTIFIC 3-PHASE SYNCHRONIZATION ANALYSIS")
    print("="*80)
    print("Phase 1: Waist → Force Plate (Reference Ground Truth)")
    print("Phase 2: Wrist → Waist (Transfer Learning)")
    print("Phase 3: Wrist → Force Plate (End-to-End Deployment)")
    print("="*80)
    print(f"Base path: {base}")
    print(f"Output: {OUT_DIR}\n")
    
    triads = discover_triads(base)
    
    if not triads:
        print("\n❌ No triads found!")
        return None
    
    total = len(triads)
    n_process = min(total, max_trials) if max_trials else total
    
    print(f"\n✅ Found {total} complete triads")
    print(f"   Processing {n_process}...\n")
    
    # Activity distribution
    by_activity = {}
    for t in triads:
        by_activity[t['activity']] = by_activity.get(t['activity'], 0) + 1
    
    print("Distribution:")
    for act, count in sorted(by_activity.items()):
        print(f"  {act:10s}: {count:3d}")
    print()
    
    # Process all triads
    results = []
    for i, triad in enumerate(triads[:n_process]):
        print(f"[{i+1:3d}/{n_process}] Processing {triad['participant']} - {triad['activity']}...")
        result = process_triad_three_phases(triad)
        results.append(result)
        
        if result['status'] == 'success':
            p1_q = "✓" if abs(result['phase1_corr_pearson']) > 0.7 else "~"
            p2_q = "✓" if abs(result['phase2_corr_pearson']) > 0.7 else "~"
            p3_q = "✓" if abs(result['phase3_corr_pearson']) > 0.7 else "~"
            
            print(f"  {p1_q} Phase 1 (Waist→FP):   r={result['phase1_corr_pearson']:+.3f}")
            print(f"  {p2_q} Phase 2 (Wrist→Waist): r={result['phase2_corr_pearson']:+.3f}")
            print(f"  {p3_q} Phase 3 (Wrist→FP):   r={result['phase3_corr_pearson']:+.3f}")
        else:
            print(f"  ✗ Failed: {result.get('error', 'Unknown')[:60]}")
        print()
    
    # Save results
    df = pd.DataFrame(results)
    csv_path = OUT_DIR / "three_phase_results.csv"
    df.to_csv(csv_path, index=False)
    
    # Generate statistics
    success = df[df['status'] == 'success']
    
    if len(success) > 0:
        print("\n" + "="*80)
        print("📊 THREE-PHASE ANALYSIS STATISTICS")
        print("="*80)
        print(f"Success rate: {len(success)}/{len(df)} ({100*len(success)/len(df):.1f}%)\n")
        
        # Phase 1 Statistics
        print("🟢 PHASE 1: Waist → Force Plate (Reference Mapping)")
        print(f"   Mean correlation: {success['phase1_corr_pearson'].mean():.3f} ± {success['phase1_corr_pearson'].std():.3f}")
        print(f"   Median lag: {success['phase1_lag_ms'].median():.1f}ms")
        print(f"   Best features: {success['phase1_feature'].value_counts().to_dict()}")
        
        # Phase 2 Statistics
        print("\n🟡 PHASE 2: Wrist → Waist (Transfer Learning)")
        print(f"   Mean correlation: {success['phase2_corr_pearson'].mean():.3f} ± {success['phase2_corr_pearson'].std():.3f}")
        print(f"   Median lag: {success['phase2_lag_ms'].median():.1f}ms")
        print(f"   Best features: {success['phase2_feature'].value_counts().to_dict()}")
        
        # Phase 3 Statistics
        print("\n🔴 PHASE 3: Wrist → Force Plate (Deployment)")
        print(f"   Mean correlation: {success['phase3_corr_pearson'].mean():.3f} ± {success['phase3_corr_pearson'].std():.3f}")
        print(f"   Median lag: {success['phase3_lag_ms'].median():.1f}ms")
        print(f"   Best features: {success['phase3_feature'].value_counts().to_dict()}")
        
        # Comparison analysis
        print("\n📈 PHASE COMPARISON:")
        print(f"   Phase 1 vs Phase 3 correlation gap: {(success['phase1_corr_pearson'].mean() - success['phase3_corr_pearson'].mean()):.3f}")
        print(f"   Phase 2 quality (transfer): {success['phase2_corr_pearson'].mean():.3f}")
        
        # By activity breakdown
        print("\n📊 BY ACTIVITY:")
        for act in sorted(success['activity'].unique()):
            act_data = success[success['activity'] == act]
            print(f"\n  {act.capitalize()}:")
            print(f"    n = {len(act_data)}")
            print(f"    Phase 1: r={act_data['phase1_corr_pearson'].mean():.3f} (Waist→FP)")
            print(f"    Phase 2: r={act_data['phase2_corr_pearson'].mean():.3f} (Wrist→Waist)")
            print(f"    Phase 3: r={act_data['phase3_corr_pearson'].mean():.3f} (Wrist→FP)")
        
        # Quality tiers
        print("\n🎯 DEPLOYMENT READINESS (Phase 3):")
        excellent = success[success['phase3_corr_pearson'].abs() > 0.8]
        good = success[(success['phase3_corr_pearson'].abs() > 0.6) & (success['phase3_corr_pearson'].abs() <= 0.8)]
        moderate = success[(success['phase3_corr_pearson'].abs() > 0.4) & (success['phase3_corr_pearson'].abs() <= 0.6)]
        poor = success[success['phase3_corr_pearson'].abs() <= 0.4]
        
        print(f"   🟢 Excellent (r > 0.8): {len(excellent)} ({100*len(excellent)/len(success):.1f}%)")
        print(f"   🟡 Good (r > 0.6):      {len(good)} ({100*len(good)/len(success):.1f}%)")
        print(f"   🟠 Moderate (r > 0.4):  {len(moderate)} ({100*len(moderate)/len(success):.1f}%)")
        print(f"   🔴 Poor (r ≤ 0.4):      {len(poor)} ({100*len(poor)/len(success):.1f}%)")
        
        # Transfer learning effectiveness
        print("\n🔄 TRANSFER LEARNING EFFECTIVENESS:")
        combined_potential = (success['phase1_corr_pearson'].abs() + success['phase2_corr_pearson'].abs()) / 2
        direct_performance = success['phase3_corr_pearson'].abs()
        transfer_gap = combined_potential - direct_performance
        
        print(f"   Combined potential (P1+P2)/2: {combined_potential.mean():.3f}")
        print(f"   Direct performance (P3):      {direct_performance.mean():.3f}")
        print(f"   Transfer gap:                 {transfer_gap.mean():.3f}")
        
        if transfer_gap.mean() < 0.1:
            print(f"   ✓ Good transfer learning (gap < 0.1)")
        else:
            print(f"   ⚠ Transfer learning needs improvement (gap = {transfer_gap.mean():.3f})")
    
    print(f"\n✅ Results saved: {csv_path}")
    print(f"   Plots saved: {OUT_DIR}/")
    print(f"\n💡 RESEARCH INSIGHTS:")
    print(f"   • Phase 1 shows the theoretical best performance (Waist at CoM)")
    print(f"   • Phase 2 shows how well Wrist can approximate Waist")
    print(f"   • Phase 3 shows real-world deployment feasibility (Wrist-only)")
    print(f"   • Gap between Phase 1 and Phase 3 indicates room for improvement")
    
    return df

# ==================== ADVANCED ANALYSIS ====================
def generate_phase_comparison_plot(df: pd.DataFrame):
    """Generate comparison plot across all three phases."""
    success = df[df['status'] == 'success']
    
    if len(success) == 0:
        print("No successful trials to plot")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Correlation comparison across phases
    ax1 = axes[0, 0]
    phases = ['Phase 1\n(Waist→FP)', 'Phase 2\n(Wrist→Waist)', 'Phase 3\n(Wrist→FP)']
    correlations = [
        success['phase1_corr_pearson'].abs().values,
        success['phase2_corr_pearson'].abs().values,
        success['phase3_corr_pearson'].abs().values
    ]
    
    bp = ax1.boxplot(correlations, labels=phases, patch_artist=True)
    colors = ['#00A878', '#FFB627', '#FF6B35']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax1.set_ylabel('|Pearson Correlation|', fontsize=11)
    ax1.set_title('Correlation Strength Across Phases', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.axhline(0.7, color='green', linestyle='--', alpha=0.5, label='Good threshold')
    ax1.axhline(0.5, color='orange', linestyle='--', alpha=0.5, label='Moderate threshold')
    ax1.legend(fontsize=9)
    
    # 2. Lag comparison
    ax2 = axes[0, 1]
    lags = [
        success['phase1_lag_ms'].abs().values,
        success['phase2_lag_ms'].abs().values,
        success['phase3_lag_ms'].abs().values
    ]
    
    bp2 = ax2.boxplot(lags, labels=phases, patch_artist=True)
    for patch, color in zip(bp2['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax2.set_ylabel('|Lag| (ms)', fontsize=11)
    ax2.set_title('Time Lag Across Phases', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # 3. Phase 1 vs Phase 3 scatter
    ax3 = axes[1, 0]
    activity_colors = {'walking': 'blue', 'running': 'red', 'jogging': 'green', 
                       'heel': 'orange', 'drop': 'purple'}
    
    for activity in success['activity'].unique():
        act_data = success[success['activity'] == activity]
        ax3.scatter(act_data['phase1_corr_pearson'].abs(), 
                   act_data['phase3_corr_pearson'].abs(),
                   c=activity_colors.get(activity, 'gray'),
                   label=activity.capitalize(),
                   alpha=0.6, s=80)
    
    # Add diagonal line (perfect transfer)
    max_val = max(success['phase1_corr_pearson'].abs().max(), 
                  success['phase3_corr_pearson'].abs().max())
    ax3.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Perfect transfer')
    
    ax3.set_xlabel('Phase 1: |r| (Waist→FP)', fontsize=11)
    ax3.set_ylabel('Phase 3: |r| (Wrist→FP)', fontsize=11)
    ax3.set_title('Transfer Learning Effectiveness', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.legend(fontsize=9)
    
    # 4. By activity comparison
    ax4 = axes[1, 1]
    activities = sorted(success['activity'].unique())
    x = np.arange(len(activities))
    width = 0.25
    
    p1_means = [success[success['activity'] == act]['phase1_corr_pearson'].abs().mean() for act in activities]
    p2_means = [success[success['activity'] == act]['phase2_corr_pearson'].abs().mean() for act in activities]
    p3_means = [success[success['activity'] == act]['phase3_corr_pearson'].abs().mean() for act in activities]
    
    ax4.bar(x - width, p1_means, width, label='Phase 1 (Waist→FP)', color='#00A878', alpha=0.8)
    ax4.bar(x, p2_means, width, label='Phase 2 (Wrist→Waist)', color='#FFB627', alpha=0.8)
    ax4.bar(x + width, p3_means, width, label='Phase 3 (Wrist→FP)', color='#FF6B35', alpha=0.8)
    
    ax4.set_ylabel('Mean |Correlation|', fontsize=11)
    ax4.set_title('Performance by Activity', fontsize=12, fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels([a.capitalize() for a in activities], rotation=45)
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    
    summary_path = OUT_DIR / "phase_comparison_summary.png"
    fig.savefig(summary_path, dpi=300, facecolor='white', bbox_inches='tight')
    plt.close()
    
    print(f"\n📊 Phase comparison plot saved: {summary_path}")

# ==================== MAIN ====================
if __name__ == "__main__":
    # Run 3-phase analysis
    df = run_three_phase_batch(BASE_PATH, max_trials=None)
    
    # Generate comparison plots
    if df is not None:
        generate_phase_comparison_plot(df)
        
        print("\n" + "="*80)
        print("✅ COMPLETE 3-PHASE ANALYSIS FINISHED!")
        print("="*80)
        print("\n📁 Output Structure:")
        print(f"   {OUT_DIR}/")
        print(f"   ├── three_phase_*.png         (Individual trial visualizations)")
        print(f"   ├── three_phase_results.csv   (Comprehensive results)")
        print(f"   └── phase_comparison_summary.png (Statistical summary)")
        print("\n🎯 Next Steps:")
        print("   1. Review Phase 1 results → Ground truth quality")
        print("   2. Analyze Phase 2 → Transfer learning potential")
        print("   3. Evaluate Phase 3 → Deployment readiness")
        print("   4. Build predictive model: Wrist features → GRF estimation")

🎯 SCIENTIFIC 3-PHASE SYNCHRONIZATION ANALYSIS
Phase 1: Waist → Force Plate (Reference Ground Truth)
Phase 2: Wrist → Waist (Transfer Learning)
Phase 3: Wrist → Force Plate (End-to-End Deployment)
Base path: /kaggle/input/dataset10/Dataset
Output: scientific_sync_analysis

🔍 Scanning directories...
  📁 P1-1990-186.5-95.8
     FP: 70, Wrist: 36, Waist: 46
     ✓ walking: walk0005_f_6.csv...
     ✓ drop: drop0008_f_8.csv...
     ✓ drop: drop0001_f_8.csv...
     ✓ walking: walk0001_f_8.csv...
     ✓ heel: heel0003_f_8.csv...
     ✓ walking: walk0006_f_7.csv...
     ✓ walking: walk0007_f_8.csv...
     ✓ drop: drop0003_f_8.csv...
     ✓ heel: heel0007_f_8.csv...
     ✓ drop: drop0010_f_8.csv...
     ✓ jogging: jog0003_f_7.csv...
     ✓ drop: drop0007_f_8.csv...
     ✓ jogging: jog0004_f_8.csv...
     ✓ walking: walk0004_f_8.csv...
     ✓ jogging: jog0004_f_7.csv...
     ✓ walking: walk0002_f_6.csv...
     ✓ walking: walk0001_f_7.csv...
     ✓ jogging: jog0001_f_6.csv...
     ✓ walking: walk0

In [ ]:
improved synch analysis

In [2]:
#!/usr/bin/env python3
"""
================================================================================
COMPLETE 3-PHASE SYNCHRONIZATION ANALYSIS WITH PUBLICATION FIGURES
Works with corrected trial matching (proper trial ID matching)
================================================================================
Usage: Copy and paste this entire script into a Jupyter notebook cell
================================================================================
"""

import os
import re
import io
import json
import warnings
from pathlib import Path
from typing import Optional, Tuple, Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import correlate
from scipy.stats import pearsonr, sem, t
from scipy.interpolate import interp1d

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - EDIT THIS SECTION
# ============================================================================

BASE_PATH = Path("/kaggle/input/dataset10/Dataset")
OUTPUT_DIR = Path("improved_sync_analysis")
PUB_FIG_DIR = OUTPUT_DIR / "publication_figures"

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def extract_participant_info(folder_name: str) -> Tuple[Optional[int], Optional[int], Optional[float], Optional[float]]:
    """Extract participant ID, year, height, weight from folder name."""
    pats = [
        r'^P(\d+)-(\d+)-([\d.]+)-([\d.]+)$',
        r'^P(\d+)_(\d+)_([\d.]+)_([\d.]+)$',
    ]
    for p in pats:
        m = re.match(p, folder_name)
        if m:
            return int(m[1]), int(m[2]), float(m[3]), float(m[4])
    return None, None, None, None

def infer_activity(name: str) -> Optional[str]:
    """Infer activity type from filename."""
    n = name.lower()
    if "heel" in n:
        return "heel"
    if "drop" in n:
        return "drop"
    if "run" in n:
        return "running"
    if "jog" in n:
        return "jogging"
    if "walk" in n:
        return "walking"
    return None

def load_force_z_from_csv(fp_path: Path) -> Tuple[np.ndarray, Dict]:
    """Load Force Z channel from force plate CSV."""
    meta = {}
    with open(fp_path, "rb") as f:
        raw = f.read().decode("utf-8-sig", errors="replace")
    
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    
    # Extract metadata
    for ln in lines[:20]:
        if "," in ln:
            k, *v = [x.strip() for x in ln.split(",")]
            if k.upper() in {"NO_OF_SAMPLES", "FREQUENCY", "TIME_STAMP"}:
                meta[k] = v[0] if v else ""
    
    # Find header row with "force z"
    header_idx = None
    for i, ln in enumerate(lines):
        if "force" in ln.lower() and "z" in ln.lower():
            header_idx = i
            break
    
    # If not found, look for numeric header
    if header_idx is None:
        for i, ln in enumerate(lines):
            if all(re.fullmatch(r'\d+', c) for c in ln.split(",")[:6]):
                header_idx = i
                break
    
    # Default to row 26 if still not found
    if header_idx is None:
        header_idx = 26
    
    data_start = header_idx + 1
    header_cols = [c.strip() for c in lines[header_idx].split(",")]
    data_block = "\n".join(lines[data_start:])
    csv_str = ",".join(header_cols) + "\n" + data_block
    
    df = pd.read_csv(io.StringIO(csv_str))
    df.columns = [c.strip() for c in df.columns]
    
    # Find Force Z column
    fz_col = next((c for c in df.columns if "z" in c.lower() and "force" in c.lower()), None)
    if fz_col is None and "5" in df.columns:
        fz_col = "5"
    if fz_col is None:
        raise ValueError("Force Z not found")
    
    fz = pd.to_numeric(df[fz_col], errors="coerce").dropna().to_numpy()
    return fz, meta

def load_watch_matrix(csv_path: Path) -> np.ndarray:
    """Load IMU data from watch CSV."""
    IMU_COLS = [
        "userAccelerationX", "userAccelerationY", "userAccelerationZ",
        "rotationRateX", "rotationRateY", "rotationRateZ"
    ]
    
    df = pd.read_csv(csv_path)
    
    # Check if all columns exist
    if not all(c in df.columns for c in IMU_COLS):
        # Try alternative names
        alt_map = {
            "userAccelerationX": ["AccX", "accel_x"],
            "userAccelerationY": ["AccY", "accel_y"],
            "userAccelerationZ": ["AccZ", "accel_z"],
            "rotationRateX": ["GyroX", "gyro_x"],
            "rotationRateY": ["GyroY", "gyro_y"],
            "rotationRateZ": ["GyroZ", "gyro_z"],
        }
        
        for std_col, alts in alt_map.items():
            if std_col not in df.columns:
                for alt in alts:
                    if alt in df.columns:
                        df[std_col] = df[alt]
                        break
    
    return df[IMU_COLS].to_numpy()

def zscore_safe(x):
    """Z-score normalization with safety."""
    x = np.asarray(x, dtype=float)
    mu = np.nanmean(x)
    sd = np.nanstd(x)
    return (x - mu) / (sd + 1e-10)

def find_optimal_lag_xcorr(ref, sig, max_lag_s=0.25, fs=1000.0) -> Dict:
    """Find optimal lag using cross-correlation."""
    n = min(len(ref), len(sig))
    
    if n < 50:
        return {"lag_samples": 0, "correlation": 0.0, "pearson_r": 0.0}
    
    ref_z = zscore_safe(ref[:n])
    sig_z = zscore_safe(sig[:n])
    
    max_lag = int(round(max_lag_s * fs))
    corr = correlate(sig_z, ref_z, mode='full')
    lags = np.arange(-n + 1, n)
    mask = (lags >= -max_lag) & (lags <= max_lag)
    corr_valid = corr[mask]
    lags_valid = lags[mask]
    
    k = int(np.argmax(np.abs(corr_valid)))
    best_lag = int(lags_valid[k])
    
    # Compute Pearson r at the best lag
    if best_lag == 0:
        r, _ = pearsonr(ref_z, sig_z)
    elif best_lag > 0:
        overlap = min(len(ref_z) - best_lag, len(sig_z))
        if overlap > 10:
            r, _ = pearsonr(ref_z[:-best_lag], sig_z[best_lag:best_lag + overlap])
        else:
            r = 0.0
    else:  # best_lag < 0
        overlap = min(len(ref_z), len(sig_z) + best_lag)
        if overlap > 10:
            r, _ = pearsonr(ref_z[-best_lag:], sig_z[:overlap])
        else:
            r = 0.0
    
    return {
        "lag_samples": best_lag,
        "correlation": float(np.max(np.abs(corr_valid))),
        "pearson_r": float(r)
    }

# ============================================================================
# CORRECTED RESULTS ANALYZER - GENERATES PUBLICATION FIGURES
# ============================================================================

class CorrectedResultsAnalyzer:
    """Analyzer for corrected results - creates publication figures."""
    
    def __init__(self, df: pd.DataFrame):
        """Initialize with results dataframe."""
        self.df = df
        self.success_df = df[df['status'] == 'success'].copy() if 'status' in df.columns else df.copy()
        
        # Add quality metrics
        self.success_df['phase1_quality'] = self.success_df['phase1_corr'].abs()
        self.success_df['phase3_quality'] = self.success_df['phase3_corr'].abs()
        
        # Ensure output directory exists
        PUB_FIG_DIR.mkdir(parents=True, exist_ok=True)
        
        print(f"\n📊 Analyzer initialized with {len(self.success_df)} successful trials")
    
    def _get_quality_tier(self, corr):
        """Assign quality tier based on correlation."""
        abs_corr = abs(corr)
        if abs_corr > 0.7:
            return 'Excellent'
        elif abs_corr > 0.5:
            return 'Good'
        elif abs_corr > 0.3:
            return 'Moderate'
        else:
            return 'Poor'
    
    def create_manifest(self):
        """Create and save trial manifest."""
        manifest_data = []
        
        for _, row in self.success_df.iterrows():
            quality_tier = self._get_quality_tier(row['phase3_corr'])
            
            manifest_data.append({
                'trial_id': f"{row.get('participant', 'N/A')}_{row.get('activity', 'N/A')}",
                'participant': row.get('participant', 'N/A'),
                'activity': row.get('activity', 'N/A'),
                'phase1_correlation': row.get('phase1_corr', np.nan),
                'phase2_correlation': row.get('phase2_corr', np.nan),
                'phase3_correlation': row.get('phase3_corr', np.nan),
                'quality_tier': quality_tier,
            })
        
        manifest_df = pd.DataFrame(manifest_data)
        manifest_path = PUB_FIG_DIR / "trial_manifest.csv"
        manifest_df.to_csv(manifest_path, index=False)
        
        print(f"\n📄 Manifest saved: {manifest_path}")
        print(f"   Total trials: {len(manifest_df)}")
        print(f"   Participants: {manifest_df['participant'].nunique()}")
        
        return manifest_df
    
    def create_overview_figure(self):
        """Create 4-panel overview figure."""
        fig = plt.figure(figsize=(16, 12))
        
        activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
        activity_labels = ['Walking', 'Jogging', 'Running', 'Heel Drop', 'Step Drop']
        activity_colors = ['#6BAED6', '#FC8D62', '#FDB462', '#B3B3E6', '#8DD3C7']
        
        # (A) Quality Distribution by Activity - Box Plot
        ax1 = plt.subplot(2, 2, 1)
        box_data = []
        valid_activities = []
        valid_colors = []
        
        for i, activity in enumerate(activity_order):
            activity_data = self.success_df[self.success_df['activity'] == activity]['phase3_quality']
            if len(activity_data) > 0:
                box_data.append(activity_data.values)
                valid_activities.append(activity_labels[i])
                valid_colors.append(activity_colors[i])
        
        if box_data:
            bp = ax1.boxplot(box_data, labels=valid_activities, patch_artist=True,
                            widths=0.6, showfliers=True,
                            medianprops=dict(color='black', linewidth=2),
                            boxprops=dict(linewidth=1.5))
            
            for patch, color in zip(bp['boxes'], valid_colors):
                patch.set_facecolor(color)
                patch.set_alpha(0.7)
        
        ax1.set_ylabel('Correlation |r|', fontsize=12, fontweight='bold')
        ax1.set_title('(A) Wrist-FP Correlation by Activity', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='y')
        ax1.tick_params(axis='x', rotation=45)
        ax1.axhline(0.7, color='green', linestyle='--', alpha=0.5, label='Excellent (>0.7)')
        ax1.axhline(0.5, color='orange', linestyle='--', alpha=0.5, label='Good (>0.5)')
        ax1.legend(fontsize=9)
        
        # (B) Quality Distribution Pie
        ax2 = plt.subplot(2, 2, 2)
        
        quality_tiers = []
        for _, row in self.success_df.iterrows():
            abs_corr = abs(row['phase3_corr'])
            if abs_corr > 0.7:
                quality_tiers.append('Excellent')
            elif abs_corr > 0.5:
                quality_tiers.append('Good')
            elif abs_corr > 0.3:
                quality_tiers.append('Moderate')
            else:
                quality_tiers.append('Poor')
        
        quality_counts = pd.Series(quality_tiers).value_counts()
        colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']
        
        wedges, texts, autotexts = ax2.pie(quality_counts.values, 
                                           labels=quality_counts.index,
                                           colors=colors_pie[:len(quality_counts)],
                                           autopct='%1.1f%%',
                                           startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax2.set_title('(B) Quality Distribution (Phase 3)', fontsize=13, fontweight='bold')
        
        # (C) Phase 1 vs Phase 3 Scatter
        ax3 = plt.subplot(2, 2, 3)
        
        for activity in activity_order:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                color = activity_colors[activity_order.index(activity)]
                ax3.scatter(act_data['phase1_quality'], act_data['phase3_quality'],
                           c=color, label=activity.capitalize(), alpha=0.6, s=50)
        
        max_val = max(self.success_df['phase1_quality'].max(), self.success_df['phase3_quality'].max())
        ax3.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Perfect Transfer')
        
        ax3.set_xlabel('Phase 1: Waist→FP |r|', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Phase 3: Wrist→FP |r|', fontsize=12, fontweight='bold')
        ax3.set_title('(C) Transfer Learning Effectiveness', fontsize=13, fontweight='bold')
        ax3.grid(True, alpha=0.3)
        ax3.legend(fontsize=9, loc='lower right')
        ax3.set_xlim([0, 1])
        ax3.set_ylim([0, 1])
        
        # (D) Trial Count by Activity
        ax4 = plt.subplot(2, 2, 4)
        
        trial_counts = []
        trial_labels = []
        trial_colors = []
        
        for i, activity in enumerate(activity_order):
            count = len(self.success_df[self.success_df['activity'] == activity])
            if count > 0:
                trial_counts.append(count)
                trial_labels.append(activity_labels[i])
                trial_colors.append(activity_colors[i])
        
        bars = ax4.bar(trial_labels, trial_counts, color=trial_colors, 
                      alpha=0.8, edgecolor='black', linewidth=1.5)
        
        for bar, count in zip(bars, trial_counts):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(count)}', ha='center', va='bottom', fontweight='bold')
        
        ax4.set_ylabel('Number of Trials', fontsize=12, fontweight='bold')
        ax4.set_title('(D) Successfully Analyzed Trials', fontsize=13, fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        ax4.tick_params(axis='x', rotation=45)
        
        fig.suptitle('3-Phase Synchronization Analysis - Dataset Overview', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        save_path = PUB_FIG_DIR / 'dataset_overview.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Overview figure saved: {save_path}")
    
    def create_statistical_summary(self):
        """Create statistical summary figure."""
        fig = plt.figure(figsize=(16, 12))
        
        activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
        activity_labels = ['Walking', 'Jogging', 'Running', 'Heel Drop', 'Step Drop']
        
        # (A) Mean with 95% CI
        ax1 = plt.subplot(2, 2, 1)
        
        means_p1, cis_p1 = [], []
        means_p3, cis_p3 = [], []
        valid_labels = []
        
        for i, activity in enumerate(activity_order):
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 1:
                mean_p1 = act_data['phase1_corr'].mean()
                sem_p1 = sem(act_data['phase1_corr'].dropna())
                ci_p1 = sem_p1 * t.ppf(0.975, len(act_data) - 1)
                
                mean_p3 = act_data['phase3_corr'].mean()
                sem_p3 = sem(act_data['phase3_corr'].dropna())
                ci_p3 = sem_p3 * t.ppf(0.975, len(act_data) - 1)
                
                means_p1.append(mean_p1)
                cis_p1.append(ci_p1)
                means_p3.append(mean_p3)
                cis_p3.append(ci_p3)
                valid_labels.append(activity_labels[i])
        
        if valid_labels:
            x = np.arange(len(valid_labels))
            width = 0.35
            
            ax1.bar(x - width/2, means_p1, width, yerr=cis_p1,
                   label='Phase 1 (Waist)', color='#3498DB', alpha=0.8, capsize=5)
            ax1.bar(x + width/2, means_p3, width, yerr=cis_p3,
                   label='Phase 3 (Wrist)', color='#9B59B6', alpha=0.8, capsize=5)
            
            ax1.set_ylabel('Mean Correlation r', fontsize=12, fontweight='bold')
            ax1.set_title('(A) Mean Correlation with 95% CI', fontsize=13, fontweight='bold')
            ax1.set_xticks(x)
            ax1.set_xticklabels(valid_labels, rotation=45, ha='right')
            ax1.legend()
            ax1.grid(True, alpha=0.3, axis='y')
        
        # (B) Summary Statistics Table
        ax2 = plt.subplot(2, 2, 2)
        ax2.axis('off')
        
        summary_data = []
        for activity in activity_order:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                summary_data.append([
                    activity_labels[activity_order.index(activity)],
                    len(act_data),
                    f"{act_data['phase1_corr'].mean():.3f}",
                    f"{act_data['phase1_corr'].std():.3f}",
                    f"{act_data['phase3_corr'].mean():.3f}",
                    f"{act_data['phase3_corr'].std():.3f}"
                ])
        
        if summary_data:
            table = ax2.table(cellText=summary_data,
                             colLabels=['Activity', 'N', 'P1 Mean', 'P1 SD', 'P3 Mean', 'P3 SD'],
                             cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
            
            table.auto_set_font_size(False)
            table.set_fontsize(10)
            table.scale(1, 2.5)
            
            for i in range(6):
                table[(0, i)].set_facecolor('#4472C4')
                table[(0, i)].set_text_props(weight='bold', color='white')
        
        ax2.set_title('(B) Summary Statistics', fontsize=13, fontweight='bold', pad=20)
        
        # (C) Distribution Histogram
        ax3 = plt.subplot(2, 2, 3)
        
        phase3_quality = self.success_df['phase3_corr'].abs()
        ax3.hist(phase3_quality, bins=30, alpha=0.7,
                color='#9ECAE1', edgecolor='black', linewidth=1.5)
        
        mean_corr = phase3_quality.mean()
        ax3.axvline(mean_corr, color='red', linestyle='--', linewidth=2.5,
                   label=f'Mean: {mean_corr:.3f}')
        
        ax3.set_xlabel('Correlation |r|', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax3.set_title('(C) Overall Quality Distribution', fontsize=13, fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3, axis='y')
        
        # (D) Deployment Readiness Pie
        ax4 = plt.subplot(2, 2, 4)
        
        phase3_quality = self.success_df['phase3_corr'].abs()
        excellent = len(self.success_df[phase3_quality > 0.7])
        good = len(self.success_df[(phase3_quality > 0.5) & (phase3_quality <= 0.7)])
        moderate = len(self.success_df[(phase3_quality > 0.3) & (phase3_quality <= 0.5)])
        poor = len(self.success_df[phase3_quality <= 0.3])
        
        sizes = [excellent, good, moderate, poor]
        labels = ['Excellent\n(>0.7)', 'Good\n(>0.5)', 'Moderate\n(>0.3)', 'Poor\n(≤0.3)']
        colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']
        
        wedges, texts, autotexts = ax4.pie(sizes, labels=labels, colors=colors_pie,
                                           autopct=lambda p: f'{p:.1f}%\n({int(p*sum(sizes)/100)})',
                                           startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax4.set_title('(D) Deployment Readiness', fontsize=13, fontweight='bold')
        
        fig.suptitle('Statistical Summary - 3-Phase Analysis', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        save_path = PUB_FIG_DIR / 'statistical_summary.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Statistical summary saved: {save_path}")
    
    def print_summary_statistics(self):
        """Print summary statistics to console."""
        if len(self.success_df) == 0:
            print("No successful trials to analyze")
            return
        
        print("\n" + "="*80)
        print("📊 SUMMARY STATISTICS")
        print("="*80)
        
        print(f"\nPhase 1 (Waist→FP):")
        print(f"  Mean r: {self.success_df['phase1_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase1_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase1_corr'].min():.3f}, {self.success_df['phase1_corr'].max():.3f}]")
        
        print(f"\nPhase 2 (Waist→Wrist):")
        print(f"  Mean r: {self.success_df['phase2_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase2_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase2_corr'].min():.3f}, {self.success_df['phase2_corr'].max():.3f}]")
        
        print(f"\nPhase 3 (Wrist→FP):")
        print(f"  Mean r: {self.success_df['phase3_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase3_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase3_corr'].min():.3f}, {self.success_df['phase3_corr'].max():.3f}]")
        
        # By activity
        print(f"\nBy Activity:")
        for activity in ['walking', 'jogging', 'running', 'heel', 'drop']:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                print(f"\n  {activity.capitalize()}:")
                print(f"    Count: {len(act_data)}")
                print(f"    Phase 1 Mean r: {act_data['phase1_corr'].mean():.3f}")
                print(f"    Phase 3 Mean r: {act_data['phase3_corr'].mean():.3f}")
                print(f"    Phase 3 Std:   {act_data['phase3_corr'].std():.3f}")
    
    def create_all_figures(self):
        """Generate all publication figures."""
        print("\n🎨 Creating publication-quality figures...")
        print("="*80)
        
        self.create_manifest()
        self.create_overview_figure()
        self.create_statistical_summary()
        self.print_summary_statistics()
        
        print("\n" + "="*80)
        print("✅ ALL FIGURES GENERATED")
        print("="*80)

# ============================================================================
# MAIN ANALYSIS PIPELINE
# ============================================================================

def run_three_phase_analysis():
    """Run complete 3-phase analysis with corrected trial matching."""
    
    print("\n" + "="*80)
    print("🎯 IMPROVED 3-PHASE SYNCHRONIZATION ANALYSIS")
    print("="*80)
    print(f"Base path: {BASE_PATH}")
    print(f"Output: {OUTPUT_DIR}")
    
    # Check if saved results exist
    results_file = OUTPUT_DIR / "three_phase_results.csv"
    
    if results_file.exists():
        print(f"\n✅ Loading saved results from: {results_file}")
        df = pd.read_csv(results_file)
        print(f"   Loaded {len(df)} trials")
    else:
        print("\n🔍 Running corrected 3-phase analysis...")
        print("   This may take several minutes...")
        
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        results = []
        trial_count = 0
        
        # Scan all participants
        for folder_name in sorted(os.listdir(BASE_PATH)):
            pdir = BASE_PATH / folder_name
            
            if not pdir.is_dir():
                continue
            
            # Extract participant info
            pid, year, height_cm, weight_kg = extract_participant_info(folder_name)
            
            if pid is None or weight_kg is None:
                continue
            
            print(f"\n👤 {folder_name} (Weight: {weight_kg}kg)")
            
            # List all files
            files = [f for f in os.listdir(pdir) if f.endswith(".csv")]
            
            # Find force plate files
            fp_files = [f for f in files if re.match(r'^(walk|jog|run|heel|drop)\d+_f_\d+\.csv$', f, re.IGNORECASE)]
            
            # Find watch files
            watch_files = [f for f in files if "Trial" in f and ("Waist" in f or "Wrist" in f)]
            
            # Build mapping: trial ID -> watch files
            id_to_watch = {}
            for wf in watch_files:
                m = re.search(r'ID_(\d+)', wf)
                if m:
                    trial_id = int(m.group(1))
                    id_to_watch.setdefault(trial_id, []).append(wf)
            
            # Process each force plate file
            for fp in sorted(fp_files):
                activity = infer_activity(fp)
                
                if not activity:
                    continue
                
                # Extract trial number from force plate filename
                m = re.match(r'^(?:walk|jog|run|heel|drop)(\d+)_f_\d+\.csv$', fp, re.IGNORECASE)
                if not m:
                    continue
                
                trial_num = int(m.group(1))
                candidates = id_to_watch.get(trial_num, [])
                
                # Find matching wrist and waist files
                wrist_f = next((x for x in candidates if "Wrist" in x and activity.capitalize() in x), None)
                waist_f = next((x for x in candidates if "Waist" in x and activity.capitalize() in x), None)
                
                if not (wrist_f and waist_f):
                    continue
                
                try:
                    # Load data
                    fz, _ = load_force_z_from_csv(pdir / fp)
                    waist_X = load_watch_matrix(pdir / waist_f)
                    wrist_X = load_watch_matrix(pdir / wrist_f)
                    
                    # Ensure same length
                    n = min(len(fz), len(waist_X), len(wrist_X))
                    if n < 100:
                        continue
                    
                    fz_short = fz[:n]
                    waist_mag = np.sqrt((waist_X[:n, :3]**2).sum(axis=1))
                    wrist_mag = np.sqrt((wrist_X[:n, :3]**2).sum(axis=1))
                    
                    # Phase 1: Waist → FP
                    p1 = find_optimal_lag_xcorr(fz_short, waist_mag, fs=1000.0)
                    
                    # Phase 2: Waist → Wrist
                    p2 = find_optimal_lag_xcorr(waist_mag, wrist_mag, fs=100.0)
                    
                    # Phase 3: Wrist → FP
                    p3 = find_optimal_lag_xcorr(fz_short, wrist_mag, fs=1000.0)
                    
                    result = {
                        'participant': folder_name,
                        'activity': activity,
                        'trial_id': trial_num,
                        'phase1_corr': p1['pearson_r'],
                        'phase2_corr': p2['pearson_r'],
                        'phase3_corr': p3['pearson_r'],
                        'status': 'success'
                    }
                    
                    results.append(result)
                    trial_count += 1
                    
                    print(f"  ✅ {activity:8s} (trial {trial_num:2d}): "
                          f"P1={p1['pearson_r']:.3f}, P2={p2['pearson_r']:.3f}, P3={p3['pearson_r']:.3f}")
                    
                    if trial_count % 50 == 0:
                        print(f"     [{trial_count} trials processed]")
                
                except Exception as e:
                    print(f"  ❌ {activity} trial {trial_num}: {str(e)[:50]}")
                    continue
        
        # Save results
        df = pd.DataFrame(results)
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        df.to_csv(results_file, index=False)
        
        print(f"\n✅ Analysis complete: {trial_count} trials")
        print(f"   Saved to: {results_file}")
    
    return df

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution."""
    
    # Run analysis
    df = run_three_phase_analysis()
    
    if df is not None and len(df) > 0:
        # Create analyzer and generate all figures
        print("\n" + "="*80)
        print("🎨 GENERATING PUBLICATION FIGURES")
        print("="*80)
        
        analyzer = CorrectedResultsAnalyzer(df)
        analyzer.create_all_figures()
        
        # Final summary
        print("\n" + "="*80)
        print("✅ COMPLETE 3-PHASE ANALYSIS FINISHED!")
        print("="*80)
        print(f"\n📁 Output Directory: {OUTPUT_DIR}")
        print(f"📁 Publication Figures: {PUB_FIG_DIR}")
        print(f"\n📊 Generated Files:")
        print(f"   • three_phase_results.csv")
        print(f"   • publication_figures/")
        print(f"     ├── dataset_overview.png")
        print(f"     ├── statistical_summary.png")
        print(f"     └── trial_manifest.csv")
        
    else:
        print("\n❌ Analysis failed - no valid trials found!")

if __name__ == "__main__":
    main()


🎯 IMPROVED 3-PHASE SYNCHRONIZATION ANALYSIS
Base path: /kaggle/input/dataset10/Dataset
Output: improved_sync_analysis

🔍 Running corrected 3-phase analysis...
   This may take several minutes...

👤 P1-1990-186.5-95.8 (Weight: 95.8kg)
  ✅ drop     (trial  1): P1=-0.517, P2=-0.264, P3=-0.723
  ✅ drop     (trial  2): P1=0.573, P2=0.305, P3=0.634
  ✅ drop     (trial  3): P1=0.498, P2=0.320, P3=-0.623
  ✅ drop     (trial  4): P1=0.505, P2=-0.177, P3=-0.528
  ✅ drop     (trial  5): P1=0.497, P2=-0.246, P3=-0.588
  ✅ drop     (trial  7): P1=0.562, P2=-0.218, P3=-0.635
  ✅ drop     (trial  8): P1=-0.481, P2=-0.265, P3=-0.639
  ✅ drop     (trial  9): P1=0.401, P2=-0.272, P3=-0.675
  ✅ drop     (trial 10): P1=0.537, P2=-0.245, P3=-0.615
  ✅ drop     (trial 11): P1=0.596, P2=-0.253, P3=-0.628
  ✅ heel     (trial  1): P1=0.430, P2=-0.137, P3=-0.581
  ✅ heel     (trial  2): P1=0.526, P2=-0.085, P3=-0.424
  ✅ heel     (trial  3): P1=0.687, P2=0.780, P3=0.716
  ✅ heel     (trial  4): P1=0.520, P2=0.

In [5]:
#!/usr/bin/env python3
"""
================================================================================
COMPLETE 3-PHASE SYNCHRONIZATION ANALYSIS
Handles flat file structure: [Activity]_[Placement]_Trial_[UUID]_ID_[NUMBER]_[DATE]_[TIME].csv
================================================================================
Usage: Copy and paste this entire script into a Jupyter notebook cell
================================================================================
"""

import os
import re
import io
import json
import warnings
from pathlib import Path
from typing import Optional, Tuple, Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import correlate
from scipy.stats import pearsonr, sem, t

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - EDIT THIS SECTION
# ============================================================================

BASE_PATH = Path("/kaggle/input/dataset10/Dataset")
OUTPUT_DIR = Path("improved_sync_analysis")
PUB_FIG_DIR = OUTPUT_DIR / "publication_figures"

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def extract_participant_info(folder_name: str) -> Tuple[Optional[int], Optional[int], Optional[float], Optional[float]]:
    """Extract participant ID, year, height, weight from folder name."""
    pats = [
        r'^P(\d+)-(\d+)-([\d.]+)-([\d.]+)$',
        r'^P(\d+)_(\d+)_([\d.]+)_([\d.]+)$',
    ]
    for p in pats:
        m = re.match(p, folder_name)
        if m:
            return int(m[1]), int(m[2]), float(m[3]), float(m[4])
    return None, None, None, None

def extract_trial_info(filename: str) -> Tuple[Optional[str], Optional[str], Optional[int]]:
    """
    Extract activity, placement (Waist/Wrist), and trial ID from filename.
    Format: [Activity]_[Placement]_Trial_[UUID]_ID_[NUMBER]_[DATE]_[TIME].csv
    """
    # Extract activity (before first underscore or before _Waist/_Wrist)
    m = re.search(r'^(\w+)_(Waist|Wrist)_Trial.*_ID_(\d+)_', filename)
    if m:
        activity = m.group(1).lower()
        placement = m.group(2)
        trial_id = int(m.group(3))
        return activity, placement, trial_id
    return None, None, None

def load_force_z_from_csv(fp_path: Path) -> Tuple[np.ndarray, Dict]:
    """Load Force Z channel from force plate CSV."""
    meta = {}
    with open(fp_path, "rb") as f:
        raw = f.read().decode("utf-8-sig", errors="replace")
    
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    
    # Extract metadata
    for ln in lines[:20]:
        if "," in ln:
            k, *v = [x.strip() for x in ln.split(",")]
            if k.upper() in {"NO_OF_SAMPLES", "FREQUENCY", "TIME_STAMP"}:
                meta[k] = v[0] if v else ""
    
    # Find header row with "force z"
    header_idx = None
    for i, ln in enumerate(lines):
        if "force" in ln.lower() and "z" in ln.lower():
            header_idx = i
            break
    
    # If not found, look for numeric header
    if header_idx is None:
        for i, ln in enumerate(lines):
            if all(re.fullmatch(r'\d+', c) for c in ln.split(",")[:6]):
                header_idx = i
                break
    
    # Default to row 26 if still not found
    if header_idx is None:
        header_idx = 26
    
    data_start = header_idx + 1
    header_cols = [c.strip() for c in lines[header_idx].split(",")]
    data_block = "\n".join(lines[data_start:])
    csv_str = ",".join(header_cols) + "\n" + data_block
    
    df = pd.read_csv(io.StringIO(csv_str))
    df.columns = [c.strip() for c in df.columns]
    
    # Find Force Z column
    fz_col = next((c for c in df.columns if "z" in c.lower() and "force" in c.lower()), None)
    if fz_col is None and "5" in df.columns:
        fz_col = "5"
    if fz_col is None:
        raise ValueError("Force Z not found")
    
    fz = pd.to_numeric(df[fz_col], errors="coerce").dropna().to_numpy()
    return fz, meta

def load_watch_matrix(csv_path: Path) -> np.ndarray:
    """Load IMU data from watch CSV."""
    IMU_COLS = [
        "userAccelerationX", "userAccelerationY", "userAccelerationZ",
        "rotationRateX", "rotationRateY", "rotationRateZ"
    ]
    
    df = pd.read_csv(csv_path)
    
    # Check if all columns exist
    if not all(c in df.columns for c in IMU_COLS):
        # Try alternative names
        alt_map = {
            "userAccelerationX": ["AccX", "accel_x"],
            "userAccelerationY": ["AccY", "accel_y"],
            "userAccelerationZ": ["AccZ", "accel_z"],
            "rotationRateX": ["GyroX", "gyro_x"],
            "rotationRateY": ["GyroY", "gyro_y"],
            "rotationRateZ": ["GyroZ", "gyro_z"],
        }
        
        for std_col, alts in alt_map.items():
            if std_col not in df.columns:
                for alt in alts:
                    if alt in df.columns:
                        df[std_col] = df[alt]
                        break
    
    return df[IMU_COLS].to_numpy()

def zscore_safe(x):
    """Z-score normalization with safety."""
    x = np.asarray(x, dtype=float)
    mu = np.nanmean(x)
    sd = np.nanstd(x)
    return (x - mu) / (sd + 1e-10)

def find_optimal_lag_xcorr(ref, sig, max_lag_s=0.25, fs=1000.0) -> Dict:
    """Find optimal lag using cross-correlation."""
    n = min(len(ref), len(sig))
    
    if n < 50:
        return {"lag_samples": 0, "correlation": 0.0, "pearson_r": 0.0}
    
    ref_z = zscore_safe(ref[:n])
    sig_z = zscore_safe(sig[:n])
    
    max_lag = int(round(max_lag_s * fs))
    corr = correlate(sig_z, ref_z, mode='full')
    lags = np.arange(-n + 1, n)
    mask = (lags >= -max_lag) & (lags <= max_lag)
    corr_valid = corr[mask]
    lags_valid = lags[mask]
    
    k = int(np.argmax(np.abs(corr_valid)))
    best_lag = int(lags_valid[k])
    
    # Compute Pearson r at the best lag
    if best_lag == 0:
        r, _ = pearsonr(ref_z, sig_z)
    elif best_lag > 0:
        overlap = min(len(ref_z) - best_lag, len(sig_z))
        if overlap > 10:
            r, _ = pearsonr(ref_z[:-best_lag], sig_z[best_lag:best_lag + overlap])
        else:
            r = 0.0
    else:  # best_lag < 0
        overlap = min(len(ref_z), len(sig_z) + best_lag)
        if overlap > 10:
            r, _ = pearsonr(ref_z[-best_lag:], sig_z[:overlap])
        else:
            r = 0.0
    
    return {
        "lag_samples": best_lag,
        "correlation": float(np.max(np.abs(corr_valid))),
        "pearson_r": float(r)
    }

# ============================================================================
# CORRECTED RESULTS ANALYZER - GENERATES PUBLICATION FIGURES
# ============================================================================

class CorrectedResultsAnalyzer:
    """Analyzer for corrected results - creates publication figures."""
    
    def __init__(self, df: pd.DataFrame):
        """Initialize with results dataframe."""
        self.df = df
        self.success_df = df[df['status'] == 'success'].copy() if 'status' in df.columns else df.copy()
        
        # Add quality metrics
        self.success_df['phase1_quality'] = self.success_df['phase1_corr'].abs()
        self.success_df['phase3_quality'] = self.success_df['phase3_corr'].abs()
        
        # Ensure output directory exists
        PUB_FIG_DIR.mkdir(parents=True, exist_ok=True)
        
        print(f"\n📊 Analyzer initialized with {len(self.success_df)} successful trials")
    
    def _get_quality_tier(self, corr):
        """Assign quality tier based on correlation."""
        abs_corr = abs(corr)
        if abs_corr > 0.7:
            return 'Excellent'
        elif abs_corr > 0.5:
            return 'Good'
        elif abs_corr > 0.3:
            return 'Moderate'
        else:
            return 'Poor'
    
    def create_manifest(self):
        """Create and save trial manifest."""
        manifest_data = []
        
        for _, row in self.success_df.iterrows():
            quality_tier = self._get_quality_tier(row['phase3_corr'])
            
            manifest_data.append({
                'trial_id': f"{row.get('participant', 'N/A')}_{row.get('activity', 'N/A')}_ID{row.get('trial_num', 'N/A')}",
                'participant': row.get('participant', 'N/A'),
                'activity': row.get('activity', 'N/A'),
                'trial_num': row.get('trial_num', 'N/A'),
                'phase1_correlation': row.get('phase1_corr', np.nan),
                'phase2_correlation': row.get('phase2_corr', np.nan),
                'phase3_correlation': row.get('phase3_corr', np.nan),
                'quality_tier': quality_tier,
            })
        
        manifest_df = pd.DataFrame(manifest_data)
        manifest_path = PUB_FIG_DIR / "trial_manifest.csv"
        manifest_df.to_csv(manifest_path, index=False)
        
        print(f"\n📄 Manifest saved: {manifest_path}")
        print(f"   Total trials: {len(manifest_df)}")
        print(f"   Participants: {manifest_df['participant'].nunique()}")
        
        return manifest_df
    
    def create_overview_figure(self):
        """Create 4-panel overview figure."""
        if len(self.success_df) == 0:
            print("⚠️  No successful trials to visualize")
            return
        
        fig = plt.figure(figsize=(16, 12))
        
        activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
        activity_labels = ['Walking', 'Jogging', 'Running', 'Heel Drop', 'Step Drop']
        activity_colors = ['#6BAED6', '#FC8D62', '#FDB462', '#B3B3E6', '#8DD3C7']
        
        # (A) Quality Distribution by Activity - Box Plot
        ax1 = plt.subplot(2, 2, 1)
        box_data = []
        valid_activities = []
        valid_colors = []
        
        for i, activity in enumerate(activity_order):
            activity_data = self.success_df[self.success_df['activity'] == activity]['phase3_quality']
            if len(activity_data) > 0:
                box_data.append(activity_data.values)
                valid_activities.append(activity_labels[i])
                valid_colors.append(activity_colors[i])
        
        if box_data:
            bp = ax1.boxplot(box_data, labels=valid_activities, patch_artist=True,
                            widths=0.6, showfliers=True,
                            medianprops=dict(color='black', linewidth=2),
                            boxprops=dict(linewidth=1.5))
            
            for patch, color in zip(bp['boxes'], valid_colors):
                patch.set_facecolor(color)
                patch.set_alpha(0.7)
        
        ax1.set_ylabel('Correlation |r|', fontsize=12, fontweight='bold')
        ax1.set_title('(A) Wrist-FP Correlation by Activity', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='y')
        ax1.tick_params(axis='x', rotation=45)
        ax1.axhline(0.7, color='green', linestyle='--', alpha=0.5, label='Excellent (>0.7)')
        ax1.axhline(0.5, color='orange', linestyle='--', alpha=0.5, label='Good (>0.5)')
        ax1.legend(fontsize=9)
        
        # (B) Quality Distribution Pie
        ax2 = plt.subplot(2, 2, 2)
        
        quality_tiers = []
        for _, row in self.success_df.iterrows():
            abs_corr = abs(row['phase3_corr'])
            if abs_corr > 0.7:
                quality_tiers.append('Excellent')
            elif abs_corr > 0.5:
                quality_tiers.append('Good')
            elif abs_corr > 0.3:
                quality_tiers.append('Moderate')
            else:
                quality_tiers.append('Poor')
        
        quality_counts = pd.Series(quality_tiers).value_counts()
        colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']
        
        wedges, texts, autotexts = ax2.pie(quality_counts.values, 
                                           labels=quality_counts.index,
                                           colors=colors_pie[:len(quality_counts)],
                                           autopct='%1.1f%%',
                                           startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax2.set_title('(B) Quality Distribution (Phase 3)', fontsize=13, fontweight='bold')
        
        # (C) Phase 1 vs Phase 3 Scatter
        ax3 = plt.subplot(2, 2, 3)
        
        for activity in activity_order:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                color = activity_colors[activity_order.index(activity)]
                ax3.scatter(act_data['phase1_quality'], act_data['phase3_quality'],
                           c=color, label=activity.capitalize(), alpha=0.6, s=50)
        
        if len(self.success_df) > 0:
            max_val = max(self.success_df['phase1_quality'].max(), self.success_df['phase3_quality'].max())
            ax3.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Perfect Transfer')
        
        ax3.set_xlabel('Phase 1: Waist→FP |r|', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Phase 3: Wrist→FP |r|', fontsize=12, fontweight='bold')
        ax3.set_title('(C) Transfer Learning Effectiveness', fontsize=13, fontweight='bold')
        ax3.grid(True, alpha=0.3)
        ax3.legend(fontsize=9, loc='lower right')
        ax3.set_xlim([0, 1])
        ax3.set_ylim([0, 1])
        
        # (D) Trial Count by Activity
        ax4 = plt.subplot(2, 2, 4)
        
        trial_counts = []
        trial_labels = []
        trial_colors = []
        
        for i, activity in enumerate(activity_order):
            count = len(self.success_df[self.success_df['activity'] == activity])
            if count > 0:
                trial_counts.append(count)
                trial_labels.append(activity_labels[i])
                trial_colors.append(activity_colors[i])
        
        bars = ax4.bar(trial_labels, trial_counts, color=trial_colors, 
                      alpha=0.8, edgecolor='black', linewidth=1.5)
        
        for bar, count in zip(bars, trial_counts):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(count)}', ha='center', va='bottom', fontweight='bold')
        
        ax4.set_ylabel('Number of Trials', fontsize=12, fontweight='bold')
        ax4.set_title('(D) Successfully Analyzed Trials', fontsize=13, fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        ax4.tick_params(axis='x', rotation=45)
        
        fig.suptitle('3-Phase Synchronization Analysis - Dataset Overview', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        save_path = PUB_FIG_DIR / 'dataset_overview.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Overview figure saved: {save_path}")
    
    def create_statistical_summary(self):
        """Create statistical summary figure."""
        if len(self.success_df) == 0:
            return
        
        fig = plt.figure(figsize=(16, 12))
        
        activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
        activity_labels = ['Walking', 'Jogging', 'Running', 'Heel Drop', 'Step Drop']
        
        # (A) Mean with 95% CI
        ax1 = plt.subplot(2, 2, 1)
        
        means_p1, cis_p1 = [], []
        means_p3, cis_p3 = [], []
        valid_labels = []
        
        for i, activity in enumerate(activity_order):
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 1:
                mean_p1 = act_data['phase1_corr'].mean()
                sem_p1 = sem(act_data['phase1_corr'].dropna())
                ci_p1 = sem_p1 * t.ppf(0.975, len(act_data) - 1)
                
                mean_p3 = act_data['phase3_corr'].mean()
                sem_p3 = sem(act_data['phase3_corr'].dropna())
                ci_p3 = sem_p3 * t.ppf(0.975, len(act_data) - 1)
                
                means_p1.append(mean_p1)
                cis_p1.append(ci_p1)
                means_p3.append(mean_p3)
                cis_p3.append(ci_p3)
                valid_labels.append(activity_labels[i])
        
        if valid_labels:
            x = np.arange(len(valid_labels))
            width = 0.35
            
            ax1.bar(x - width/2, means_p1, width, yerr=cis_p1,
                   label='Phase 1 (Waist)', color='#3498DB', alpha=0.8, capsize=5)
            ax1.bar(x + width/2, means_p3, width, yerr=cis_p3,
                   label='Phase 3 (Wrist)', color='#9B59B6', alpha=0.8, capsize=5)
            
            ax1.set_ylabel('Mean Correlation r', fontsize=12, fontweight='bold')
            ax1.set_title('(A) Mean Correlation with 95% CI', fontsize=13, fontweight='bold')
            ax1.set_xticks(x)
            ax1.set_xticklabels(valid_labels, rotation=45, ha='right')
            ax1.legend()
            ax1.grid(True, alpha=0.3, axis='y')
        
        # (B) Summary Statistics Table
        ax2 = plt.subplot(2, 2, 2)
        ax2.axis('off')
        
        summary_data = []
        for activity in activity_order:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                summary_data.append([
                    activity_labels[activity_order.index(activity)],
                    len(act_data),
                    f"{act_data['phase1_corr'].mean():.3f}",
                    f"{act_data['phase1_corr'].std():.3f}",
                    f"{act_data['phase3_corr'].mean():.3f}",
                    f"{act_data['phase3_corr'].std():.3f}"
                ])
        
        if summary_data:
            table = ax2.table(cellText=summary_data,
                             colLabels=['Activity', 'N', 'P1 Mean', 'P1 SD', 'P3 Mean', 'P3 SD'],
                             cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
            
            table.auto_set_font_size(False)
            table.set_fontsize(10)
            table.scale(1, 2.5)
            
            for i in range(6):
                table[(0, i)].set_facecolor('#4472C4')
                table[(0, i)].set_text_props(weight='bold', color='white')
        
        ax2.set_title('(B) Summary Statistics', fontsize=13, fontweight='bold', pad=20)
        
        # (C) Distribution Histogram
        ax3 = plt.subplot(2, 2, 3)
        
        phase3_quality = self.success_df['phase3_corr'].abs()
        ax3.hist(phase3_quality, bins=30, alpha=0.7,
                color='#9ECAE1', edgecolor='black', linewidth=1.5)
        
        mean_corr = phase3_quality.mean()
        ax3.axvline(mean_corr, color='red', linestyle='--', linewidth=2.5,
                   label=f'Mean: {mean_corr:.3f}')
        
        ax3.set_xlabel('Correlation |r|', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax3.set_title('(C) Overall Quality Distribution', fontsize=13, fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3, axis='y')
        
        # (D) Deployment Readiness Pie
        ax4 = plt.subplot(2, 2, 4)
        
        phase3_quality = self.success_df['phase3_corr'].abs()
        excellent = len(self.success_df[phase3_quality > 0.7])
        good = len(self.success_df[(phase3_quality > 0.5) & (phase3_quality <= 0.7)])
        moderate = len(self.success_df[(phase3_quality > 0.3) & (phase3_quality <= 0.5)])
        poor = len(self.success_df[phase3_quality <= 0.3])
        
        sizes = [excellent, good, moderate, poor]
        labels = ['Excellent\n(>0.7)', 'Good\n(>0.5)', 'Moderate\n(>0.3)', 'Poor\n(≤0.3)']
        colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']
        
        wedges, texts, autotexts = ax4.pie(sizes, labels=labels, colors=colors_pie,
                                           autopct=lambda p: f'{p:.1f}%\n({int(p*sum(sizes)/100)})',
                                           startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax4.set_title('(D) Deployment Readiness', fontsize=13, fontweight='bold')
        
        fig.suptitle('Statistical Summary - 3-Phase Analysis', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        save_path = PUB_FIG_DIR / 'statistical_summary.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Statistical summary saved: {save_path}")
    
    def print_summary_statistics(self):
        """Print summary statistics to console."""
        if len(self.success_df) == 0:
            print("No successful trials to analyze")
            return
        
        print("\n" + "="*80)
        print("📊 SUMMARY STATISTICS")
        print("="*80)
        
        print(f"\nPhase 1 (Waist→FP):")
        print(f"  Mean r: {self.success_df['phase1_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase1_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase1_corr'].min():.3f}, {self.success_df['phase1_corr'].max():.3f}]")
        
        print(f"\nPhase 2 (Waist→Wrist):")
        print(f"  Mean r: {self.success_df['phase2_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase2_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase2_corr'].min():.3f}, {self.success_df['phase2_corr'].max():.3f}]")
        
        print(f"\nPhase 3 (Wrist→FP):")
        print(f"  Mean r: {self.success_df['phase3_corr'].mean():.3f}")
        print(f"  Std:    {self.success_df['phase3_corr'].std():.3f}")
        print(f"  Range:  [{self.success_df['phase3_corr'].min():.3f}, {self.success_df['phase3_corr'].max():.3f}]")
        
        # By activity
        print(f"\nBy Activity:")
        for activity in ['walking', 'jogging', 'running', 'heel', 'drop']:
            act_data = self.success_df[self.success_df['activity'] == activity]
            if len(act_data) > 0:
                print(f"\n  {activity.capitalize()}:")
                print(f"    Count: {len(act_data)}")
                print(f"    Phase 1 Mean r: {act_data['phase1_corr'].mean():.3f}")
                print(f"    Phase 3 Mean r: {act_data['phase3_corr'].mean():.3f}")
                print(f"    Phase 3 Std:   {act_data['phase3_corr'].std():.3f}")
    
    def create_all_figures(self):
        """Generate all publication figures."""
        print("\n🎨 Creating publication-quality figures...")
        print("="*80)
        
        self.create_manifest()
        self.create_overview_figure()
        self.create_statistical_summary()
        self.print_summary_statistics()
        
        print("\n" + "="*80)
        print("✅ ALL FIGURES GENERATED")
        print("="*80)

# ============================================================================
# MAIN ANALYSIS PIPELINE
# ============================================================================

def run_three_phase_analysis():
    """Run complete 3-phase analysis with corrected trial matching."""
    
    print("\n" + "="*80)
    print("🎯 IMPROVED 3-PHASE SYNCHRONIZATION ANALYSIS")
    print("="*80)
    print(f"Base path: {BASE_PATH}")
    print(f"Output: {OUTPUT_DIR}")
    
    # Check if saved results exist
    results_file = OUTPUT_DIR / "three_phase_results.csv"
    
    if results_file.exists():
        print(f"\n✅ Loading saved results from: {results_file}")
        df = pd.read_csv(results_file)
        print(f"   Loaded {len(df)} trials")
    else:
        print("\n🔍 Running corrected 3-phase analysis...")
        print("   This may take several minutes...")
        
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        results = []
        trial_count = 0
        
        # Scan all participants
        for folder_name in sorted(os.listdir(BASE_PATH)):
            pdir = BASE_PATH / folder_name
            
            if not pdir.is_dir():
                continue
            
            # Extract participant info
            pid, year, height_cm, weight_kg = extract_participant_info(folder_name)
            
            if pid is None or weight_kg is None:
                print(f"⚠️  Skipped {folder_name}: Could not extract participant info")
                continue
            
            print(f"\n👤 {folder_name} (Weight: {weight_kg}kg)")
            
            # List all CSV files
            files = [f for f in os.listdir(pdir) if f.endswith(".csv")]
            
            # Build mapping: (activity, trial_id) -> {waist_file, wrist_file, fp_file}
            trial_map = {}
            
            for filename in files:
                activity, placement, trial_id = extract_trial_info(filename)
                
                if activity is None or trial_id is None:
                    continue
                
                key = (activity, trial_id)
                if key not in trial_map:
                    trial_map[key] = {}
                
                if placement == "Waist":
                    trial_map[key]['waist'] = filename
                elif placement == "Wrist":
                    trial_map[key]['wrist'] = filename
            
            # Now find force plate files (different format)
            # They should match by activity and trial ID
            fp_files = [f for f in files if "_f_" in f.lower()]  # Force plate indicator
            
            for fp_file in fp_files:
                # Try to extract activity from force plate filename
                # They might be named like: walk01_f_X.csv, Drop_FP_Trial_...csv, etc.
                activity = None
                trial_id = None
                
                # Pattern 1: [activity][number]_f_X.csv
                m = re.match(r'^([a-z]+)(\d+)_f_\d+\.csv$', fp_file, re.IGNORECASE)
                if m:
                    act_str = m.group(1).lower()
                    trial_id = int(m.group(2))
                    # Map to standard activity names
                    if 'walk' in act_str:
                        activity = 'walking'
                    elif 'run' in act_str:
                        activity = 'running'
                    elif 'jog' in act_str:
                        activity = 'jogging'
                    elif 'heel' in act_str:
                        activity = 'heel'
                    elif 'drop' in act_str:
                        activity = 'drop'
                
                if activity is None or trial_id is None:
                    continue
                
                key = (activity, trial_id)
                if key in trial_map:
                    trial_map[key]['fp'] = fp_file
            
            # Process each complete trial (has all three files)
            for (activity, trial_id), files_dict in trial_map.items():
                if not all(k in files_dict for k in ['waist', 'wrist', 'fp']):
                    continue
                
                try:
                    # Load data
                    fz, _ = load_force_z_from_csv(pdir / files_dict['fp'])
                    waist_X = load_watch_matrix(pdir / files_dict['waist'])
                    wrist_X = load_watch_matrix(pdir / files_dict['wrist'])
                    
                    # Ensure same length
                    n = min(len(fz), len(waist_X), len(wrist_X))
                    if n < 100:
                        continue
                    
                    fz_short = fz[:n]
                    waist_mag = np.sqrt((waist_X[:n, :3]**2).sum(axis=1))
                    wrist_mag = np.sqrt((wrist_X[:n, :3]**2).sum(axis=1))
                    
                    # Phase 1: Waist → FP
                    p1 = find_optimal_lag_xcorr(fz_short, waist_mag, fs=1000.0)
                    
                    # Phase 2: Waist → Wrist
                    p2 = find_optimal_lag_xcorr(waist_mag, wrist_mag, fs=100.0)
                    
                    # Phase 3: Wrist → FP
                    p3 = find_optimal_lag_xcorr(fz_short, wrist_mag, fs=1000.0)
                    
                    result = {
                        'participant': folder_name,
                        'activity': activity,
                        'trial_num': trial_id,
                        'phase1_corr': p1['pearson_r'],
                        'phase2_corr': p2['pearson_r'],
                        'phase3_corr': p3['pearson_r'],
                        'status': 'success'
                    }
                    
                    results.append(result)
                    trial_count += 1
                    
                    print(f"  ✅ {activity:8s} (trial {trial_id:2d}): "
                          f"P1={p1['pearson_r']:.3f}, P2={p2['pearson_r']:.3f}, P3={p3['pearson_r']:.3f}")
                    
                    if trial_count % 50 == 0:
                        print(f"     [{trial_count} trials processed]")
                
                except Exception as e:
                    print(f"  ❌ {activity} trial {trial_id}: {str(e)[:60]}")
                    continue
        
        # Save results
        df = pd.DataFrame(results)
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        df.to_csv(results_file, index=False)
        
        print(f"\n✅ Analysis complete: {trial_count} trials")
        print(f"   Saved to: {results_file}")
    
    return df

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution."""
    
    # Run analysis
    df = run_three_phase_analysis()
    
    if df is not None and len(df) > 0:
        # Create analyzer and generate all figures
        print("\n" + "="*80)
        print("🎨 GENERATING PUBLICATION FIGURES")
        print("="*80)
        
        analyzer = CorrectedResultsAnalyzer(df)
        analyzer.create_all_figures()
        
        # Final summary
        print("\n" + "="*80)
        print("✅ COMPLETE 3-PHASE ANALYSIS FINISHED!")
        print("="*80)
        print(f"\n📁 Output Directory: {OUTPUT_DIR}")
        print(f"📁 Publication Figures: {PUB_FIG_DIR}")
        print(f"\n📊 Generated Files:")
        print(f"   • three_phase_results.csv")
        print(f"   • publication_figures/")
        print(f"     ├── dataset_overview.png")
        print(f"     ├── statistical_summary.png")
        print(f"     └── trial_manifest.csv")
        
    else:
        print("\n❌ Analysis failed - no valid trials found!")

if __name__ == "__main__":
    main()


🎯 IMPROVED 3-PHASE SYNCHRONIZATION ANALYSIS
Base path: /kaggle/input/dataset10/Dataset
Output: improved_sync_analysis

✅ Loading saved results from: improved_sync_analysis/three_phase_results.csv
   Loaded 579 trials

🎨 GENERATING PUBLICATION FIGURES

📊 Analyzer initialized with 579 successful trials

🎨 Creating publication-quality figures...

📄 Manifest saved: improved_sync_analysis/publication_figures/trial_manifest.csv
   Total trials: 579
   Participants: 10
✅ Overview figure saved: improved_sync_analysis/publication_figures/dataset_overview.png
✅ Statistical summary saved: improved_sync_analysis/publication_figures/statistical_summary.png

📊 SUMMARY STATISTICS

Phase 1 (Waist→FP):
  Mean r: 0.463
  Std:    0.347
  Range:  [-0.602, 0.838]

Phase 2 (Waist→Wrist):
  Mean r: 0.211
  Std:    0.301
  Range:  [-0.486, 0.830]

Phase 3 (Wrist→FP):
  Mean r: -0.002
  Std:    0.589
  Range:  [-0.800, 0.872]

By Activity:

  Walking:
    Count: 182
    Phase 1 Mean r: 0.528
    Phase 3 Mean 

In [6]:
#!/usr/bin/env python3
"""
================================================================================
SCIENTIFIC 3-PHASE ANALYSIS - FIXED FOR YOUR CSV STRUCTURE
Uses phase1_corr_pearson, phase2_corr_pearson, phase3_corr_pearson
================================================================================
"""

import warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

RESULTS_FILE = Path("/kaggle/working/improved_sync_analysis/three_phase_results.csv")#scientific_sync_analysis/three_phase_results.csv")
OUTPUT_DIR = Path("scientific_sync_analysis")
PUB_FIG_DIR = OUTPUT_DIR / "publication_figures"

PUB_FIG_DIR.mkdir(parents=True, exist_ok=True)

# ============================================================================
# LOAD AND PREPARE DATA
# ============================================================================

print("\n" + "="*80)
print("🎯 SCIENTIFIC 3-PHASE SYNCHRONIZATION ANALYSIS")
print("="*80)
print(f"Loading: {RESULTS_FILE}\n")

if not RESULTS_FILE.exists():
    print(f"❌ Results file not found: {RESULTS_FILE}")
    exit(1)

# Load data
df = pd.read_csv(RESULTS_FILE)
print(f"✅ Loaded {len(df)} trials")

# Filter successful trials
success_df = df[df['status'] == 'success'].copy()
print(f"✅ {len(success_df)} successful trials\n")

# Map columns to standard names (using pearson correlation)
success_df['phase1_corr'] = success_df['phase1_corr_pearson']
success_df['phase2_corr'] = success_df['phase2_corr_pearson']
success_df['phase3_corr'] = success_df['phase3_corr_pearson']

# Add quality metrics
success_df['phase1_quality'] = success_df['phase1_corr'].abs()
success_df['phase2_quality'] = success_df['phase2_corr'].abs()
success_df['phase3_quality'] = success_df['phase3_corr'].abs()

# ============================================================================
# FIGURE 1: THREE-PHASE COMPARISON
# ============================================================================

print("🎨 Creating Figure 1: Three-Phase Comparison...")

fig = plt.figure(figsize=(18, 12))

# (A) Correlation by phase (box plot)
ax1 = plt.subplot(2, 3, 1)
phases_data = [
    success_df['phase1_quality'].values,
    success_df['phase2_quality'].values,
    success_df['phase3_quality'].values
]
phases_labels = ['Phase 1\n(Waist→FP)', 'Phase 2\n(Wrist→Waist)', 'Phase 3\n(Wrist→FP)']
colors_phase = ['#00A878', '#FFB627', '#FF6B35']

bp = ax1.boxplot(phases_data, labels=phases_labels, patch_artist=True,
                widths=0.6, showfliers=True,
                medianprops=dict(color='black', linewidth=2),
                boxprops=dict(linewidth=1.5))

for patch, color in zip(bp['boxes'], colors_phase):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax1.set_ylabel('|Correlation r|', fontsize=12, fontweight='bold')
ax1.set_title('(A) Correlation Strength Across Phases', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')
ax1.axhline(0.7, color='green', linestyle='--', alpha=0.5, linewidth=2, label='Excellent (>0.7)')
ax1.axhline(0.5, color='orange', linestyle='--', alpha=0.5, linewidth=2, label='Good (>0.5)')
ax1.legend(fontsize=9)

# (B) Lag comparison
ax2 = plt.subplot(2, 3, 2)
lags_data = [
    success_df['phase1_lag_ms'].abs().values,
    success_df['phase2_lag_ms'].abs().values,
    success_df['phase3_lag_ms'].abs().values
]

bp2 = ax2.boxplot(lags_data, labels=phases_labels, patch_artist=True,
                 widths=0.6, showfliers=True,
                 medianprops=dict(color='black', linewidth=2),
                 boxprops=dict(linewidth=1.5))

for patch, color in zip(bp2['boxes'], colors_phase):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax2.set_ylabel('|Lag| (ms)', fontsize=12, fontweight='bold')
ax2.set_title('(B) Time Lag Across Phases', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# (C) Phase 1 vs Phase 3 scatter
ax3 = plt.subplot(2, 3, 3)
activity_order = ['walking', 'jogging', 'running', 'heel', 'drop']
activity_colors = ['#6BAED6', '#FC8D62', '#FDB462', '#B3B3E6', '#8DD3C7']

for i, activity in enumerate(activity_order):
    act_data = success_df[success_df['activity'] == activity]
    if len(act_data) > 0:
        ax3.scatter(act_data['phase1_quality'], act_data['phase3_quality'],
                   c=activity_colors[i], label=activity.capitalize(),
                   alpha=0.6, s=80)

max_val = max(success_df['phase1_quality'].max(), success_df['phase3_quality'].max())
ax3.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, linewidth=2, label='Perfect transfer')

ax3.set_xlabel('Phase 1: |r| (Waist→FP)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Phase 3: |r| (Wrist→FP)', fontsize=12, fontweight='bold')
ax3.set_title('(C) Transfer Learning Effectiveness', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.legend(fontsize=9, loc='lower right')
ax3.set_xlim([0, 1])
ax3.set_ylim([0, 1])

# (D) By activity bars
ax4 = plt.subplot(2, 3, 4)
activities = sorted(success_df['activity'].unique())
x = np.arange(len(activities))
width = 0.25

p1_means = [success_df[success_df['activity'] == act]['phase1_quality'].mean() for act in activities]
p2_means = [success_df[success_df['activity'] == act]['phase2_quality'].mean() for act in activities]
p3_means = [success_df[success_df['activity'] == act]['phase3_quality'].mean() for act in activities]

ax4.bar(x - width, p1_means, width, label='Phase 1', color='#00A878', alpha=0.8, edgecolor='black', linewidth=1.5)
ax4.bar(x, p2_means, width, label='Phase 2', color='#FFB627', alpha=0.8, edgecolor='black', linewidth=1.5)
ax4.bar(x + width, p3_means, width, label='Phase 3', color='#FF6B35', alpha=0.8, edgecolor='black', linewidth=1.5)

ax4.set_ylabel('Mean |Correlation|', fontsize=12, fontweight='bold')
ax4.set_title('(D) Performance by Activity', fontsize=13, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels([a.capitalize() for a in activities], rotation=45, ha='right')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3, axis='y')

# (E) Transfer learning gap
ax5 = plt.subplot(2, 3, 5)
success_df['transfer_gap'] = (success_df['phase1_quality'] + success_df['phase2_quality']) / 2 - success_df['phase3_quality']

ax5.hist(success_df['transfer_gap'], bins=30, alpha=0.7, color='#6C63FF', edgecolor='black', linewidth=1.5)
mean_gap = success_df['transfer_gap'].mean()
ax5.axvline(mean_gap, color='red', linestyle='--', linewidth=2.5, label=f'Mean gap: {mean_gap:.3f}')

ax5.set_xlabel('Transfer Gap (P1+P2)/2 - P3', fontsize=12, fontweight='bold')
ax5.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax5.set_title('(E) Transfer Learning Gap Distribution', fontsize=13, fontweight='bold')
ax5.legend(fontsize=10)
ax5.grid(True, alpha=0.3, axis='y')

# (F) Deployment readiness pie
ax6 = plt.subplot(2, 3, 6)
excellent = len(success_df[success_df['phase3_quality'] > 0.7])
good = len(success_df[(success_df['phase3_quality'] > 0.5) & (success_df['phase3_quality'] <= 0.7)])
moderate = len(success_df[(success_df['phase3_quality'] > 0.3) & (success_df['phase3_quality'] <= 0.5)])
poor = len(success_df[success_df['phase3_quality'] <= 0.3])

sizes = [excellent, good, moderate, poor]
labels = ['Excellent\n(>0.7)', 'Good\n(>0.5)', 'Moderate\n(>0.3)', 'Poor\n(≤0.3)']
colors_pie = ['#2ECC71', '#F39C12', '#E67E22', '#E74C3C']

wedges, texts, autotexts = ax6.pie(sizes, labels=labels, colors=colors_pie,
                                   autopct=lambda p: f'{p:.1f}%\n({int(p*sum(sizes)/100)})',
                                   startangle=90)

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

ax6.set_title('(F) Phase 3 Deployment Readiness', fontsize=13, fontweight='bold')

plt.suptitle('Scientific 3-Phase Synchronization Analysis - Comprehensive Comparison',
            fontsize=16, fontweight='bold', y=0.995)

plt.tight_layout(rect=[0, 0, 1, 0.99])

save_path = PUB_FIG_DIR / 'three_phase_analysis.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()

print(f"✅ Figure 1 saved: {save_path}\n")

# ============================================================================
# FIGURE 2: CORRELATION HEATMAPS
# ============================================================================

print("🎨 Creating Figure 2: Correlation Heatmaps...")

fig, axes = plt.subplots(1, 3, figsize=(16, 6))

for idx, (phase, col) in enumerate([('Phase 1', 'phase1_corr'), 
                                     ('Phase 2', 'phase2_corr'), 
                                     ('Phase 3', 'phase3_corr')]):
    try:
        pivot_data = success_df.pivot_table(
            values=col, 
            index='participant', 
            columns='activity', 
            aggfunc='mean'
        )
        
        sns.heatmap(pivot_data, annot=True, fmt='.2f', cmap='RdYlGn', 
                   center=0, vmin=-1, vmax=1, ax=axes[idx],
                   cbar_kws={'label': 'Correlation r'})
        
        axes[idx].set_title(f'{phase} Correlations (Waist→FP, Wrist→Waist, Wrist→FP)', 
                           fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Activity', fontsize=11, fontweight='bold')
        axes[idx].set_ylabel('Participant', fontsize=11, fontweight='bold')
    except Exception as e:
        print(f"  Note: Could not create heatmap for {phase}: {e}")

plt.tight_layout()
save_path = PUB_FIG_DIR / 'correlation_heatmaps.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()

print(f"✅ Figure 2 saved: {save_path}\n")

# ============================================================================
# FIGURE 3: PHASE PROGRESSION
# ============================================================================

print("🎨 Creating Figure 3: Phase Progression...")

fig, ax = plt.subplots(figsize=(12, 8))

phases = ['Phase 1\n(Waist→FP)', 'Phase 2\n(Wrist→Waist)', 'Phase 3\n(Wrist→FP)']

for activity in sorted(success_df['activity'].unique()):
    act_data = success_df[success_df['activity'] == activity]
    means = [
        act_data['phase1_corr'].abs().mean(),
        act_data['phase2_corr'].abs().mean(),
        act_data['phase3_corr'].abs().mean()
    ]
    ax.plot(phases, means, marker='o', linewidth=2.5, markersize=10, 
           label=activity.capitalize(), alpha=0.8)

ax.axhline(0.7, color='green', linestyle='--', alpha=0.5, linewidth=2, label='Excellent threshold')
ax.axhline(0.5, color='orange', linestyle='--', alpha=0.5, linewidth=2, label='Good threshold')

ax.set_ylabel('Mean |Correlation|', fontsize=12, fontweight='bold')
ax.set_title('3-Phase Performance Progression by Activity', fontsize=13, fontweight='bold')
ax.legend(fontsize=11, loc='best')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 1])

plt.tight_layout()
save_path = PUB_FIG_DIR / 'phase_progression.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()

print(f"✅ Figure 3 saved: {save_path}\n")

# ============================================================================
# DETAILED STATISTICS
# ============================================================================

print("\n" + "="*80)
print("📊 COMPREHENSIVE 3-PHASE STATISTICS")
print("="*80)

print(f"\n✅ Dataset Summary:")
print(f"   Total trials: {len(success_df)}")
print(f"   Participants: {success_df['participant'].nunique()}")
print(f"   Activities: {success_df['activity'].nunique()}")

print(f"\n🟢 PHASE 1: Waist → Force Plate (Reference Ground Truth)")
print(f"   Mean r:     {success_df['phase1_corr'].mean():.3f} ± {success_df['phase1_corr'].std():.3f}")
print(f"   Median r:   {success_df['phase1_corr'].median():.3f}")
print(f"   Range:      [{success_df['phase1_corr'].min():.3f}, {success_df['phase1_corr'].max():.3f}]")
print(f"   Median lag: {success_df['phase1_lag_ms'].median():.1f} ms")

print(f"\n🟡 PHASE 2: Wrist → Waist (Transfer Learning)")
print(f"   Mean r:     {success_df['phase2_corr'].mean():.3f} ± {success_df['phase2_corr'].std():.3f}")
print(f"   Median r:   {success_df['phase2_corr'].median():.3f}")
print(f"   Range:      [{success_df['phase2_corr'].min():.3f}, {success_df['phase2_corr'].max():.3f}]")
print(f"   Median lag: {success_df['phase2_lag_ms'].median():.1f} ms")

print(f"\n🔴 PHASE 3: Wrist → Force Plate (Deployment)")
print(f"   Mean r:     {success_df['phase3_corr'].mean():.3f} ± {success_df['phase3_corr'].std():.3f}")
print(f"   Median r:   {success_df['phase3_corr'].median():.3f}")
print(f"   Range:      [{success_df['phase3_corr'].min():.3f}, {success_df['phase3_corr'].max():.3f}]")
print(f"   Median lag: {success_df['phase3_lag_ms'].median():.1f} ms")

print(f"\n📈 TRANSFER LEARNING ANALYSIS:")
print(f"   P1 Mean:              {success_df['phase1_quality'].mean():.3f}")
print(f"   P2 Mean:              {success_df['phase2_quality'].mean():.3f}")
print(f"   P3 Mean:              {success_df['phase3_quality'].mean():.3f}")

transfer_gap = ((success_df['phase1_quality'] + success_df['phase2_quality']) / 2 - success_df['phase3_quality']).mean()
print(f"   Transfer Gap (avg):   {transfer_gap:.3f}")

if transfer_gap < 0.1:
    print(f"   ✓ EXCELLENT transfer learning (gap < 0.1)")
elif transfer_gap < 0.2:
    print(f"   ✓ GOOD transfer learning (gap < 0.2)")
else:
    print(f"   ⚠ Transfer learning needs improvement (gap = {transfer_gap:.3f})")

print(f"\n   Quality Comparison:")
p1_excellent = len(success_df[success_df['phase1_quality'] > 0.7])
p3_excellent = len(success_df[success_df['phase3_quality'] > 0.7])
print(f"   Phase 1 Excellent (r>0.7): {p1_excellent} ({100*p1_excellent/len(success_df):.1f}%)")
print(f"   Phase 3 Excellent (r>0.7): {p3_excellent} ({100*p3_excellent/len(success_df):.1f}%)")

deployment_feasible = p3_excellent > len(success_df) * 0.5
print(f"   Deployment Feasibility:    {'✓ READY FOR DEPLOYMENT' if deployment_feasible else '⚠ Needs Optimization'}")

print(f"\n📊 BY ACTIVITY:")
for activity in sorted(success_df['activity'].unique()):
    act_data = success_df[success_df['activity'] == activity]
    print(f"\n   {activity.upper()} (n={len(act_data)}):")
    print(f"     Phase 1 (Waist→FP):    r={act_data['phase1_corr'].mean():.3f} ± {act_data['phase1_corr'].std():.3f}")
    print(f"     Phase 2 (Wrist→Waist): r={act_data['phase2_corr'].mean():.3f} ± {act_data['phase2_corr'].std():.3f}")
    print(f"     Phase 3 (Wrist→FP):    r={act_data['phase3_corr'].mean():.3f} ± {act_data['phase3_corr'].std():.3f}")
    
    gap = ((act_data['phase1_quality'] + act_data['phase2_quality']) / 2 - act_data['phase3_quality']).mean()
    print(f"     Transfer Gap:          {gap:.3f}")

# ============================================================================
# QUALITY TIERS
# ============================================================================

print(f"\n🎯 DEPLOYMENT READINESS BREAKDOWN (Phase 3):")
excellent = len(success_df[success_df['phase3_quality'] > 0.7])
good = len(success_df[(success_df['phase3_quality'] > 0.5) & (success_df['phase3_quality'] <= 0.7)])
moderate = len(success_df[(success_df['phase3_quality'] > 0.3) & (success_df['phase3_quality'] <= 0.5)])
poor = len(success_df[success_df['phase3_quality'] <= 0.3])

print(f"   🟢 Excellent (r > 0.7):  {excellent} trials ({100*excellent/len(success_df):.1f}%)")
print(f"   🟡 Good (r > 0.5):       {good} trials ({100*good/len(success_df):.1f}%)")
print(f"   🟠 Moderate (r > 0.3):   {moderate} trials ({100*moderate/len(success_df):.1f}%)")
print(f"   🔴 Poor (r ≤ 0.3):       {poor} trials ({100*poor/len(success_df):.1f}%)")

print(f"\n" + "="*80)
print("✅ SCIENTIFIC 3-PHASE ANALYSIS COMPLETE!")
print("="*80)
print(f"\n📁 Output Directory: {PUB_FIG_DIR}")
print(f"\n📊 Generated Files:")
print(f"   • three_phase_analysis.png (6-panel comparison)")
print(f"   • correlation_heatmaps.png (by participant & activity)")
print(f"   • phase_progression.png (line plot by activity)")
print(f"\n💡 KEY INSIGHTS:")
print(f"   • Phase 1 shows theoretical best (waist at center of mass)")
print(f"   • Phase 2 shows transfer potential (wrist approximates waist)")
print(f"   • Phase 3 shows real-world feasibility (wrist-only deployment)")
print(f"   • Gap between P1 and P3 = room for optimization")
print("\n")


🎯 SCIENTIFIC 3-PHASE SYNCHRONIZATION ANALYSIS
Loading: /kaggle/working/improved_sync_analysis/three_phase_results.csv

✅ Loaded 579 trials
✅ 579 successful trials



KeyError: 'phase1_corr_pearson'